In [40]:
# --------------------------------------------------
# Cell 0: 【请在这里手动修改本期，例如 '202506'】
# --------------------------------------------------
period = "202506"

print(f"本期报告期数已设置为: {period}")

本期报告期数已设置为: 202506


In [41]:
# --------------------------------------------------
# Cell 1: 导入库并设置项目结构
# --------------------------------------------------
import math
import os
import re
import sys
import warnings
from datetime import datetime
from pathlib import Path
from time import perf_counter

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("库导入完成。")

# --- 项目路径设置  ---
# .
# ├── 1_data_preprocess.ipynb
# └── 报告数据/
#     ├── 输入/
#     │   ├── 安监数据/              (存放老师给的原始Excel文件；10家公司，10个文件)
#     │   └── basic_data.xlsx          (城市信息、线路信息)
#     ├── 输出/                      (存放所有最终生成的报告)
#     ├── temp/
#     │   ├── 1_待上传猪猪云数据/        (需要逐个手动上传到猪猪云的文件；8家公司，16个文件，排除顺丰和中通)
#     │   ├── 2_猪猪云下载数据/          (【手动放入】存放从猪猪云下载的结果文件；8家公司，16个文件，排除顺丰和中通)
#     │   ├── 3_猪猪云合并数据/         （猪猪云下载数据按公司合并后数据；8家公司，8个文件，排除顺丰和中通）
#     │   ├── 4_logistics数据         （存放logistics数据——提取完整物流信息的时间戳后的数据；8家公司，8个文件，排除顺丰和中通）
#     └── └── 5_中转数据/               (存放中转数据——提取中转城市和平均中转次数后的数据；8家公司，8个文件，排除顺丰和中通)
# 根目录
base_path = Path.cwd()
report_path = base_path / "报告数据"
# 输入路径
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
# 输出路径
output_path = report_path / "输出"
# 中间过程文件路径（自动创建，用于存放临时文件）
temp_path = report_path / "temp"
upload_split_path = temp_path / "1_待上传猪猪云文件"  # 存放拆分后待上传的文件
zhuzhuyun_download_path = temp_path / "2_猪猪云下载数据"  # 关键：这是手动放置文件的目录
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"
transit_data_path = temp_path / "5_中转数据"

# 创建所有需要的文件夹
for p in [
    report_path,
    input_path,
    anjian_data_path,
    zhuzhuyun_download_path,
    zhuzhuyun_merge_path,
    transit_data_path,
    output_path,
    temp_path,
    upload_split_path,
    pycharm_input_path,
]:
    p.mkdir(exist_ok=True)

print("项目文件夹结构设置/检查完毕。请按以下结构组织文件：")
print(
    f"将10个安监数据Excel文件放入: {anjian_data_path}\n⚠️ 请修改“京邦达”的名字为“京东；\n⚠️ 请修改“中国邮政”的名字为“邮政”"
)
# 计算口径问题
# 1.所有的均值、最优值、排名均不计算快包（和EMS本质是一家公司的不同产品）
# 2.线路的送达天数计算使用的是80分位数
# 3.线路层的均值不加权（计算全部线路均值，需要用∑（单一线路均值*线路快递数量）/快递总数量，或直接对全部线路求均值不延续适用线路均值；但行业层面可直接对公司层面的均值（不包括快包），直接求简单的算术平均（并非全样本真实均值、只是简单对各公司水平求均值）
# 4.“单一数据源原则”：logistics经过
### (data["寄出地处理时限"].between(0.1，48))
### (data["运输时限"].between(0.5，200))&(data["寄达地处理时限"].between(0.1，60))&
### (data[“投递时限"].between(0，36))]
### 筛选后得到data_analysis_result；后续的数据只能够基于data_analysis_result进一步计算，不能再重新回到 logistics 数据
# 5.“48小时妥投率”口径：经过筛选后的数据（即data_analysis_result）的基础上，仅使用“Top 30城市互寄”的数据来计算；而“72小时妥投率”直接用data_analysis_result数据，不经过“Top 30城市“筛选
# 6.邮政月报.xlsx中的“分城市明细”和“分省份明细”sheet中
### 全程时限（双挂）是该城市按照所有以该城市为寄出地和寄达地的邮件进行聚合
### 全程时限（寄出地）是对应的城市按照所有以该城市为寄出地的邮件进行聚合
### 寄出地处理时限、揽收-到达寄出地分拣中心时长、到达寄出地分拣中心-离开寄出地城市时长是按照所有该城市作为寄出地的邮件聚合
### 寄达地处理时限、到达寄达地城市-离开寄达地分拣中心时长、离开寄达地分拣中心-派件是按照所有该城市作为寄达地的邮件聚合
### 投递时限是按照所有该城市作为寄达地的邮件聚合
### 寄出地处理时限+寄达地处理时限+投递时限是前面的简单加和，即寄出地处理时限（按寄出地聚合）+寄达地处理时限（按寄达地聚合）+投递时限（按寄达地聚合）

库导入完成。
项目文件夹结构设置/检查完毕。请按以下结构组织文件：
将10个安监数据Excel文件放入: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输入/安监数据
⚠️ 请修改“京邦达”的名字为“京东；
⚠️ 请修改“中国邮政”的名字为“邮政”


In [42]:
# --------------------------------------------------
# Cell 2: 拆分安监数据，准备用于猪猪云上传的数据
# --------------------------------------------------
def prepare_data_for_upload(
    source_dir: Path,
    target_dir: Path,
    companies_to_exclude: list,
    period: str,
    chunk_size: int = 50000,
):
    """
    根据指定`period`精确查找对应的安监数据，保留'企业'和'单号'列，
    并按chunk_size动态拆分为n个文件，为手动上传做准备。
    """
    import math
    from pathlib import Path

    import pandas as pd

    print(">>> 第1部分：开始准备上传数据...")
    target_dir.mkdir(parents=True, exist_ok=True)
    for f in target_dir.glob("*.xlsx"):
        f.unlink()

    # --- 文件查找精准匹配 ---
    # 1. 从 period (例如 '202507') 中解析出年份和月份
    year = period[:4]
    month = int(period[4:])  # 使用 int() 去掉可能存在的前导零

    # 2. 定义要处理的公司列表
    companies = [
        "EMS",
        "中通",
        "京东",
        "圆通",
        "德邦",
        "极兔",
        "申通",
        "韵达",
        "邮政",
        "顺丰",
    ]

    # 3. 不再遍历文件夹里的所有文件，而是遍历公司列表，主动构造文件名进行精确查找
    for company_name in companies:
        if company_name in companies_to_exclude:
            print(f"公司 '{company_name}' 在排除列表内，跳过准备上传。")
            continue

        # 4. 根据 period 和公司名，构建期望的精确文件名
        target_anjian_filename = f"{year}年{month}月{company_name}抽样.xlsx"
        file_path = source_dir / target_anjian_filename

        # 为了兼容性，也检查 .xls 后缀
        target_anjian_filename_xls = target_anjian_filename.replace(".xlsx", ".xls")
        file_path_xls = source_dir / target_anjian_filename_xls

        # 5. 精确检查文件是否存在
        if not file_path.exists() and not file_path_xls.exists():
            print(
                f"未找到 {company_name} 的本期安监文件: '{target_anjian_filename}' 或 '.xls'，已跳过。"
            )
            continue

        # 如果.xlsx不存在但.xls存在，则使用.xls的路径
        if not file_path.exists():
            file_path = file_path_xls

        print(f"正在处理精确匹配的文件: {file_path.name} (公司: {company_name})")
        # --- 精准匹配逻辑修改结束 ---

        # 读取并校验Excel文件
        try:
            df = pd.read_excel(file_path, dtype={"单号": str})
            if "企业" not in df.columns or "单号" not in df.columns:
                print(
                    f"  -> 警告: 文件 {file_path.name} 缺少 '企业' 或 '单号' 列，已跳过。"
                )
                continue
            df_to_upload = df[["企业", "单号"]].copy()
        except Exception as e:
            print(f"  -> 错误: 读取文件 {file_path.name} 时出错: {e}，已跳过。")
            continue

        # --- 动态拆分文件的逻辑保持不变 ---
        total_rows = len(df_to_upload)
        if total_rows == 0:
            print(f"  -> {company_name} 数据为空，跳过保存。")
            continue

        num_chunks = math.ceil(total_rows / chunk_size)

        for i in range(num_chunks):
            start_row = i * chunk_size
            end_row = start_row + chunk_size
            chunk_df = df_to_upload.iloc[start_row:end_row]

            # 输出文件名已符合原则1，无需改动
            output_filename = f"{company_name}_{period}_{i + 1}.xlsx"
            output_filepath = target_dir / output_filename
            chunk_df.to_excel(output_filepath, index=False)

        print(f"  -> {company_name} 数据已拆分为 {num_chunks} 个文件。")

    print("\n" + "=" * 100)
    print("【第一部分完成】数据准备完毕！")
    print(f"请前往文件夹: \n{target_dir.resolve()}")
    print(
        "将里面的所有Excel文件手动上传到“猪猪快递云”网站，\n\n最后将所有下载结果放入“/报告数据/temp/2_猪猪云下载数据/”文件夹中。"
    )
    print("=" * 100)


# 定义需要排除的公司列表
companies_to_exclude_from_upload = ["顺丰", "中通"]
# 调用函数的代码无需修改
prepare_data_for_upload(
    anjian_data_path, upload_split_path, companies_to_exclude_from_upload, period
)

>>> 第1部分：开始准备上传数据...
正在处理精确匹配的文件: 2025年6月EMS抽样.xlsx (公司: EMS)
  -> EMS 数据已拆分为 1 个文件。
公司 '中通' 在排除列表内，跳过准备上传。
正在处理精确匹配的文件: 2025年6月京东抽样.xlsx (公司: 京东)
  -> 京东 数据已拆分为 2 个文件。
正在处理精确匹配的文件: 2025年6月圆通抽样.xlsx (公司: 圆通)
  -> 圆通 数据已拆分为 2 个文件。
正在处理精确匹配的文件: 2025年6月德邦抽样.xlsx (公司: 德邦)
  -> 德邦 数据已拆分为 2 个文件。
正在处理精确匹配的文件: 2025年6月极兔抽样.xlsx (公司: 极兔)
  -> 极兔 数据已拆分为 2 个文件。
正在处理精确匹配的文件: 2025年6月申通抽样.xlsx (公司: 申通)
  -> 申通 数据已拆分为 2 个文件。
正在处理精确匹配的文件: 2025年6月韵达抽样.xlsx (公司: 韵达)
  -> 韵达 数据已拆分为 2 个文件。
正在处理精确匹配的文件: 2025年6月邮政抽样.xlsx (公司: 邮政)
  -> 邮政 数据已拆分为 2 个文件。
公司 '顺丰' 在排除列表内，跳过准备上传。

【第一部分完成】数据准备完毕！
请前往文件夹: 
/Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/temp/1_待上传猪猪云文件
将里面的所有Excel文件手动上传到“猪猪快递云”网站，

最后将所有下载结果放入“/报告数据/temp/2_猪猪云下载数据/”文件夹中。


In [43]:
# --------------------------------------------------
# Cell 3: 合并猪猪云下载数据，生成猪猪云合并数据
# --------------------------------------------------
import pandas as pd

# 定义要处理的公司列表
companies = [
    "EMS",
    "中通",
    "京东",
    "圆通",
    "德邦",
    "极兔",
    "申通",
    "韵达",
    "邮政",
    "顺丰",
]

companies_lower = [comp.lower() for comp in companies]
company_files = {}

# 关键：请确保手动从猪猪云下载的文件，放入 "2_猪猪云下载数据" 文件夹
print(f"开始从文件夹 <{zhuzhuyun_download_path.resolve()}> 读取猪猪云下载数据...")
for file_path in zhuzhuyun_download_path.iterdir():
    if file_path.is_file() and file_path.suffix == ".xlsx":
        filename_stem = file_path.stem
        filename_stem_lower = filename_stem.lower()

        # 检查文件名是否包含当前处理的 period，如果不包含则跳过
        if period not in filename_stem:
            print(
                f"  -> 文件 '{file_path.name}' 的周期与当前 ({period}) 不符，已跳过。"
            )
            continue

        company_name_found = False
        found_company = None
        for i, company_lower in enumerate(companies_lower):
            if company_lower in filename_stem_lower:
                found_company = companies[i]
                company_name_found = True
                break

        if company_name_found:
            if found_company not in company_files:
                company_files[found_company] = []
            company_files[found_company].append(file_path)
        else:
            print(
                f"  警告: 文件 '{file_path.name}' 未匹配到任何已知公司名，将跳过此文件。"
            )

print("\n开始处理各公司文件...")

for company_name, files_list in company_files.items():
    print(f"\n正在合并 {company_name} 的文件 ({len(files_list)} 个)...")

    all_dfs = []

    for file_path in files_list:
        try:
            df = pd.read_excel(file_path)
            all_dfs.append(df)
            print(f"  已读取: {file_path.name}")
        except Exception as e:
            print(f"  读取文件失败: {file_path.name}, 错误: {e}")

    if all_dfs:
        merged_df = pd.concat(all_dfs, ignore_index=True)
        # 输出文件名已符合要求，无需改动
        output_filename = f"{company_name}_{period}.xlsx"
        output_file_path = zhuzhuyun_merge_path / output_filename

        if "快递单号" in merged_df.columns:
            rows_before_dedup = len(merged_df)
            merged_df.drop_duplicates(subset=["快递单号"], keep="first", inplace=True)
            rows_after_dedup = len(merged_df)
            num_duplicates_removed = rows_before_dedup - rows_after_dedup
            if num_duplicates_removed > 0:
                print(f"  已基于'快递单号'移除 {num_duplicates_removed} 个重复值。")
            else:
                print(f"  未发现'快递单号'的重复值。")
        else:
            print(f"  警告: 合并后的数据中未找到 '快递单号' 列，无法执行去重操作。")

        try:
            merged_df.to_excel(output_file_path, index=False)
            print(
                f"  成功保存合并数据到: {output_file_path.name} (共 {len(merged_df)} 行)"
            )
        except Exception as e:
            print(f"  保存文件失败: {output_file_path.name}, 错误: {e}")

    else:
        print(f"  没有成功读取 {company_name} 的任何文件，跳过合并。")

print(f"\n所有公司文件合并完成！合并后的文件已存入 <{zhuzhuyun_merge_path.resolve()}>")

开始从文件夹 </Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/temp/2_猪猪云下载数据> 读取猪猪云下载数据...

开始处理各公司文件...

正在合并 申通 的文件 (2 个)...
  已读取: 申通_202506_1.xlsx
  已读取: 申通_202506_2.xlsx
  未发现'快递单号'的重复值。
  成功保存合并数据到: 申通_202506.xlsx (共 83029 行)

正在合并 京东 的文件 (2 个)...
  已读取: 京东_202506_2.xlsx
  已读取: 京东_202506_1.xlsx
  未发现'快递单号'的重复值。
  成功保存合并数据到: 京东_202506.xlsx (共 82770 行)

正在合并 圆通 的文件 (2 个)...
  已读取: 圆通_202506_1.xlsx
  已读取: 圆通_202506_2.xlsx
  未发现'快递单号'的重复值。
  成功保存合并数据到: 圆通_202506.xlsx (共 82733 行)

正在合并 德邦 的文件 (2 个)...
  已读取: 德邦_202506_2.xlsx
  已读取: 德邦_202506_1.xlsx
  未发现'快递单号'的重复值。
  成功保存合并数据到: 德邦_202506.xlsx (共 78551 行)

正在合并 邮政 的文件 (2 个)...
  已读取: 邮政_202506_2.xlsx
  已读取: 邮政_202506_1.xlsx
  未发现'快递单号'的重复值。
  成功保存合并数据到: 邮政_202506.xlsx (共 81571 行)

正在合并 EMS 的文件 (1 个)...
  已读取: EMS_202506_1.xlsx
  未发现'快递单号'的重复值。
  成功保存合并数据到: EMS_202506.xlsx (共 43390 行)

正在合并 极兔 的文件 (2 个)...
  已读取: 极兔_202506_2.xlsx
  已读取: 极兔_202506_1.xlsx
  未发现'快递单号'的重复值。
  成功保存合并数据到: 极兔_202506.xlsx (共 81703 行)

正在合并 韵达 的文件 (2 个).

In [ ]:
#####-------- logistics数据提取原则-------------#####
# 1. “到达分拣中心时间”本质是“到达寄件城市分拣中心时间”
#### 寄达城市（即收件城市）分拣中心时间以提取到的第一个分拣中心为准
#### 寄出城市（即寄件城市）分拣时间以提取到的最后一个分拣中心为准
# 2. “签收时间”分为3种情况，
#### 1）上门送件，以上门的时间为签收时间；
#### 2）放在取件网点，顾客之后自己取走，以放在取件网点的时间为签收时间；
#### 3）放在快递柜、丰巢等，顾客之后自己取走，以放在快递柜、丰巢的时间为签收时间。
# 3. “转运中心”中只有省份名称，没有城市名称的，认为该转运中心就在该省的省会，如果相应的寄出/寄达城市就是该省的省会，则认为快件仍在该寄出地/寄达地

In [11]:
# --------------------------------------------------
# Cell 4.0: logistics数据提取-通用库导入与项目结构设置
# --------------------------------------------------
import re
import warnings
from pathlib import Path
from time import perf_counter

import pandas as pd

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("通用库导入完成。")

# --- 项目路径设置 ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
temp_path = report_path / "temp"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"

# 确保文件夹存在
for p in [anjian_data_path, zhuzhuyun_merge_path, pycharm_input_path]:
    p.mkdir(parents=True, exist_ok=True)

print("项目文件夹结构设置/检查完毕。")


# --- 通用辅助函数 ---
def extractor_apply_based(
    df: pd.DataFrame, profile: dict, location_maps: dict, parser_func
) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame()
    print(f"  -> 使用 '{parser_func.__name__}' 解析器通过 .apply() 运行...")
    return df.apply(lambda row: parser_func(row, profile, location_maps), axis=1)

通用库导入完成。
项目文件夹结构设置/检查完毕。


In [6]:
# ====================================================================================================
# CELL 4.0.1: 全局数据源配置中心 (Global Data Source Configuration Hub)
# ====================================================================================================
#
# 在此集中配置所有物流公司5个核心时间戳的数据来源。
# 后续所有公司处理单元格都将读取此处的配置。
#
# 可选值:
#   'anjian':    使用安监数据
#   'zhuzhuyun': 使用猪猪云合并数据
#
# 注意:
#  - “到达分拣中心时间” 和 “离开收件城市分拣中心时间” 默认从猪猪云数据提取，无需在此配置。
#  - 请确保公司名称的键 (e.g., "EMS", "邮政") 与后续处理单元格中的 `COMPANY_NAME` 变量完全一致。
#
# ====================================================================================================

DATA_SOURCE_CONFIG = {
    # --------------------------- EMS ---------------------------
    "EMS": {
        "collection_time_source": "anjian",  # 揽收时间
        "departure_time_source": "zhuzhuyun",  # 离开寄件城市时间
        "arrival_time_source": "zhuzhuyun",  # 到达收件城市时间
        "dispatch_time_source": "zhuzhuyun",  # 派送时间
        "signed_time_source": "zhuzhuyun",  # 签收时间
    },
    # --------------------------- 邮政 ---------------------------
    "邮政": {
        "collection_time_source": "anjian",
        "departure_time_source": "zhuzhuyun",
        "arrival_time_source": "zhuzhuyun",
        "dispatch_time_source": "zhuzhuyun",
        "signed_time_source": "zhuzhuyun",
    },
    # --------------------------- 京东 ---------------------------
    "京东": {
        "collection_time_source": "anjian",
        "departure_time_source": "zhuzhuyun",
        "arrival_time_source": "zhuzhuyun",
        "dispatch_time_source": "zhuzhuyun",
        "signed_time_source": "zhuzhuyun",
    },
    # --------------------------- 圆通 ---------------------------
    "圆通": {
        "collection_time_source": "anjian",
        "departure_time_source": "zhuzhuyun",
        "arrival_time_source": "zhuzhuyun",
        "dispatch_time_source": "zhuzhuyun",
        "signed_time_source": "zhuzhuyun",
    },
    # --------------------------- 申通 ---------------------------
    "申通": {
        "collection_time_source": "anjian",
        "departure_time_source": "zhuzhuyun",
        "arrival_time_source": "zhuzhuyun",
        "dispatch_time_source": "zhuzhuyun",
        "signed_time_source": "zhuzhuyun",
    },
    # --------------------------- 韵达 ---------------------------
    "韵达": {
        "collection_time_source": "anjian",
        "departure_time_source": "zhuzhuyun",
        "arrival_time_source": "zhuzhuyun",
        "dispatch_time_source": "zhuzhuyun",
        "signed_time_source": "zhuzhuyun",
    },
    # --------------------------- 极兔 ---------------------------
    "极兔": {
        "collection_time_source": "anjian",
        "departure_time_source": "zhuzhuyun",
        "arrival_time_source": "zhuzhuyun",
        "dispatch_time_source": "zhuzhuyun",
        "signed_time_source": "zhuzhuyun",
    },
    # --------------------------- 德邦 ---------------------------
    "德邦": {
        "collection_time_source": "anjian",
        "departure_time_source": "zhuzhuyun",
        "arrival_time_source": "zhuzhuyun",
        "dispatch_time_source": "zhuzhuyun",
        "signed_time_source": "zhuzhuyun",
    },
}

print("✅ 全局数据源配置加载成功！")

✅ 全局数据源配置加载成功！


In [29]:
# --------------------------------------------------
# Cell 4.1: EMS logistics数据提取 (已修改)
# --------------------------------------------------
import re
import warnings
from pathlib import Path
from time import perf_counter

import pandas as pd

# --- 初始化 (仅在独立运行此单元格时需要) ---
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("--- [开始] 处理 EMS ---")

# --- 项目路径设置 (请确保在Cell 0中已正确设置) ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
temp_path = report_path / "temp"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"


# --- 通用辅助函数 (保持不变) ---
def extractor_apply_based(
    df: pd.DataFrame, profile: dict, location_maps: dict, parser_func
) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame()
    print(f"  -> 使用 '{parser_func.__name__}' 解析器通过 .apply() 运行...")
    return df.apply(lambda row: parser_func(row, profile, location_maps), axis=1)


# --- 公司档案: EMS (保持不变) ---
COMPANY_PROFILES_EMS = {
    "default": {
        "collect": r"揽收|已收取|揽件|已取件",
        "delivery": r"派送中|派件员|为您派送|正在为您派件|已安排派送",
        "leave": r"离开|已发出|发往",
        "arrive": r"到达|抵达",
        "center": r"处理中心|分拨中心|转运场|运营区|枢纽中心|集散中心",
        "exclude": r"揽投部|营业部|经营分部|网点|集货点|营销中心",
        "sign_p1": r"已派送至|已投至|已到站|自提柜|智能柜|菜鸟驿站|邮政营业网点",
        "sign_p2": r"本人签收|他人代签收|已代收|放至家门口|指定位置签收",
        "sign_fallback": r"已签收|已妥投|已妥收",
    },
    "EMS": {
        "collect": r"已收取快件|已收寄|已收取邮件|揽收|物流项目组",
        "delivery": r"已安排派送|正在派送中|已派送(?!至)|为您派件|为您派送",
        "leave_p1": r"已乘机|已搭乘邮航专机",
        "leave_hard": r"离开",
        "leave_soft": r"准备发出|已发出",
        "center_p1": r"邮区中心|航空枢纽|处理中心|网路中心|航空中心",
        "center_p2": r"包件车间|集散中心|集散点|航站|快件处理车间|快件处理中心",
        "exclude": r"揽投部|邮政支局|直投中心|揽收部|营销中心",
        "sign_p1": r"已派送至|已投至|到达【拼多多中转仓】|已到站|自提柜|智能柜|菜鸟驿站|邮政营业网点|已暂存至",
        "sign_p2": r"本人签收|他人代签收|已代收|放至家门口|指定位置签收|家门口签收",
        "sign_fallback": r"已签收|已妥投|已妥收",
        "sign_ignore": r"完成取件",
    },
}


# --- 解析器: EMS (终极版) (保持不变, 仅为统一命名规范修改后缀) ---
def parse_logistics_events_ems_ultimate(
    row: pd.Series, profile: dict, location_maps: dict
) -> pd.Series:
    log_text = row["完整物流信息"]
    sender_city_key, dest_city_key = row["寄出城市"], row["寄达城市"]
    # 【改动】为统一命名规范，将列后缀从 _zzy 改为 _zhuzhuyun
    time_cols = [
        "揽收时间_zhuzhuyun",
        "离开寄件城市时间_zhuzhuyun",
        "到达收件城市时间_zhuzhuyun",
        "派送时间_zhuzhuyun",
        "签收时间_zhuzhuyun",
        "到达分拣中心时间_zhuzhuyun",
        "离开收件城市分拣中心时间_zhuzhuyun",
    ]
    extracted_times = {col: pd.NaT for col in time_cols}
    if not isinstance(log_text, str) or not log_text.strip():
        return pd.Series(extracted_times)
    # --- 您其他的解析器逻辑保持完全不变 ---
    kw_collect, kw_delivery = profile.get("collect"), profile.get("delivery")
    kw_leave_p1, kw_leave_hard, kw_leave_soft = (
        profile.get("leave_p1"),
        profile.get("leave_hard"),
        profile.get("leave_soft"),
    )
    kw_arrive, kw_exclude = profile.get("arrive"), profile.get("exclude", "")
    kw_sign_p1, kw_sign_p2, kw_sign_fallback, kw_sign_ignore = (
        profile.get("sign_p1"),
        profile.get("sign_p2"),
        profile.get("sign_fallback"),
        profile.get("sign_ignore"),
    )
    kw_center_p1, kw_center_p2 = profile.get("center_p1"), profile.get("center_p2")
    kw_all_centers = "|".join(filter(None, [kw_center_p1, kw_center_p2]))
    datetime_capture_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    all_events = []
    for line in log_text.strip().split("\n"):
        match = re.search(datetime_capture_pattern, line)
        if match:
            dt = pd.to_datetime(match.group(1), errors="coerce")
            if pd.notna(dt):
                all_events.append({"dt": dt, "line": line})
    if not all_events:
        return pd.Series(extracted_times)
    all_events.sort(key=lambda x: x["dt"])

    def get_location_pattern(city_key, location_maps):
        if not city_key or not isinstance(city_key, str):
            return None
        aliases = location_maps["city_alias_map"].get(
            city_key, [city_key.replace("市", "")]
        )
        province = location_maps["city_to_province_map"].get(city_key)
        if province:
            aliases.append(province.replace("省", "").replace("市", ""))
        clean_aliases = {str(a) for a in aliases if pd.notna(a) and a != ""}
        return "|".join(map(re.escape, clean_aliases))

    for event in all_events:
        if kw_collect and re.search(kw_collect, event["line"]):
            extracted_times["揽收时间_zhuzhuyun"] = event["dt"]
            break
    for event in all_events:
        if (
            pd.isna(extracted_times["派送时间_zhuzhuyun"])
            and kw_delivery
            and re.search(kw_delivery, event["line"])
        ):
            extracted_times["派送时间_zhuzhuyun"] = event["dt"]
            break
    if kw_sign_p1:
        for event in all_events:
            if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
                continue
            if re.search(kw_sign_p1, event["line"]):
                extracted_times["签收时间_zhuzhuyun"] = event["dt"]
                break
    if pd.isna(extracted_times["签收时间_zhuzhuyun"]) and kw_sign_p2:
        for event in all_events:
            if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
                continue
            if re.search(kw_sign_p2, event["line"]):
                extracted_times["签收时间_zhuzhuyun"] = event["dt"]
                break
    if pd.isna(extracted_times["签收时间_zhuzhuyun"]) and kw_sign_fallback:
        for event in all_events:
            if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
                continue
            if re.search(kw_sign_fallback, event["line"]):
                extracted_times["签收时间_zhuzhuyun"] = event["dt"]
                break
    sender_pattern = get_location_pattern(sender_city_key, location_maps)
    if pd.notna(extracted_times["揽收时间_zhuzhuyun"]) and sender_pattern:
        true_leave_kws_parts = [kw_leave_p1, kw_leave_hard]
        true_leave_kws = "|".join(filter(None, true_leave_kws_parts))
        last_true_leave_event = None
        if true_leave_kws:
            for event in all_events:
                if event["dt"] > extracted_times["揽收时间_zhuzhuyun"]:
                    if re.search(true_leave_kws, event["line"]) and re.search(
                        sender_pattern, event["line"]
                    ):
                        last_true_leave_event = event
        if last_true_leave_event:
            extracted_times["离开寄件城市时间_zhuzhuyun"] = last_true_leave_event["dt"]
        last_sender_center_arrival = None
        for event in all_events:
            if event["dt"] > extracted_times["揽收时间_zhuzhuyun"]:
                if (
                    pd.notna(extracted_times["离开寄件城市时间_zhuzhuyun"])
                    and event["dt"] >= extracted_times["离开寄件城市时间_zhuzhuyun"]
                ):
                    break
                if (
                    kw_all_centers
                    and re.search(kw_all_centers, event["line"])
                    and kw_arrive
                    and re.search(kw_arrive, event["line"])
                    and re.search(sender_pattern, event["line"])
                ):
                    if not (kw_exclude and re.search(kw_exclude, event["line"])):
                        last_sender_center_arrival = event
        if last_sender_center_arrival:
            extracted_times["到达分拣中心时间_zhuzhuyun"] = last_sender_center_arrival[
                "dt"
            ]
    dest_pattern = get_location_pattern(dest_city_key, location_maps)
    if dest_pattern:
        if pd.isna(extracted_times["到达收件城市时间_zhuzhuyun"]):
            for event in all_events:
                if (
                    kw_arrive
                    and re.search(kw_arrive, event["line"])
                    and re.search(dest_pattern, event["line"])
                ):
                    extracted_times["到达收件城市时间_zhuzhuyun"] = event["dt"]
                    break
        start_time_for_dest_search = extracted_times.get(
            "到达收件城市时间_zhuzhuyun", pd.NaT
        )
        if pd.isna(start_time_for_dest_search) and pd.notna(
            extracted_times["揽收时间_zhuzhuyun"]
        ):
            start_time_for_dest_search = extracted_times[
                "揽收时间_zhuzhuyun"
            ] + pd.Timedelta(hours=12)
        if pd.notna(start_time_for_dest_search) and kw_all_centers:
            if kw_leave_hard:
                for event in all_events:
                    if event["dt"] > start_time_for_dest_search:
                        if (
                            re.search(kw_all_centers, event["line"])
                            and re.search(kw_leave_hard, event["line"])
                            and re.search(dest_pattern, event["line"])
                        ):
                            if kw_exclude and re.search(kw_exclude, event["line"]):
                                continue
                            extracted_times["离开收件城市分拣中心时间_zhuzhuyun"] = (
                                event["dt"]
                            )
                            break
            if (
                pd.isna(extracted_times["离开收件城市分拣中心时间_zhuzhuyun"])
                and kw_leave_soft
            ):
                for event in all_events:
                    if event["dt"] > start_time_for_dest_search:
                        if (
                            re.search(kw_all_centers, event["line"])
                            and re.search(kw_leave_soft, event["line"])
                            and re.search(dest_pattern, event["line"])
                        ):
                            if kw_exclude and re.search(kw_exclude, event["line"]):
                                continue
                            extracted_times["离开收件城市分拣中心时间_zhuzhuyun"] = (
                                event["dt"]
                            )
                            break
    return pd.Series(extracted_times)


# --- 主流程: EMS (已修改数据读取逻辑) ---
def process_ems_data(zhuzhuyun_dir, anjian_dir, output_dir, base_data_file, period):
    company_name = "EMS"
    config = {
        # 【无需改动】猪猪云文件名已按period读取，符合要求
        "zhuzhu_filename": f"EMS_{period}.xlsx",
        "internal_name": "EMS",
        "anjian_map_key": "EMS",
        "parser": parse_logistics_events_ems_ultimate,
    }
    company_start_time = perf_counter()
    location_maps = {
        "city_alias_map": {},
        "capital_cities_set": set(),
        "city_to_province_map": {},
    }
    if base_data_file.exists():
        try:
            df_hierarchy = pd.read_excel(
                base_data_file,
                sheet_name="city_hierarchy",
                dtype={"Province": str, "City": str, "District": str},
            )
            df_hierarchy.dropna(subset=["Province", "City"], inplace=True)
            df_hierarchy["City_clean"] = df_hierarchy["City"].str.replace(
                "市", "", regex=False
            )
            df_hierarchy["District_clean"] = df_hierarchy["District"].str.replace(
                "市", "", regex=False
            )
            for city, group in df_hierarchy.groupby("City"):
                aliases = [group.iloc[0]["City_clean"]] + group[
                    "District_clean"
                ].unique().tolist()
                location_maps["city_alias_map"][city] = [a for a in aliases if a]
            if "IsCapital" in df_hierarchy.columns:
                df_capitals = df_hierarchy[df_hierarchy["IsCapital"] == 1]
                location_maps["capital_cities_set"] = set(df_capitals["City"].unique())
            city_to_province = df_hierarchy.drop_duplicates("City")[
                ["City", "Province"]
            ].set_index("City")
            location_maps["city_to_province_map"] = city_to_province[
                "Province"
            ].to_dict()
        except Exception as e:
            print(f"[ERROR] 构建地理位置地图失败: {e}。")
    df_base_subset = None
    if base_data_file.exists():
        try:
            df_base = pd.read_excel(base_data_file, sheet_name="inter-city_routes")
            df_base["路线_std"] = (
                df_base["寄出城市"].str.replace("市", "", regex=False)
                + "-"
                + df_base["寄达城市"].str.replace("市", "", regex=False)
            )
            df_base_subset = df_base[
                ["路线_std", "公里", "寄出省份", "寄达省份"]
            ].drop_duplicates("路线_std")
        except Exception as e:
            print(f"[WARNING] 处理 'basic_data.xlsx' 出错: {e}。")
    year = period[:4]
    month = int(period[4:])  # '202506' -> 6, 以匹配 '6月' 而非 '06月'
    anjian_filename = f"{year}年{month}月{company_name}抽样.xlsx"
    anjian_file_path = anjian_dir / anjian_filename

    if not anjian_file_path.exists():
        print(f"[ERROR] 安监数据文件未找到，流程中止: {anjian_file_path.name}")
        return

    print(f"  -> 正在读取安监数据: {anjian_file_path.name}")
    try:
        all_anjian_df = pd.read_excel(
            anjian_file_path, dtype={"单号": str, "寄出城市": str, "寄达城市": str}
        )
    except Exception as e:
        print(f"[ERROR] 读取安监文件 '{anjian_filename}' 时出错: {e}")
        return
    # -----------------------------------------------------------------

    if "快递公司" in all_anjian_df.columns:
        all_anjian_df.rename(columns={"快递公司": "企业"}, inplace=True)

    # 【新增】准备安监数据源，并为时间戳列添加 "_anjian" 后缀
    df_anjian_ems = all_anjian_df[
        all_anjian_df["企业"] == config["anjian_map_key"]
    ].copy()
    if not df_anjian_ems.empty:
        print(
            f"  -> 已从安监数据中筛选出 {len(df_anjian_ems)} 条 {company_name} 记录。"
        )
        df_anjian_ems.rename(
            columns={
                "揽收时间": "揽收时间_anjian",
                "离开寄件城市时间": "离开寄件城市时间_anjian",
                "到达收件城市时间": "到达收件城市时间_anjian",
                "派送时间": "派送时间_anjian",
                "签收时间": "签收时间_anjian",
            },
            inplace=True,
        )

    # --- 猪猪云数据处理 (保持不变) ---
    base_info_df = all_anjian_df[["单号", "企业", "寄出城市", "寄达城市"]].copy()
    company_file = zhuzhuyun_dir / config["zhuzhu_filename"]
    if not company_file.exists():
        print(f"[WARNING] 未找到文件: {config['zhuzhu_filename']}，已跳过。")
        return
    df_company_zhu = pd.read_excel(company_file, dtype={"快递单号": str})
    df_company_zhu.rename(
        columns={"快递单号": "单号", "快递公司": "企业"}, inplace=True
    )
    df_merged = pd.merge(df_company_zhu, base_info_df, on=["单号", "企业"], how="left")
    base_profile = COMPANY_PROFILES_EMS["default"].copy()
    company_specific_profile = COMPANY_PROFILES_EMS.get(config["internal_name"], {})
    profile = {**base_profile, **company_specific_profile}
    extracted_df = extractor_apply_based(
        df_merged, profile, location_maps, config["parser"]
    )
    df_zhuzhuyun_final = pd.concat([df_merged, extracted_df], axis=1)

    # 【新增】将安监数据与处理后的猪猪云数据进行外连接，整合所有信息
    # 保留猪猪云数据中的基础列和解析出的时间戳列
    zhuzhuyun_cols_to_keep = [
        "单号",
        "企业",
        "寄出城市",
        "寄达城市",
        "完整物流信息",
    ] + [col for col in df_zhuzhuyun_final if "_zhuzhuyun" in col]
    df_zhuzhuyun_subset = df_zhuzhuyun_final[zhuzhuyun_cols_to_keep]

    # 保留安监数据中的基础列和时间戳列
    anjian_cols_to_keep = ["单号", "企业"] + [
        col for col in df_anjian_ems if "_anjian" in col
    ]
    df_anjian_subset = df_anjian_ems[anjian_cols_to_keep]

    # 执行合并
    df_final = pd.merge(
        df_zhuzhuyun_subset, df_anjian_subset, on=["单号", "企业"], how="outer"
    )
    print(f"  -> 已整合安监与猪猪云数据，共 {len(df_final)} 条唯一记录。")

    print("  -> 根据全局配置选择时间戳来源...")

    source_config = DATA_SOURCE_CONFIG.get(company_name, {})
    if not source_config:
        raise ValueError(
            f"错误: 在 DATA_SOURCE_CONFIG 中未找到 '{company_name}' 的配置!"
        )

    # 1. 定义需要配置的5个时间戳
    timestamp_map = {
        "揽收时间": "collection_time_source",
        "离开寄件城市时间": "departure_time_source",
        "到达收件城市时间": "arrival_time_source",
        "派送时间": "dispatch_time_source",
        "签收时间": "signed_time_source",
    }

    # 2. 循环处理这5个时间戳，根据配置选择主数据源，并用另一数据源填充空值
    for final_col, config_key in timestamp_map.items():
        source_type = source_config.get(config_key, "anjian")  # 若未配置，默认使用安监
        primary_col = f"{final_col}_{source_type}"
        fallback_col = (
            f"{final_col}_{'zhuzhuyun' if source_type == 'anjian' else 'anjian'}"
        )

        df_final[final_col] = df_final.get(primary_col)
        if fallback_col in df_final:
            df_final[final_col] = df_final[final_col].fillna(df_final[fallback_col])
        print(f"     - '{final_col}' 已配置使用 '{source_type}' 数据源。")

    # 3. 处理固定来源的时间戳 (这2个始终来自猪猪云)
    df_final["到达分拣中心时间"] = df_final.get("到达分拣中心时间_zhuzhuyun")
    df_final["离开收件城市分拣中心时间"] = df_final.get(
        "离开收件城市分拣中心时间_zhuzhuyun"
    )
    # --------------------------------------------------------------------------------------

    # --- 后续处理与保存 (保持不变) ---
    if df_base_subset is not None:
        df_final["路线_std"] = (
            df_final["寄出城市"].astype(str).str.replace("市", "", regex=False)
            + "-"
            + df_final["寄达城市"].astype(str).str.replace("市", "", regex=False)
        )
        df_final = pd.merge(df_final, df_base_subset, on="路线_std", how="left")

    df_final["企业"] = company_name
    output_columns = [
        "企业",
        "单号",
        "寄出城市",
        "寄达城市",
        "揽收时间",
        "离开寄件城市时间",
        "到达收件城市时间",
        "派送时间",
        "签收时间",
        "公里",
        "寄出省份",
        "寄达省份",
        "到达分拣中心时间",
        "离开收件城市分拣中心时间",
        "完整物流信息",
    ]
    final_df_to_save = df_final.reindex(columns=output_columns)

    output_file = output_dir / f"{company_name}_logistics_data_{period}.xlsx"
    final_df_to_save.to_excel(output_file, index=False)
    company_end_time = perf_counter()
    print(
        f"✅ {company_name} 处理完成，耗时 {company_end_time - company_start_time:.2f} 秒。文件已保存至: {output_file.name}"
    )


# --- 运行: EMS ---
if "period" in locals() and "DATA_SOURCE_CONFIG" in locals():
    process_ems_data(
        zhuzhuyun_merge_path,
        anjian_data_path,
        pycharm_input_path,
        base_data_path,
        period,
    )
else:
    print(
        "[警告] 未定义 'period' 或 'DATA_SOURCE_CONFIG' 变量，跳过 process_ems_data 的执行。请在前面的单元格中定义这些变量。"
    )

--- [开始] 处理 EMS ---
  -> 正在读取安监数据: 2025年6月EMS抽样.xlsx
  -> 已从安监数据中筛选出 43390 条 EMS 记录。
  -> 使用 'parse_logistics_events_ems_ultimate' 解析器通过 .apply() 运行...
  -> 已整合安监与猪猪云数据，共 43390 条唯一记录。
  -> 根据全局配置选择时间戳来源...
     - '揽收时间' 已配置使用 'anjian' 数据源。
     - '离开寄件城市时间' 已配置使用 'zhuzhuyun' 数据源。
     - '到达收件城市时间' 已配置使用 'zhuzhuyun' 数据源。
     - '派送时间' 已配置使用 'zhuzhuyun' 数据源。
     - '签收时间' 已配置使用 'zhuzhuyun' 数据源。
✅ EMS 处理完成，耗时 44.77 秒。文件已保存至: EMS_logistics_data_202506.xlsx


In [30]:
# --------------------------------------------------
# Cell 4.2: 邮政logistics数据提取
# --------------------------------------------------
import re
import warnings
from pathlib import Path
from time import perf_counter

import pandas as pd

# --- 初始化 (仅在独立运行此单元格时需要) ---
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("--- [开始] 处理 邮政 ---")

# --- 项目路径设置 (请确保在Cell 0中已正确设置) ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
temp_path = report_path / "temp"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"


# --- 通用辅助函数 (保持不变) ---
def extractor_apply_based(
    df: pd.DataFrame, profile: dict, location_maps: dict, parser_func
) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame()
    print(f"  -> 使用 '{parser_func.__name__}' 解析器通过 .apply() 运行...")
    return df.apply(lambda row: parser_func(row, profile, location_maps), axis=1)


# --- 公司档案: 邮政 (保持不变) ---
COMPANY_PROFILES_POSTAL = {
    "default": {
        "collect": r"揽收|已收取|揽件|已取件",
        "delivery": r"派送中|派件员|为您派送|正在为您派件|已安排派送",
        "leave": r"离开|已发出|发往",
        "arrive": r"到达|抵达",
        "center": r"处理中心|分拨中心|转运场|运营区|枢纽中心|集散中心",
        "exclude": r"揽投部|营业部|经营分部|网点|集货点|营销中心",
        "sign_p1": r"已派送至|已投至|已到站|自提柜|智能柜|菜鸟驿站|邮政营业网点",
        "sign_p2": r"本人签收|他人代签收|已代收|放至家门口|指定位置签收",
        "sign_fallback": r"已签收|已妥投|已妥收",
    },
    "邮政国内小包": {
        "collect": r"已收取快件|已收寄|已收取邮件|揽收|物流项目组",
        "delivery": r"派送中|派件员|为您派件|已安排派送",
        "leave_p1": r"已乘机|已搭乘邮航专机",
        "leave_hard": r"离开",
        "leave_soft": r"准备发出|已发出|发往",
        "leave": r"离开|已发出|发往|准备发出",
        "center_p1": r"邮区中心|航空枢纽|处理中心|网路中心|航空中心",
        "center_p2": r"包件车间|集散中心|集散点|航站|快件处理车间|快件处理班",
        "exclude": r"揽投部|邮政支局|直投中心|揽收部|营销中心|直投点",
        "sign_p1": r"已派送至|已投至|到达【拼多多中转仓】|已到站|自提柜|智能柜|菜鸟驿站|邮政营业网点|已暂存至",
        "sign_p2": r"本人签收|他人代签收|已代收|放至家门口|指定位置签收|家门口签收",
        "sign_fallback": r"已签收|已妥投|已妥收",
        "sign_ignore": r"完成取件",
    },
}


# --- 解析器: 邮政 (终极完美版) (保持不变, 仅为统一命名规范修改后缀) ---
def parse_logistics_events_postal_perfected(
    row: pd.Series, profile: dict, location_maps: dict
) -> pd.Series:
    log_text = row["完整物流信息"]
    sender_city_key, dest_city_key = row["寄出城市"], row["寄达城市"]
    # 【改动】为统一命名规范，将列后缀从 _zzy 改为 _zhuzhuyun
    time_cols = [
        "揽收时间_zhuzhuyun",
        "离开寄件城市时间_zhuzhuyun",
        "到达收件城市时间_zhuzhuyun",
        "派送时间_zhuzhuyun",
        "签收时间_zhuzhuyun",
        "到达分拣中心时间_zhuzhuyun",
        "离开收件城市分拣中心时间_zhuzhuyun",
    ]
    extracted_times = {col: pd.NaT for col in time_cols}
    if not isinstance(log_text, str) or not log_text.strip():
        return pd.Series(extracted_times)
    # --- 您其他的解析器逻辑保持完全不变 ---
    kw_collect, kw_delivery = profile.get("collect"), profile.get("delivery")
    kw_leave_p1, kw_leave_hard, kw_leave_soft, kw_leave = (
        profile.get("leave_p1"),
        profile.get("leave_hard"),
        profile.get("leave_soft"),
        profile.get("leave"),
    )
    kw_arrive, kw_exclude = profile.get("arrive"), profile.get("exclude", "")
    kw_sign_p1, kw_sign_p2, kw_sign_fallback, kw_sign_ignore = (
        profile.get("sign_p1"),
        profile.get("sign_p2"),
        profile.get("sign_fallback"),
        profile.get("sign_ignore"),
    )
    kw_center_p1, kw_center_p2 = profile.get("center_p1"), profile.get("center_p2")
    kw_all_centers = "|".join(filter(None, [kw_center_p1, kw_center_p2]))
    datetime_capture_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    all_events = []
    for line in log_text.strip().split("\n"):
        match = re.search(datetime_capture_pattern, line)
        if match:
            dt = pd.to_datetime(match.group(1), errors="coerce")
            if pd.notna(dt):
                all_events.append({"dt": dt, "line": line})
    if not all_events:
        return pd.Series(extracted_times)
    all_events.sort(key=lambda x: x["dt"])

    def get_location_patterns(city_key, location_maps):
        if not city_key or not isinstance(city_key, str):
            return None, None
        aliases = location_maps["city_alias_map"].get(
            city_key, [city_key.replace("市", "")]
        )
        city_pattern = "|".join(
            map(re.escape, {str(a) for a in aliases if pd.notna(a) and a != ""})
        )
        province = location_maps["city_to_province_map"].get(city_key)
        province_pattern = None
        if province:
            province_clean = province.replace("省", "").replace("市", "")
            if province_clean and not any(
                alias in province_clean
                for alias in {
                    a.replace("市", "") for a in aliases if pd.notna(a) and a != ""
                }
            ):
                province_pattern = re.escape(province_clean)
        return city_pattern, province_pattern

    for event in all_events:
        if (
            pd.isna(extracted_times["揽收时间_zhuzhuyun"])
            and kw_collect
            and re.search(kw_collect, event["line"])
        ):
            extracted_times["揽收时间_zhuzhuyun"] = event["dt"]
    for event in all_events:
        if (
            pd.isna(extracted_times["派送时间_zhuzhuyun"])
            and kw_delivery
            and re.search(kw_delivery, event["line"])
        ):
            extracted_times["派送时间_zhuzhuyun"] = event["dt"]
    if pd.isna(extracted_times["签收时间_zhuzhuyun"]) and kw_sign_p1:
        for event in all_events:
            if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
                continue
            if re.search(kw_sign_p1, event["line"]):
                extracted_times["签收时间_zhuzhuyun"] = event["dt"]
                break
    if pd.isna(extracted_times["签收时间_zhuzhuyun"]) and kw_sign_p2:
        for event in all_events:
            if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
                continue
            if re.search(kw_sign_p2, event["line"]):
                extracted_times["签收时间_zhuzhuyun"] = event["dt"]
                break
    if pd.isna(extracted_times["签收时间_zhuzhuyun"]) and kw_sign_fallback:
        for event in all_events:
            if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
                continue
            if re.search(kw_sign_fallback, event["line"]):
                extracted_times["签收时间_zhuzhuyun"] = event["dt"]
                break
    sender_city_pattern, sender_province_pattern = get_location_patterns(
        sender_city_key, location_maps
    )
    if pd.notna(extracted_times["揽收时间_zhuzhuyun"]):
        sender_candidates = []
        if sender_city_pattern:
            for event in all_events:
                if event["dt"] > extracted_times["揽收时间_zhuzhuyun"] and re.search(
                    sender_city_pattern, event["line"]
                ):
                    sender_candidates.append(event)
        if not sender_candidates and sender_province_pattern:
            for event in all_events:
                if event["dt"] > extracted_times["揽收时间_zhuzhuyun"] and re.search(
                    sender_province_pattern, event["line"]
                ):
                    sender_candidates.append(event)
        last_center_arrival, last_center_leave = None, None
        true_leave_kws = "|".join(filter(None, [kw_leave_p1, kw_leave]))
        for event in sender_candidates:
            if (
                kw_all_centers
                and re.search(kw_all_centers, event["line"])
                and not (kw_exclude and re.search(kw_exclude, event["line"]))
            ):
                if kw_arrive and re.search(kw_arrive, event["line"]):
                    last_center_arrival = event
                if true_leave_kws and re.search(true_leave_kws, event["line"]):
                    last_center_leave = event
            elif kw_leave_p1 and re.search(kw_leave_p1, event["line"]):
                last_center_leave = event
        if last_center_arrival:
            extracted_times["到达分拣中心时间_zhuzhuyun"] = last_center_arrival["dt"]
        if last_center_leave:
            extracted_times["离开寄件城市时间_zhuzhuyun"] = last_center_leave["dt"]
    dest_city_pattern, dest_province_pattern = get_location_patterns(
        dest_city_key, location_maps
    )
    if dest_city_pattern:
        for event in all_events:
            if (
                pd.isna(extracted_times["到达收件城市时间_zhuzhuyun"])
                and kw_all_centers
                and re.search(kw_all_centers, event["line"])
                and not (kw_exclude and re.search(kw_exclude, event["line"]))
                and kw_arrive
                and re.search(kw_arrive, event["line"])
                and re.search(dest_city_pattern, event["line"])
            ):
                extracted_times["到达收件城市时间_zhuzhuyun"] = event["dt"]
                break
    if pd.isna(extracted_times["到达收件城市时间_zhuzhuyun"]) and dest_province_pattern:
        for event in all_events:
            if (
                pd.isna(extracted_times["到达收件城市时间_zhuzhuyun"])
                and kw_all_centers
                and re.search(kw_all_centers, event["line"])
                and not (kw_exclude and re.search(kw_exclude, event["line"]))
                and kw_arrive
                and re.search(kw_arrive, event["line"])
                and re.search(dest_province_pattern, event["line"])
            ):
                extracted_times["到达收件城市时间_zhuzhuyun"] = event["dt"]
                break
    start_time = extracted_times.get("到达收件城市时间_zhuzhuyun", pd.NaT)
    if pd.isna(start_time) and pd.notna(extracted_times["揽收时间_zhuzhuyun"]):
        start_time = extracted_times["揽收时间_zhuzhuyun"]
    if pd.notna(start_time):
        if kw_leave_hard:
            for event in all_events:
                if event["dt"] > start_time:
                    if (
                        dest_city_pattern
                        and re.search(dest_city_pattern, event["line"])
                        or (
                            dest_province_pattern
                            and re.search(dest_province_pattern, event["line"])
                        )
                    ):
                        if (
                            kw_all_centers
                            and re.search(kw_all_centers, event["line"])
                            and not (
                                kw_exclude and re.search(kw_exclude, event["line"])
                            )
                        ):
                            if re.search(kw_leave_hard, event["line"]):
                                extracted_times[
                                    "离开收件城市分拣中心时间_zhuzhuyun"
                                ] = event["dt"]
                                break
        if (
            pd.isna(extracted_times["离开收件城市分拣中心时间_zhuzhuyun"])
            and kw_leave_soft
        ):
            for event in all_events:
                if event["dt"] > start_time:
                    if (
                        dest_city_pattern
                        and re.search(dest_city_pattern, event["line"])
                        or (
                            dest_province_pattern
                            and re.search(dest_province_pattern, event["line"])
                        )
                    ):
                        if (
                            kw_all_centers
                            and re.search(kw_all_centers, event["line"])
                            and not (
                                kw_exclude and re.search(kw_exclude, event["line"])
                            )
                        ):
                            if re.search(kw_leave_soft, event["line"]):
                                extracted_times[
                                    "离开收件城市分拣中心时间_zhuzhuyun"
                                ] = event["dt"]
                                break
    return pd.Series(extracted_times)


# --- 主流程: 邮政 (已修改数据读取逻辑) ---
def process_postal_data(zhuzhuyun_dir, anjian_dir, output_dir, base_data_file, period):
    company_name = "邮政"
    company_start_time = perf_counter()
    # 简化config，只保留当前公司需要的信息
    config = {
        # 【无需改动】猪猪云文件名已按period读取，符合要求
        "zhuzhu_filename": f"邮政_{period}.xlsx",
        "internal_name": "邮政国内小包",  # 用于匹配profile
        "anjian_map_key": "ZGYZ",  # 用于匹配安监数据中的公司名
        "parser": parse_logistics_events_postal_perfected,
    }

    location_maps = {
        "city_alias_map": {},
        "capital_cities_set": set(),
        "city_to_province_map": {},
    }
    if base_data_file.exists():
        try:
            df_hierarchy = pd.read_excel(
                base_data_file,
                sheet_name="city_hierarchy",
                dtype={"Province": str, "City": str, "District": str},
            )
            df_hierarchy.dropna(subset=["Province", "City"], inplace=True)
            df_hierarchy["City_clean"] = df_hierarchy["City"].str.replace(
                "市", "", regex=False
            )
            df_hierarchy["District_clean"] = df_hierarchy["District"].str.replace(
                "市", "", regex=False
            )
            for city, group in df_hierarchy.groupby("City"):
                aliases = [group.iloc[0]["City_clean"]] + group[
                    "District_clean"
                ].unique().tolist()
                location_maps["city_alias_map"][city] = [
                    a for a in aliases if pd.notna(a) and a != ""
                ]
            city_to_province = df_hierarchy.drop_duplicates("City")[
                ["City", "Province"]
            ].set_index("City")
            location_maps["city_to_province_map"] = city_to_province[
                "Province"
            ].to_dict()
        except Exception as e:
            print(f"[ERROR] 构建地理位置地图失败: {e}。")

    df_base_subset = None
    if base_data_file.exists():
        try:
            df_base = pd.read_excel(base_data_file, sheet_name="inter-city_routes")
            df_base["路线_std"] = (
                df_base["寄出城市"].str.replace("市", "", regex=False)
                + "-"
                + df_base["寄达城市"].str.replace("市", "", regex=False)
            )
            df_base_subset = df_base[
                ["路线_std", "公里", "寄出省份", "寄达省份"]
            ].drop_duplicates("路线_std")
        except Exception as e:
            print(f"[WARNING] 处理 'basic_data.xlsx' 出错: {e}。")

    year = period[:4]
    month = int(period[4:])  # '202506' -> 6, 以匹配 '6月' 而非 '06月'
    anjian_filename = f"{year}年{month}月{company_name}抽样.xlsx"
    anjian_file_path = anjian_dir / anjian_filename

    if not anjian_file_path.exists():
        print(f"[ERROR] 安监数据文件未找到，流程中止: {anjian_file_path.name}")
        return

    print(f"  -> 正在读取安监数据: {anjian_file_path.name}")
    try:
        all_anjian_df = pd.read_excel(
            anjian_file_path, dtype={"单号": str, "寄出城市": str, "寄达城市": str}
        )
    except Exception as e:
        print(f"[ERROR] 读取安监文件 '{anjian_filename}' 时出错: {e}")
        return
    # -----------------------------------------------------------------

    if "快递公司" in all_anjian_df.columns:
        all_anjian_df.rename(columns={"快递公司": "企业"}, inplace=True)

    # 【新增】准备安监数据源，并为时间戳列添加 "_anjian" 后缀
    df_anjian_postal = all_anjian_df[
        all_anjian_df["企业"] == config["anjian_map_key"]
    ].copy()
    if not df_anjian_postal.empty:
        print(
            f"  -> 已从安监数据中筛选出 {len(df_anjian_postal)} 条 {company_name} (ZGYZ) 记录。"
        )
        df_anjian_postal.rename(
            columns={
                "揽收时间": "揽收时间_anjian",
                "离开寄件城市时间": "离开寄件城市时间_anjian",
                "到达收件城市时间": "到达收件城市时间_anjian",
                "派送时间": "派送时间_anjian",
                "签收时间": "签收时间_anjian",
                "企业": "企业_anjian_raw",  # 临时重命名以避免合并冲突
            },
            inplace=True,
        )

    # --- 猪猪云数据处理 (保持不变) ---
    company_file = zhuzhuyun_dir / config["zhuzhu_filename"]
    if not company_file.exists():
        print(f"[WARNING] 未找到文件: {config['zhuzhu_filename']}，已跳过。")
        return
    df_company_zhu = pd.read_excel(company_file, dtype={"快递单号": str})
    df_company_zhu.rename(
        columns={"快递单号": "单号", "快递公司": "企业"}, inplace=True
    )
    df_company_zhu["企业"] = company_name  # 将企业名统一为最终要显示的名称

    # 为了让解析器获得城市信息，需要从安监数据中合并
    # 仅合并基础信息，不合并时间戳
    base_info_df = (
        all_anjian_df[all_anjian_df["企业"] == config["anjian_map_key"]][
            ["单号", "寄出城市", "寄达城市"]
        ]
        .copy()
        .drop_duplicates(subset=["单号"])
    )
    df_merged = pd.merge(df_company_zhu, base_info_df, on="单号", how="left")

    base_profile = COMPANY_PROFILES_POSTAL["default"].copy()
    company_specific_profile = COMPANY_PROFILES_POSTAL.get(config["internal_name"], {})
    profile = {**base_profile, **company_specific_profile}
    extracted_df = extractor_apply_based(
        df_merged, profile, location_maps, config["parser"]
    )
    df_zhuzhuyun_final = pd.concat([df_merged, extracted_df], axis=1)

    # 【新增】整合两大数据源
    zhuzhuyun_cols_to_keep = [
        "单号",
        "企业",
        "寄出城市",
        "寄达城市",
        "完整物流信息",
    ] + [col for col in df_zhuzhuyun_final if "_zhuzhuyun" in col]
    df_zhuzhuyun_subset = df_zhuzhuyun_final[zhuzhuyun_cols_to_keep]

    anjian_cols_to_keep = ["单号"] + [
        col for col in df_anjian_postal if "_anjian" in col
    ]
    df_anjian_subset = df_anjian_postal[anjian_cols_to_keep]

    # 执行合并
    df_final = pd.merge(df_zhuzhuyun_subset, df_anjian_subset, on="单号", how="outer")
    # 合并后，企业名称可能来自猪猪云（已统一）或为空（安监独有），统一设置为最终名称
    df_final["企业"] = company_name
    print(f"  -> 已整合安监与猪猪云数据，共 {len(df_final)} 条唯一记录。")
    print("  -> 根据全局配置选择时间戳来源...")
    source_config = DATA_SOURCE_CONFIG.get(company_name, {})
    if not source_config:
        raise ValueError(
            f"错误: 在 DATA_SOURCE_CONFIG 中未找到 '{company_name}' 的配置!"
        )

    timestamp_map = {
        "揽收时间": "collection_time_source",
        "离开寄件城市时间": "departure_time_source",
        "到达收件城市时间": "arrival_time_source",
        "派送时间": "dispatch_time_source",
        "签收时间": "signed_time_source",
    }

    for final_col, config_key in timestamp_map.items():
        source_type = source_config.get(config_key, "anjian")
        primary_col = f"{final_col}_{source_type}"
        fallback_col = (
            f"{final_col}_{'zhuzhuyun' if source_type == 'anjian' else 'anjian'}"
        )

        df_final[final_col] = df_final.get(primary_col)
        if fallback_col in df_final:
            df_final[final_col] = df_final[final_col].fillna(df_final[fallback_col])
        print(f"     - '{final_col}' 已配置使用 '{source_type}' 数据源。")

    df_final["到达分拣中心时间"] = df_final.get("到达分拣中心时间_zhuzhuyun")
    df_final["离开收件城市分拣中心时间"] = df_final.get(
        "离开收件城市分拣中心时间_zhuzhuyun"
    )
    # --------------------------------------------------------------------------------------

    # --- 后续处理与保存 (保持不变) ---
    if df_base_subset is not None:
        # 填充可能因 outer merge 产生的缺失城市信息
        if "寄出城市_x" in df_final.columns and "寄出城市_y" in df_final.columns:
            df_final["寄出城市"] = df_final["寄出城市_x"].fillna(df_final["寄出城市_y"])
            df_final["寄达城市"] = df_final["寄达城市_x"].fillna(df_final["寄达城市_y"])

        df_final["路线_std"] = (
            df_final["寄出城市"].astype(str).str.replace("市", "", regex=False)
            + "-"
            + df_final["寄达城市"].astype(str).str.replace("市", "", regex=False)
        )
        df_final = pd.merge(df_final, df_base_subset, on="路线_std", how="left")

    output_columns = [
        "企业",
        "单号",
        "寄出城市",
        "寄达城市",
        "揽收时间",
        "离开寄件城市时间",
        "到达收件城市时间",
        "派送时间",
        "签收时间",
        "公里",
        "寄出省份",
        "寄达省份",
        "到达分拣中心时间",
        "离开收件城市分拣中心时间",
        "完整物流信息",
    ]
    final_df_to_save = df_final.reindex(columns=output_columns)

    output_file = output_dir / f"{company_name}_logistics_data_{period}.xlsx"
    final_df_to_save.to_excel(output_file, index=False)
    company_end_time = perf_counter()
    print(
        f"✅ {company_name} 处理完成，耗时 {company_end_time - company_start_time:.2f} 秒。文件已保存至: {output_file.name}"
    )


# --- 运行: 邮政 (保持不变) ---
if "period" in locals() and "DATA_SOURCE_CONFIG" in locals():
    process_postal_data(
        zhuzhuyun_merge_path,
        anjian_data_path,
        pycharm_input_path,
        base_data_path,
        period,
    )
else:
    print(
        "[警告] 未定义 'period' 或 'DATA_SOURCE_CONFIG' 变量，跳过 process_postal_data 的执行。请在前面的单元格中定义这些变量。"
    )

--- [开始] 处理 邮政 ---
  -> 正在读取安监数据: 2025年6月邮政抽样.xlsx
  -> 已从安监数据中筛选出 81571 条 邮政 (ZGYZ) 记录。
  -> 使用 'parse_logistics_events_postal_perfected' 解析器通过 .apply() 运行...
  -> 已整合安监与猪猪云数据，共 81571 条唯一记录。
  -> 根据全局配置选择时间戳来源...
     - '揽收时间' 已配置使用 'anjian' 数据源。
     - '离开寄件城市时间' 已配置使用 'zhuzhuyun' 数据源。
     - '到达收件城市时间' 已配置使用 'zhuzhuyun' 数据源。
     - '派送时间' 已配置使用 'zhuzhuyun' 数据源。
     - '签收时间' 已配置使用 'zhuzhuyun' 数据源。
✅ 邮政 处理完成，耗时 55.18 秒。文件已保存至: 邮政_logistics_data_202506.xlsx


In [26]:
# --------------------------------------------------
# Cell 4.3: 京东logistics数据提取
# --------------------------------------------------
import re
import warnings
from pathlib import Path
from time import perf_counter

import pandas as pd

# --- 初始化 (仅在独立运行此单元格时需要) ---
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("--- [开始] 处理 京东 ---")

# --- 项目路径设置 (请确保在Cell 0中已正确设置) ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
temp_path = report_path / "temp"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"


# --- 通用辅助函数 (保持不变) ---
def extractor_apply_based(
    df: pd.DataFrame, profile: dict, location_maps: dict, parser_func
) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame()
    print(f"  -> 使用 '{parser_func.__name__}' 解析器通过 .apply() 运行...")
    return df.apply(lambda row: parser_func(row, profile, location_maps), axis=1)


# --- 公司档案: 京东 (保持不变) ---
COMPANY_PROFILES_JD = {
    "default": {
        "collect": r"揽收|已收取|揽件|已取件|揽收完成",
        "delivery": r"派送中|派件员|为您派送|正在为您派件|已安排派送",
        "leave": r"离开|已发出|发往",
        "arrive": r"到达|抵达",
        "center": r"处理中心|分拨中心|转运场|运营区|枢纽中心|集散中心|分拣中心|接货仓",
        "exclude": r"揽投部|营业部|经营分部|网点|集货点|营销中心|项目营业点|校园服务站|接驳点",
        "sign_p1": r"已派送至|已投至|已到站|自提柜|智能柜|菜鸟驿站|邮政营业网点|已送达至|已送至|校园服务站|快递柜|云柜|丰巢柜|便民驿站",
        "sign_p2": r"本人签收|他人代签收|已代收|放至家门口|指定位置签收|已送达至|已由.*?代收|已由.*?签收",
        "sign_fallback": r"已签收|已妥投|已妥收|已送达",
        "sign_ignore": r"完成取件",
    },
    "京东": {},
}


# --- 解析器: 京东 (保持不变) ---
def parse_logistics_events_jd(
    row: pd.Series, profile: dict, location_maps: dict
) -> pd.Series:
    log_text = row["完整物流信息"]
    sender_city_key, dest_city_key = row["寄出城市"], row["寄达城市"]
    time_cols = [
        "揽收时间_zhuzhuyun",
        "离开寄件城市时间_zhuzhuyun",
        "到达收件城市时间_zhuzhuyun",
        "派送时间_zhuzhuyun",
        "签收时间_zhuzhuyun",
        "到达分拣中心时间_zhuzhuyun",
        "离开收件城市分拣中心时间_zhuzhuyun",
    ]
    extracted_times = {col: pd.NaT for col in time_cols}
    if not isinstance(log_text, str) or not log_text.strip():
        return pd.Series(extracted_times)
    kw = {
        key: profile.get(key)
        for key in [
            "collect",
            "delivery",
            "leave",
            "arrive",
            "center",
            "exclude",
            "sign_p1",
            "sign_p2",
            "sign_fallback",
            "sign_ignore",
        ]
    }
    datetime_capture_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    all_events = []
    for line in log_text.strip().split("\n"):
        match = re.search(datetime_capture_pattern, line)
        if match:
            location_in_brackets = re.search(r"【(.*?)】", line)
            is_center = False
            location_name = ""
            if location_in_brackets:
                location_name = location_in_brackets.group(1)
                if kw["center"] and re.search(kw["center"], location_name):
                    if not (kw["exclude"] and re.search(kw["exclude"], location_name)):
                        is_center = True
            all_events.append(
                {
                    "dt": pd.to_datetime(match.group(1), errors="coerce"),
                    "line": line,
                    "is_center": is_center,
                    "location": location_name,
                }
            )
    all_events = sorted(
        [e for e in all_events if pd.notna(e["dt"])], key=lambda x: x["dt"]
    )
    if not all_events:
        return pd.Series(extracted_times)

    def get_location_pattern(city_key, location_maps):
        if not city_key or not isinstance(city_key, str):
            return None
        aliases = set(
            location_maps["city_alias_map"].get(city_key, [city_key.replace("市", "")])
        )
        province = location_maps["city_to_province_map"].get(city_key)
        if province:
            aliases.add(province.replace("省", "").replace("市", ""))
            if location_maps.get("province_capital_map", {}).get(province) == city_key:
                aliases.add(province.replace("省", ""))
        return "|".join(map(re.escape, {a for a in aliases if a}))

    sender_pattern = get_location_pattern(sender_city_key, location_maps)
    dest_pattern = get_location_pattern(dest_city_key, location_maps)
    for event in all_events:
        if kw["collect"] and re.search(kw["collect"], event["line"]):
            extracted_times["揽收时间_zhuzhuyun"] = event["dt"]
            break
    if pd.isna(extracted_times["揽收时间_zhuzhuyun"]):
        extracted_times["揽收时间_zhuzhuyun"] = all_events[0]["dt"]
    for event in all_events:
        if kw["delivery"] and re.search(kw["delivery"], event["line"]):
            extracted_times["派送时间_zhuzhuyun"] = event["dt"]
            break
    for p_key in ["sign_p1", "sign_p2", "sign_fallback"]:
        if pd.notna(extracted_times["签收时间_zhuzhuyun"]):
            break
        if kw[p_key]:
            for event in reversed(all_events):
                if kw["sign_ignore"] and re.search(kw["sign_ignore"], event["line"]):
                    continue
                if re.search(kw[p_key], event["line"]):
                    extracted_times["签收时间_zhuzhuyun"] = event["dt"]
                    break
    if pd.notna(extracted_times["揽收时间_zhuzhuyun"]) and sender_pattern:
        origin_center_events = [
            e
            for e in all_events
            if e["dt"] > extracted_times["揽收时间_zhuzhuyun"]
            and e["is_center"]
            and re.search(sender_pattern, e["location"])
        ]
        for event in reversed(origin_center_events):
            if kw["leave"] and re.search(kw["leave"], event["line"]):
                match = re.search(r"发往【(.*?)】", event["line"])
                if match:
                    destination_of_leave = match.group(1)
                    if not re.search(sender_pattern, destination_of_leave):
                        extracted_times["离开寄件城市时间_zhuzhuyun"] = event["dt"]
                        location_match = re.search(r"【(.*?)】", event["line"])
                        if location_match:
                            location_name = re.escape(
                                location_match.group(1).split("】")[0]
                            )
                            for arr_event in origin_center_events:
                                if (
                                    arr_event["dt"] <= event["dt"]
                                    and re.search(location_name, arr_event["line"])
                                    and re.search(kw["arrive"], arr_event["line"])
                                ):
                                    extracted_times["到达分拣中心时间_zhuzhuyun"] = (
                                        arr_event["dt"]
                                    )
                                    break
                        break
    if dest_pattern:
        for event in all_events:
            if (
                not event["is_center"]
                and kw["arrive"]
                and re.search(kw["arrive"], event["line"])
            ):
                location_name = event["location"].replace("市", "")
                if dest_city_key and location_name == dest_city_key.replace("市", ""):
                    extracted_times["到达收件城市时间_zhuzhuyun"] = event["dt"]
                    break
        if pd.isna(extracted_times["到达收件城市时间_zhuzhuyun"]):
            for event in all_events:
                if (
                    event["is_center"]
                    and kw["arrive"]
                    and re.search(kw["arrive"], event["line"])
                    and re.search(dest_pattern, event["location"])
                ):
                    extracted_times["到达收件城市时间_zhuzhuyun"] = event["dt"]
                    break
        if pd.notna(extracted_times["派送时间_zhuzhuyun"]):
            dest_center_events = [
                e
                for e in all_events
                if e["dt"] < extracted_times["派送时间_zhuzhuyun"]
                and e["is_center"]
                and re.search(dest_pattern, e["location"])
            ]
            for event in dest_center_events:
                if (
                    pd.notna(extracted_times["到达收件城市时间_zhuzhuyun"])
                    and event["dt"] < extracted_times["到达收件城市时间_zhuzhuyun"]
                ):
                    continue
                if kw["leave"] and re.search(kw["leave"], event["line"]):
                    extracted_times["离开收件城市分拣中心时间_zhuzhuyun"] = event["dt"]
                    break
    return pd.Series(extracted_times)


# --- 主流程: 京东 ---
def process_jd_data(zhuzhuyun_dir, anjian_dir, output_dir, base_data_file, period):
    company_name = "京东"
    config = {
        "zhuzhu_filename": f"京东_{period}.xlsx",
        "internal_name": "京东",
        "anjian_map_key": "JBD",
        "parser": parse_logistics_events_jd,
    }
    company_start_time = perf_counter()
    location_maps = {
        "city_alias_map": {},
        "capital_cities_set": set(),
        "city_to_province_map": {},
        "province_capital_map": {},
    }
    if base_data_file.exists():
        try:
            excel_file = pd.ExcelFile(base_data_file)
            df_hierarchy = pd.read_excel(excel_file, sheet_name="city_hierarchy")
            df_hierarchy_no_na = df_hierarchy.dropna(subset=["Province", "City"])
            df_hierarchy_no_na["District"] = df_hierarchy_no_na["District"].fillna("")
            df_hierarchy_no_na["City_clean"] = df_hierarchy_no_na["City"].str.replace(
                "市", "", regex=False
            )
            df_hierarchy_no_na["District_clean"] = df_hierarchy_no_na[
                "District"
            ].str.replace("市", "", regex=False)
            for city, group in df_hierarchy_no_na.groupby("City"):
                aliases = [group.iloc[0]["City_clean"]] + group[
                    "District_clean"
                ].unique().tolist()
                location_maps["city_alias_map"][city] = [a for a in aliases if a]
            city_to_province = df_hierarchy_no_na.drop_duplicates("City")[
                ["City", "Province"]
            ].set_index("City")
            location_maps["city_to_province_map"] = city_to_province[
                "Province"
            ].to_dict()
            if "IsCapital" in df_hierarchy.columns:
                df_capitals = (
                    df_hierarchy[df_hierarchy["IsCapital"] == 1]
                    .copy()
                    .drop_duplicates(subset=["Province"])
                )
                location_maps["province_capital_map"] = df_capitals.set_index(
                    "Province"
                )["City"].to_dict()
                location_maps["capital_cities_set"] = set(df_capitals["City"].unique())
        except Exception as e:
            print(f"[ERROR] 构建地理位置地图失败: {e}。")
    df_base_subset = None
    if base_data_file.exists():
        try:
            df_base = pd.read_excel(base_data_file, sheet_name="inter-city_routes")
            df_base["路线_std"] = (
                df_base["寄出城市"].str.replace("市", "", regex=False)
                + "-"
                + df_base["寄达城市"].str.replace("市", "", regex=False)
            )
            df_base_subset = df_base[
                ["路线_std", "公里", "寄出省份", "寄达省份"]
            ].drop_duplicates("路线_std")
        except Exception as e:
            print(f"[WARNING] 处理 'basic_data.xlsx' 出错: {e}。")

    year = period[:4]
    month = period[4:].lstrip("0")
    anjian_search_pattern = f"{year}年{month}月*.xlsx"
    print(f"  -> 正在安监数据目录中搜索匹配 '{anjian_search_pattern}' 的文件...")
    all_anjian_files = [
        f for f in anjian_dir.glob(anjian_search_pattern) if not f.name.startswith("~$")
    ]
    if not all_anjian_files:
        print(
            f"[ERROR] 在目录 {anjian_dir} 中未找到与 '{anjian_search_pattern}' 匹配的安监数据Excel文件，处理中断。"
        )
        return
    print(f"  -> 找到 {len(all_anjian_files)} 个匹配的安监文件，准备合并。")
    all_anjian_df = pd.concat(
        [pd.read_excel(f, dtype={"单号": str}) for f in all_anjian_files],
        ignore_index=True,
    )
    if "快递公司" in all_anjian_df.columns:
        all_anjian_df.rename(columns={"快递公司": "企业"}, inplace=True)

    company_file = zhuzhuyun_dir / config["zhuzhu_filename"]
    if not company_file.exists():
        print(f"[WARNING] 未找到文件: {config['zhuzhu_filename']}，已跳过。")
        return
    df_company_zhu = pd.read_excel(company_file, dtype={"快递单号": str})
    df_company_zhu.rename(
        columns={"快递单号": "单号", "快递公司": "企业"}, inplace=True
    )

    base_info_df = all_anjian_df[all_anjian_df["企业"] == config["anjian_map_key"]][
        ["单号", "企业", "寄出城市", "寄达城市"]
    ].drop_duplicates("单号")
    base_info_df["企业"] = config["internal_name"]
    df_merged = pd.merge(df_company_zhu, base_info_df, on=["单号", "企业"], how="left")

    base_profile = COMPANY_PROFILES_JD["default"].copy()
    company_specific_profile = COMPANY_PROFILES_JD.get(config["internal_name"], {})
    profile = {**base_profile, **company_specific_profile}
    extracted_df = extractor_apply_based(
        df_merged, profile, location_maps, config["parser"]
    )
    df_zhuzhuyun_final = pd.concat([df_merged, extracted_df], axis=1)

    df_anjian_jd = all_anjian_df[
        all_anjian_df["企业"] == config["anjian_map_key"]
    ].copy()
    df_anjian_jd.rename(
        columns={
            "揽收时间": "揽收时间_anjian",
            "离开寄件城市时间": "离开寄件城市时间_anjian",
            "到达收件城市时间": "到达收件城市时间_anjian",
            "派送时间": "派送时间_anjian",
            "签收时间": "签收时间_anjian",
        },
        inplace=True,
    )
    df_anjian_jd["企业"] = config["internal_name"]

    zhuzhuyun_cols_to_keep = ["单号", "企业", "完整物流信息"] + [
        col for col in df_zhuzhuyun_final if "_zhuzhuyun" in col
    ]
    anjian_cols_to_keep = ["单号", "企业", "寄出城市", "寄达城市"] + [
        col for col in df_anjian_jd if "_anjian" in col
    ]
    df_final = pd.merge(
        df_zhuzhuyun_final[zhuzhuyun_cols_to_keep],
        df_anjian_jd[anjian_cols_to_keep],
        on=["单号", "企业"],
        how="outer",
    )
    print(f"  -> 已整合安监与猪猪云数据，共 {len(df_final)} 条唯一记录。")

    # --- 【新增】在最终合并后，删除缺少“完整物流信息”的行 ---
    initial_final_rows = len(df_final)
    df_final.dropna(subset=["完整物流信息"], inplace=True)
    rows_dropped_final = initial_final_rows - len(df_final)
    if rows_dropped_final > 0:
        print(
            f"  -> 已从最终结果中删除 {rows_dropped_final} 行缺少'完整物流信息'的数据。"
        )
    # --- 新增结束 ---

    if "寄出城市_x" in df_final.columns:
        df_final["寄出城市"] = df_final["寄出城市_y"].fillna(df_final["寄出城市_x"])
        df_final["寄达城市"] = df_final["寄达城市_y"].fillna(df_final["寄达城市_x"])
        df_final.drop(
            columns=["寄出城市_x", "寄出城市_y", "寄达城市_x", "寄达城市_y"],
            inplace=True,
        )
        print("  -> 已合并城市信息，安监数据优先。")

    print("  -> 根据全局配置选择时间戳来源...")
    source_config = DATA_SOURCE_CONFIG.get(company_name, {})
    if not source_config:
        raise ValueError(
            f"错误: 在 DATA_SOURCE_CONFIG 中未找到 '{company_name}' 的配置!"
        )
    timestamp_map = {
        "揽收时间": "collection_time_source",
        "离开寄件城市时间": "departure_time_source",
        "到达收件城市时间": "arrival_time_source",
        "派送时间": "dispatch_time_source",
        "签收时间": "signed_time_source",
    }
    for final_col, config_key in timestamp_map.items():
        source_type = source_config.get(config_key, "anjian")
        primary_col = f"{final_col}_{source_type}"
        fallback_col = (
            f"{final_col}_{'zhuzhuyun' if source_type == 'anjian' else 'anjian'}"
        )
        df_final[final_col] = df_final.get(primary_col)
        if fallback_col in df_final:
            df_final[final_col] = df_final[final_col].fillna(df_final[fallback_col])
        print(f"     - '{final_col}' 已配置使用 '{source_type}' 数据源。")
    df_final["到达分拣中心时间"] = df_final.get("到达分拣中心时间_zhuzhuyun")
    df_final["离开收件城市分拣中心时间"] = df_final.get(
        "离开收件城市分拣中心时间_zhuzhuyun"
    )

    if df_base_subset is not None:
        df_final["寄出城市"] = df_final["寄出城市"].astype(str)
        df_final["寄达城市"] = df_final["寄达城市"].astype(str)
        df_final["路线_std"] = (
            df_final["寄出城市"].str.replace("市", "", regex=False)
            + "-"
            + df_final["寄达城市"].str.replace("市", "", regex=False)
        )
        df_final = pd.merge(df_final, df_base_subset, on="路线_std", how="left")
    df_final["企业"] = company_name
    output_columns = [
        "企业",
        "单号",
        "寄出城市",
        "寄达城市",
        "揽收时间",
        "离开寄件城市时间",
        "到达收件城市时间",
        "派送时间",
        "签收时间",
        "公里",
        "寄出省份",
        "寄达省份",
        "到达分拣中心时间",
        "离开收件城市分拣中心时间",
        "完整物流信息",
    ]
    final_df_to_save = df_final.reindex(columns=output_columns)
    output_file = output_dir / f"{company_name}_logistics_data_{period}.xlsx"
    final_df_to_save.to_excel(output_file, index=False)
    company_end_time = perf_counter()
    print(
        f"✅ {company_name} 处理完成，耗时 {company_end_time - company_start_time:.2f} 秒。文件已保存至: {output_file.name}"
    )


# --- 运行: 京东 ---
if "period" in locals() and "DATA_SOURCE_CONFIG" in locals():
    process_jd_data(
        zhuzhuyun_merge_path,
        anjian_data_path,
        pycharm_input_path,
        base_data_path,
        period,
    )
else:
    print(
        "[警告] 未定义 'period' 或 'DATA_SOURCE_CONFIG' 变量，跳过 process_jd_data 的执行。"
    )

--- [开始] 处理 京东 ---
  -> 正在安监数据目录中搜索匹配 '2025年6月*.xlsx' 的文件...
  -> 找到 10 个匹配的安监文件，准备合并。
  -> 使用 'parse_logistics_events_jd' 解析器通过 .apply() 运行...
  -> 已整合安监与猪猪云数据，共 82770 条唯一记录。
  -> 已从最终结果中删除 3082 行缺少'完整物流信息'的数据。
  -> 根据全局配置选择时间戳来源...
     - '揽收时间' 已配置使用 'anjian' 数据源。
     - '离开寄件城市时间' 已配置使用 'zhuzhuyun' 数据源。
     - '到达收件城市时间' 已配置使用 'zhuzhuyun' 数据源。
     - '派送时间' 已配置使用 'zhuzhuyun' 数据源。
     - '签收时间' 已配置使用 'zhuzhuyun' 数据源。
✅ 京东 处理完成，耗时 92.07 秒。文件已保存至: 京东_logistics_data_202506.xlsx


In [31]:
# --------------------------------------------------
# Cell 4.4: 极兔logistics数据提取 (已修改)
# --------------------------------------------------
import re
import warnings
from pathlib import Path
from time import perf_counter

import pandas as pd

# --- 初始化 (仅在独立运行此单元格时需要) ---
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("--- [开始] 处理 极兔 ---")

# --- 项目路径设置 (请确保在Cell 0中已正确设置) ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
temp_path = report_path / "temp"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"


# --- 通用辅助函数 (保持不变) ---
def extractor_apply_based(
    df: pd.DataFrame, profile: dict, location_maps: dict, parser_func
) -> pd.DataFrame:
    """通用行应用提取器"""
    if df.empty:
        return pd.DataFrame()
    print(f"  -> 使用 '{parser_func.__name__}' 解析器通过 .apply() 运行...")
    return df.apply(lambda row: parser_func(row, profile, location_maps), axis=1)


# --- 公司档案: 极兔 (规则已完善) (保持不变) ---
COMPANY_PROFILES_JITU = {
    "default": {
        "collect": r"揽收|已收取|揽件|已取件",
        "delivery": r"派送中|派件员|为您派送|正在为您派件|已安排派送",
        "leave": r"离开|已发出|发往",
        "arrive": r"到达|抵达",
        "center": r"处理中心|分拨中心|转运场|运营区|枢纽中心|集散中心|分拣中心",
        "sign_p1": r"已暂存至|已投至|已到站|自提柜|智能柜|菜鸟驿站|快递超市|代收点|存放.*(快递柜|驿站|自提点)",
        "sign_p2": r"本人签收|他人代签收|已代收|放至家门口|指定位置签收|已上门",
        "sign_fallback": r"已签收|已妥投|已妥收|已投递",
    },
    "极兔": {
        "center": r"转运中心|集散中心|集散点",
        "center_exclude": r"公司|分部|服务部|网格仓|揽投部|营业部|经营分部|网点|集货点|营销中心|站点",
        "dest_arrival_exclude": r"服务部|网点",
        "sign_p1": r"已存放至.*?【|已送达.*?【|暂由.*?代为保管",
        "sign_p2": r"已按址投递|已由本人签收|客户签收",
        "sign_fallback": r"快件已签收|快件已按约定投递",
    },
}


# --- 解析器: 极兔 (已应用最终修复) (保持不变, 仅为统一命名规范修改后缀) ---
def parse_logistics_events_jitu(
    row: pd.Series, profile: dict, location_maps: dict
) -> pd.Series:
    """
    解析极兔的物流事件，并根据预设原则提取关键时间点。
    """
    log_text = row["完整物流信息"]
    sender_city_key, dest_city_key = row["寄出城市"], row["寄达城市"]
    # 【改动】为统一命名规范，将列后缀从 _zzy 改为 _zhuzhuyun
    time_cols = [
        "揽收时间_zhuzhuyun",
        "离开寄件城市时间_zhuzhuyun",
        "到达收件城市时间_zhuzhuyun",
        "派送时间_zhuzhuyun",
        "签收时间_zhuzhuyun",
        "到达分拣中心时间_zhuzhuyun",
        "离开收件城市分拣中心时间_zhuzhuyun",
    ]
    extracted_times = {col: pd.NaT for col in time_cols}
    if not isinstance(log_text, str) or not log_text.strip():
        return pd.Series(extracted_times)
    # --- 您其他的解析器逻辑保持完全不变 ---
    kw_collect, kw_delivery = profile.get("collect"), profile.get("delivery")
    kw_leave, kw_arrive = profile.get("leave"), profile.get("arrive")
    kw_sign_p1, kw_sign_p2, kw_sign_fallback, kw_sign_ignore = (
        profile.get("sign_p1"),
        profile.get("sign_p2"),
        profile.get("sign_fallback"),
        profile.get("sign_ignore"),
    )
    kw_all_centers = profile.get("center")
    kw_center_exclude = profile.get("center_exclude", "")
    kw_dest_arrival_exclude = profile.get("dest_arrival_exclude", "")
    datetime_capture_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    all_events = []
    for line in log_text.strip().split("\n"):
        match = re.search(datetime_capture_pattern, line)
        if match:
            dt = pd.to_datetime(match.group(1), errors="coerce")
            if pd.notna(dt):
                all_events.append({"dt": dt, "line": line})
    if not all_events:
        return pd.Series(extracted_times)
    all_events.sort(key=lambda x: x["dt"])

    def get_location_pattern(city_key, location_maps):
        if not city_key or not isinstance(city_key, str) or pd.isna(city_key):
            return None
        aliases = location_maps.get("city_alias_map", {}).get(
            city_key, [city_key.replace("市", "")]
        )
        clean_aliases = {
            str(a)
            for a in aliases
            if a is not None and pd.notna(a) and str(a).strip() != ""
        }
        if not clean_aliases:
            return None
        return "|".join(map(re.escape, clean_aliases))

    for event in all_events:
        if kw_collect and re.search(kw_collect, event["line"]):
            extracted_times["揽收时间_zhuzhuyun"] = event["dt"]
            break
    for event in all_events:
        if kw_delivery and re.search(kw_delivery, event["line"]):
            extracted_times["派送时间_zhuzhuyun"] = event["dt"]
            break
    p1_event, p2_event, p3_event = None, None, None
    for event in all_events:
        if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
            continue
        if kw_sign_p1 and re.search(kw_sign_p1, event["line"]):
            p1_event = event
            break
    for event in reversed(all_events):
        if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
            continue
        if kw_sign_p2 and re.search(kw_sign_p2, event["line"]):
            p2_event = event
            break
    for event in reversed(all_events):
        if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
            continue
        if kw_sign_fallback and re.search(kw_sign_fallback, event["line"]):
            p3_event = event
            break
    if p1_event:
        extracted_times["签收时间_zhuzhuyun"] = p1_event["dt"]
    elif p2_event:
        extracted_times["签收时间_zhuzhuyun"] = p2_event["dt"]
    elif p3_event:
        extracted_times["签收时间_zhuzhuyun"] = p3_event["dt"]
    sender_pattern = get_location_pattern(sender_city_key, location_maps)
    dest_pattern = get_location_pattern(dest_city_key, location_maps)
    if (
        pd.notna(extracted_times["揽收时间_zhuzhuyun"])
        and sender_pattern
        and kw_all_centers
    ):
        last_sender_leave_event = None
        last_sender_center_arrival = None
        for event in all_events:
            if event["dt"] > extracted_times["揽收时间_zhuzhuyun"]:
                action_part = re.split(r"；|已发往|发往", event["line"])[0]
                is_real_center = re.search(kw_all_centers, action_part) and not (
                    kw_center_exclude and re.search(kw_center_exclude, action_part)
                )
                if (
                    kw_leave
                    and re.search(kw_leave, action_part)
                    and re.search(sender_pattern, action_part)
                    and is_real_center
                ):
                    last_sender_leave_event = event
                if (
                    kw_arrive
                    and re.search(kw_arrive, action_part)
                    and re.search(sender_pattern, action_part)
                    and is_real_center
                ):
                    last_sender_center_arrival = event
        if last_sender_leave_event:
            extracted_times["离开寄件城市时间_zhuzhuyun"] = last_sender_leave_event[
                "dt"
            ]
        if last_sender_center_arrival:
            extracted_times["到达分拣中心时间_zhuzhuyun"] = last_sender_center_arrival[
                "dt"
            ]
    if dest_pattern:
        for event in all_events:
            action_part = re.split(r"；|已发往|发往", event["line"])[0]
            if (
                kw_arrive
                and re.search(kw_arrive, action_part)
                and re.search(dest_pattern, action_part)
            ):
                if not (
                    kw_dest_arrival_exclude
                    and re.search(kw_dest_arrival_exclude, action_part)
                ):
                    extracted_times["到达收件城市时间_zhuzhuyun"] = event["dt"]
                    break
        if pd.notna(extracted_times["派送时间_zhuzhuyun"]):
            search_end_time = extracted_times["派送时间_zhuzhuyun"]
            for event in reversed(all_events):
                if event["dt"] < search_end_time:
                    action_part = re.split(r"；|已发往|发往", event["line"])[0]
                    is_real_center = re.search(kw_all_centers, action_part) and not (
                        kw_center_exclude and re.search(kw_center_exclude, action_part)
                    )
                    if (
                        kw_leave
                        and re.search(kw_leave, action_part)
                        and re.search(dest_pattern, action_part)
                        and is_real_center
                    ):
                        extracted_times["离开收件城市分拣中心时间_zhuzhuyun"] = event[
                            "dt"
                        ]
                        break
    return pd.Series(extracted_times)


# --- 主流程: 极兔 (已修改数据读取逻辑) ---
def process_jitu_data(zhuzhuyun_dir, anjian_dir, output_dir, base_data_file, period):
    """主流程函数，用于处理极兔快递的数据"""
    company_name = "极兔"
    company_start_time = perf_counter()
    config = {
        "zhuzhu_filename": f"极兔_{period}.xlsx",
        "internal_name": "极兔",
        "anjian_map_key": "JT",
        "parser": parse_logistics_events_jitu,
    }
    location_maps = {}
    if base_data_file.exists():
        try:
            df_hierarchy = pd.read_excel(
                base_data_file,
                sheet_name="city_hierarchy",
                dtype={"Province": str, "City": str, "District": str},
            )
            df_hierarchy.dropna(subset=["Province", "City"], inplace=True)
            df_hierarchy["City_clean"] = df_hierarchy["City"].str.replace(
                "市", "", regex=False
            )
            df_hierarchy["District_clean"] = df_hierarchy["District"].str.replace(
                r"区|县|自治州", "", regex=True
            )
            location_maps["city_alias_map"] = {
                city: [group.iloc[0]["City_clean"]]
                + group["District_clean"].unique().tolist()
                for city, group in df_hierarchy.groupby("City")
            }
        except Exception as e:
            print(f"[ERROR] 构建地理位置地图失败: {e}。")

    df_base_subset = None
    if base_data_file.exists():
        try:
            df_base = pd.read_excel(base_data_file, sheet_name="inter-city_routes")
            df_base["路线_std"] = (
                df_base["寄出城市"].str.replace("市", "", regex=False)
                + "-"
                + df_base["寄达城市"].str.replace("市", "", regex=False)
            )
            df_base_subset = df_base[
                ["路线_std", "公里", "寄出省份", "寄达省份"]
            ].drop_duplicates("路线_std")
        except Exception as e:
            print(f"[WARNING] 处理 'basic_data.xlsx' 出错: {e}。")

    year = period[:4]
    month = int(period[4:])  # '202506' -> 6, 以匹配 '6月' 而非 '06月'
    anjian_filename = f"{year}年{month}月{company_name}抽样.xlsx"
    anjian_file_path = anjian_dir / anjian_filename

    if not anjian_file_path.exists():
        print(f"[ERROR] 安监数据文件未找到，流程中止: {anjian_file_path.name}")
        return

    print(f"  -> 正在读取安监数据: {anjian_file_path.name}")
    try:
        all_anjian_df = pd.read_excel(
            anjian_file_path, dtype={"单号": str, "寄出城市": str, "寄达城市": str}
        )
    except Exception as e:
        print(f"[ERROR] 读取安监文件 '{anjian_filename}' 时出错: {e}")
        return
    # -----------------------------------------------------------------

    if "快递公司" in all_anjian_df.columns:
        all_anjian_df.rename(columns={"快递公司": "企业"}, inplace=True)

    # 【新增】准备安监数据源
    df_anjian_jitu = all_anjian_df[
        all_anjian_df["企业"] == config["anjian_map_key"]
    ].copy()
    if not df_anjian_jitu.empty:
        print(
            f"  -> 已从安监数据中筛选出 {len(df_anjian_jitu)} 条 {company_name} (JT) 记录。"
        )
        df_anjian_jitu.rename(
            columns={
                "揽收时间": "揽收时间_anjian",
                "离开寄件城市时间": "离开寄件城市时间_anjian",
                "到达收件城市时间": "到达收件城市时间_anjian",
                "派送时间": "派送时间_anjian",
                "签收时间": "签收时间_anjian",
            },
            inplace=True,
        )
        # 统一安监数据中的企业名称，以便后续合并
        df_anjian_jitu["企业"] = config["internal_name"]

    # --- 猪猪云数据处理 (保持不变) ---
    company_file = zhuzhuyun_merge_path / config["zhuzhu_filename"]
    if not company_file.exists():
        print(f"[WARNING] 未找到文件: {config['zhuzhu_filename']}，已跳过。")
        return
    df_company_zhu = pd.read_excel(company_file, dtype={"快递单号": str})
    df_company_zhu.rename(columns={"快递单号": "单号"}, inplace=True)
    df_company_zhu["企业"] = config["internal_name"]
    df_company_zhu["单号"] = df_company_zhu["单号"].str.strip()
    base_info_df = all_anjian_df[all_anjian_df["企业"] == config["anjian_map_key"]][
        ["单号", "企业", "寄出城市", "寄达城市"]
    ].drop_duplicates("单号")
    base_info_df["企业"] = config["internal_name"]  # 统一企业名
    df_merged = pd.merge(df_company_zhu, base_info_df, on=["单号", "企业"], how="left")
    base_profile = COMPANY_PROFILES_JITU["default"].copy()
    company_specific_profile = COMPANY_PROFILES_JITU.get(config["internal_name"], {})
    profile = base_profile.copy()
    for key, value in company_specific_profile.items():
        if (
            key in profile
            and value
            and key not in ["center_exclude", "dest_arrival_exclude"]
        ):
            profile[key] = f"{value}|{profile[key]}"
        elif value:
            profile[key] = value
    extracted_df = extractor_apply_based(
        df_merged, profile, location_maps, config["parser"]
    )
    df_zhuzhuyun_final = pd.concat([df_merged, extracted_df], axis=1)

    # 【新增】整合两大数据源
    zhuzhuyun_cols_to_keep = [
        "单号",
        "企业",
        "寄出城市",
        "寄达城市",
        "完整物流信息",
    ] + [col for col in df_zhuzhuyun_final if "_zhuzhuyun" in col]
    df_zhuzhuyun_subset = df_zhuzhuyun_final[zhuzhuyun_cols_to_keep]
    anjian_cols_to_keep = ["单号", "企业"] + [
        col for col in df_anjian_jitu if "_anjian" in col
    ]
    df_anjian_subset = df_anjian_jitu[anjian_cols_to_keep]
    df_final = pd.merge(
        df_zhuzhuyun_subset, df_anjian_subset, on=["单号", "企业"], how="outer"
    )
    print(f"  -> 已整合安监与猪猪云数据，共 {len(df_final)} 条唯一记录。")

    print("  -> 根据全局配置选择时间戳来源...")
    source_config = DATA_SOURCE_CONFIG.get(company_name, {})
    if not source_config:
        raise ValueError(
            f"错误: 在 DATA_SOURCE_CONFIG 中未找到 '{company_name}' 的配置!"
        )
    timestamp_map = {
        "揽收时间": "collection_time_source",
        "离开寄件城市时间": "departure_time_source",
        "到达收件城市时间": "arrival_time_source",
        "派送时间": "dispatch_time_source",
        "签收时间": "signed_time_source",
    }
    for final_col, config_key in timestamp_map.items():
        source_type = source_config.get(config_key, "anjian")
        primary_col = f"{final_col}_{source_type}"
        fallback_col = (
            f"{final_col}_{'zhuzhuyun' if source_type == 'anjian' else 'anjian'}"
        )
        df_final[final_col] = df_final.get(primary_col)
        if fallback_col in df_final:
            df_final[final_col] = df_final[final_col].fillna(df_final[fallback_col])
        print(f"    - '{final_col}' 已配置使用 '{source_type}' 数据源。")
    df_final["到达分拣中心时间"] = df_final.get("到达分拣中心时间_zhuzhuyun")
    df_final["离开收件城市分拣中心时间"] = df_final.get(
        "离开收件城市分拣中心时间_zhuzhuyun"
    )
    # --------------------------------------------------------------------------------------

    # --- 后续处理与保存 (保持不变) ---
    if df_base_subset is not None:
        # 填充可能因 outer merge 产生的缺失城市信息
        if "寄出城市_x" in df_final.columns and "寄出城市_y" in df_final.columns:
            df_final["寄出城市"] = df_final["寄出城市_x"].fillna(df_final["寄出城市_y"])
            df_final["寄达城市"] = df_final["寄达城市_x"].fillna(df_final["寄达城市_y"])
        df_final["路线_std"] = (
            df_final["寄出城市"].astype(str).str.replace("市", "", regex=False)
            + "-"
            + df_final["寄达城市"].astype(str).str.replace("市", "", regex=False)
        )
        df_final = pd.merge(df_final, df_base_subset, on="路线_std", how="left")
    df_final["企业"] = company_name
    output_columns = [
        "企业",
        "单号",
        "寄出城市",
        "寄达城市",
        "揽收时间",
        "离开寄件城市时间",
        "到达收件城市时间",
        "派送时间",
        "签收时间",
        "公里",
        "寄出省份",
        "寄达省份",
        "到达分拣中心时间",
        "离开收件城市分拣中心时间",
        "完整物流信息",
    ]
    final_df_to_save = df_final.reindex(columns=output_columns)
    output_file = output_dir / f"{company_name}_logistics_data_{period}.xlsx"
    final_df_to_save.to_excel(output_file, index=False)
    company_end_time = perf_counter()
    print(
        f"✅ {company_name} 处理完成，耗时 {company_end_time - company_start_time:.2f} 秒。文件已保存至: {output_file.name}"
    )


# --- 运行: 极兔 (保持不变) ---
if "period" in locals() and "DATA_SOURCE_CONFIG" in locals():
    process_jitu_data(
        zhuzhuyun_merge_path,
        anjian_data_path,
        pycharm_input_path,
        base_data_path,
        period,
    )
else:
    print(
        "[警告] 未定义 'period' 或 'DATA_SOURCE_CONFIG' 变量，跳过 process_jitu_data 的执行。请在前面的单元格中定义这些变量。"
    )

--- [开始] 处理 极兔 ---
  -> 正在读取安监数据: 2025年6月极兔抽样.xlsx
  -> 已从安监数据中筛选出 81703 条 极兔 (JT) 记录。
  -> 使用 'parse_logistics_events_jitu' 解析器通过 .apply() 运行...
  -> 已整合安监与猪猪云数据，共 81703 条唯一记录。
  -> 根据全局配置选择时间戳来源...
    - '揽收时间' 已配置使用 'anjian' 数据源。
    - '离开寄件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '到达收件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '派送时间' 已配置使用 'zhuzhuyun' 数据源。
    - '签收时间' 已配置使用 'zhuzhuyun' 数据源。
✅ 极兔 处理完成，耗时 52.73 秒。文件已保存至: 极兔_logistics_data_202506.xlsx


In [33]:
# --------------------------------------------------
# Cell 4.5: 韵达logistics数据提取(已修改)
# --------------------------------------------------
import re
import warnings
from pathlib import Path
from time import perf_counter

import pandas as pd

# --- 初始化 (仅在独立运行此单元格时需要) ---
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("--- [开始] 处理 韵达 ---")

# --- 项目路径设置 (请确保在Cell 0中已正确设置) ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
temp_path = report_path / "temp"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"


# --- 通用辅助函数 (保持不变) ---
def extractor_apply_based(
    df: pd.DataFrame, profile: dict, location_maps: dict, parser_func
) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame()
    print(f"  -> 使用 '{parser_func.__name__}' 解析器通过 .apply() 运行...")
    return df.apply(lambda row: parser_func(row, profile, location_maps), axis=1)


# --- 公司档案: 韵达 (修正“网格仓”定义) ---
COMPANY_PROFILES_YUNDA = {
    "default": {
        "collect": r"揽收|已收取|揽件|已取件",
        "delivery": r"派送中|派件员|为您派送|正在为您派件|已安排派送",
        "leave": r"离开|已发出|发往",
        "arrive": r"到达|抵达",
        # <<< 修正点1 >>>：将“网格仓”提升为一种通用的中心类型
        "center": r"处理中心|分拨中心|转运场|运营区|枢纽中心|集散中心|网格仓",
        "sign_p1": r"已暂存至|已投至|已到站|自提柜|智能柜|菜鸟驿站|邮政营业网点",
        "sign_p2": r"本人签收|他人代签收|已代收|放至家门口|指定位置签收",
        "sign_fallback": r"已签收|已妥投|已妥收|已投递",
    },
    "韵达": {
        "center": r"分拨交付中心",
        # <<< 修正点2 >>>：从严格排除列表中移除“网格仓”，因为它现在是中心了
        "center_exclude": r"公司|分部|服务部|揽投部|营业部|经营分部|网点|集货点|营销中心",
        # 宽松标准：用于识别“到达收件城市”这一首个落地动作
        "dest_arrival_exclude": r"服务部",  # 根据案例，仅排除“服务部”
        "sign_p2": r"已送货上门签收|已由邮政派送签收",
        "sign_fallback": r"快件已投递|快件已按址投递",
    },
}


# --- 解析器: 韵达 (分场景调用不同排除标准) (保持不变, 仅为统一命名规范修改后缀) ---
def parse_logistics_events_yunda(
    row: pd.Series, profile: dict, location_maps: dict
) -> pd.Series:
    log_text = row["完整物流信息"]
    sender_city_key, dest_city_key = row["寄出城市"], row["寄达城市"]
    # 【改动】为统一命名规范，将列后缀从 _zzy 改为 _zhuzhuyun
    time_cols = [
        "揽收时间_zhuzhuyun",
        "离开寄件城市时间_zhuzhuyun",
        "到达收件城市时间_zhuzhuyun",
        "派送时间_zhuzhuyun",
        "签收时间_zhuzhuyun",
        "到达分拣中心时间_zhuzhuyun",
        "离开收件城市分拣中心时间_zhuzhuyun",
    ]
    extracted_times = {col: pd.NaT for col in time_cols}
    if not isinstance(log_text, str) or not log_text.strip():
        return pd.Series(extracted_times)
    # --- 您其他的解析器逻辑保持完全不变 ---
    kw_collect, kw_delivery = profile.get("collect"), profile.get("delivery")
    kw_leave, kw_arrive = profile.get("leave"), profile.get("arrive")
    kw_sign_p1, kw_sign_p2, kw_sign_fallback, kw_sign_ignore = (
        profile.get("sign_p1"),
        profile.get("sign_p2"),
        profile.get("sign_fallback"),
        profile.get("sign_ignore"),
    )
    kw_all_centers = profile.get("center")
    kw_center_exclude = profile.get("center_exclude", "")
    kw_dest_arrival_exclude = profile.get("dest_arrival_exclude", "")
    datetime_capture_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    all_events = []
    for line in log_text.strip().split("\n"):
        match = re.search(datetime_capture_pattern, line)
        if match:
            dt = pd.to_datetime(match.group(1), errors="coerce")
            if pd.notna(dt):
                all_events.append({"dt": dt, "line": line})
    if not all_events:
        return pd.Series(extracted_times)
    all_events.sort(key=lambda x: x["dt"])

    def get_location_pattern(city_key, location_maps):
        if not city_key or not isinstance(city_key, str) or pd.isna(city_key):
            return None
        aliases = location_maps["city_alias_map"].get(
            city_key, [city_key.replace("市", "")]
        )
        clean_aliases = {
            str(a)
            for a in aliases
            if a is not None and pd.notna(a) and str(a).strip() != ""
        }
        if not clean_aliases:
            return None
        return "|".join(map(re.escape, clean_aliases))

    for event in all_events:
        if kw_collect and re.search(kw_collect, event["line"]):
            extracted_times["揽收时间_zhuzhuyun"] = event["dt"]
            break
    for event in all_events:
        if kw_delivery and re.search(kw_delivery, event["line"]):
            extracted_times["派送时间_zhuzhuyun"] = event["dt"]
            break
    p1_event, p2_event, p3_event = None, None, None
    for event in all_events:
        if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
            continue
        if kw_sign_p1 and re.search(kw_sign_p1, event["line"]):
            p1_event = event
            break
    for event in reversed(all_events):
        if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
            continue
        if kw_sign_p2 and re.search(kw_sign_p2, event["line"]):
            p2_event = event
            break
    for event in reversed(all_events):
        if kw_sign_ignore and re.search(kw_sign_ignore, event["line"]):
            continue
        if kw_sign_fallback and re.search(kw_sign_fallback, event["line"]):
            p3_event = event
            break
    if p1_event:
        extracted_times["签收时间_zhuzhuyun"] = p1_event["dt"]
    elif p2_event:
        extracted_times["签收时间_zhuzhuyun"] = p2_event["dt"]
    elif p3_event:
        extracted_times["签收时间_zhuzhuyun"] = p3_event["dt"]
    sender_pattern = get_location_pattern(sender_city_key, location_maps)
    if (
        pd.notna(extracted_times["揽收时间_zhuzhuyun"])
        and sender_pattern
        and kw_all_centers
    ):
        last_sender_leave_event = None
        for event in all_events:
            if event["dt"] > extracted_times["揽收时间_zhuzhuyun"]:
                action_part = re.split(r"；|发往", event["line"])[0]
                if (
                    kw_leave
                    and re.search(kw_leave, action_part)
                    and re.search(kw_all_centers, action_part)
                    and re.search(sender_pattern, action_part)
                    and not (
                        kw_center_exclude and re.search(kw_center_exclude, action_part)
                    )
                ):
                    last_sender_leave_event = event
        if last_sender_leave_event:
            extracted_times["离开寄件城市时间_zhuzhuyun"] = last_sender_leave_event[
                "dt"
            ]
        last_sender_center_arrival = None
        search_end_time = extracted_times.get(
            "离开寄件城市时间_zhuzhuyun", all_events[-1]["dt"] + pd.Timedelta(seconds=1)
        )
        for event in all_events:
            if not (
                extracted_times["揽收时间_zhuzhuyun"] < event["dt"] < search_end_time
            ):
                continue
            action_part = re.split(r"；|发往", event["line"])[0]
            if (
                kw_arrive
                and re.search(kw_arrive, action_part)
                and re.search(kw_all_centers, action_part)
                and re.search(sender_pattern, action_part)
                and not (
                    kw_center_exclude and re.search(kw_center_exclude, action_part)
                )
            ):
                last_sender_center_arrival = event
        if last_sender_center_arrival:
            extracted_times["到达分拣中心时间_zhuzhuyun"] = last_sender_center_arrival[
                "dt"
            ]
    dest_pattern = get_location_pattern(dest_city_key, location_maps)
    if dest_pattern:
        if pd.isna(extracted_times["到达收件城市时间_zhuzhuyun"]):
            for event in all_events:
                action_part = re.split(r"；|发往", event["line"])[0]
                if (
                    kw_arrive
                    and re.search(kw_arrive, action_part)
                    and re.search(dest_pattern, action_part)
                ):
                    if not (
                        kw_dest_arrival_exclude
                        and re.search(kw_dest_arrival_exclude, action_part)
                    ):
                        extracted_times["到达收件城市时间_zhuzhuyun"] = event["dt"]
                        break
        if pd.notna(extracted_times["派送时间_zhuzhuyun"]):
            for event in reversed(all_events):
                if event["dt"] < extracted_times["派送时间_zhuzhuyun"]:
                    action_part = re.split(r"；|发往", event["line"])[0]
                    if (
                        kw_leave
                        and re.search(kw_leave, action_part)
                        and kw_all_centers
                        and re.search(kw_all_centers, action_part)
                        and re.search(dest_pattern, action_part)
                        and not (
                            kw_center_exclude
                            and re.search(kw_center_exclude, action_part)
                        )
                    ):
                        extracted_times["离开收件城市分拣中心时间_zhuzhuyun"] = event[
                            "dt"
                        ]
                        break
    return pd.Series(extracted_times)


# --- 主流程: 韵达 (已修改数据读取逻辑) ---
def process_yunda_data(zhuzhuyun_dir, anjian_dir, output_dir, base_data_file, period):
    company_name = "韵达"
    company_start_time = perf_counter()
    config = {
        "zhuzhu_filename": f"韵达_{period}.xlsx",
        "internal_name": "韵达",
        "anjian_map_key": "YUNDA",
        "parser": parse_logistics_events_yunda,
    }

    location_maps = {}
    if base_data_file.exists():
        try:
            df_hierarchy = pd.read_excel(
                base_data_file,
                sheet_name="city_hierarchy",
                dtype={"Province": str, "City": str, "District": str},
            )
            df_hierarchy.dropna(subset=["Province", "City"], inplace=True)
            df_hierarchy["City_clean"] = df_hierarchy["City"].str.replace(
                "市", "", regex=False
            )
            df_hierarchy["District_clean"] = df_hierarchy["District"].str.replace(
                "市", "", regex=False
            )
            location_maps["city_alias_map"] = {
                city: [group.iloc[0]["City_clean"]]
                + group["District_clean"].unique().tolist()
                for city, group in df_hierarchy.groupby("City")
            }
        except Exception as e:
            print(f"[ERROR] 构建地理位置地图失败: {e}。")

    df_base_subset = None
    if base_data_file.exists():
        try:
            df_base = pd.read_excel(base_data_file, sheet_name="inter-city_routes")
            df_base["路线_std"] = (
                df_base["寄出城市"].str.replace("市", "", regex=False)
                + "-"
                + df_base["寄达城市"].str.replace("市", "", regex=False)
            )
            df_base_subset = df_base[
                ["路线_std", "公里", "寄出省份", "寄达省份"]
            ].drop_duplicates("路线_std")
        except Exception as e:
            print(f"[WARNING] 处理 'basic_data.xlsx' 出错: {e}。")

    year = period[:4]
    month = int(period[4:])  # '202506' -> 6, 以匹配 '6月' 而非 '06月'
    anjian_filename = f"{year}年{month}月{company_name}抽样.xlsx"
    anjian_file_path = anjian_dir / anjian_filename

    if not anjian_file_path.exists():
        print(f"[ERROR] 安监数据文件未找到，流程中止: {anjian_file_path.name}")
        return

    print(f"  -> 正在读取安监数据: {anjian_file_path.name}")
    try:
        all_anjian_df = pd.read_excel(
            anjian_file_path, dtype={"单号": str, "寄出城市": str, "寄达城市": str}
        )
    except Exception as e:
        print(f"[ERROR] 读取安监文件 '{anjian_filename}' 时出错: {e}")
        return
    # -----------------------------------------------------------------

    if "快递公司" in all_anjian_df.columns:
        all_anjian_df.rename(columns={"快递公司": "企业"}, inplace=True)

    # 【新增】准备安监数据源
    df_anjian_yunda = all_anjian_df[
        all_anjian_df["企业"] == config["anjian_map_key"]
    ].copy()
    if not df_anjian_yunda.empty:
        print(
            f"  -> 已从安监数据中筛选出 {len(df_anjian_yunda)} 条 {company_name} (YUNDA) 记录。"
        )
        df_anjian_yunda.rename(
            columns={
                "揽收时间": "揽收时间_anjian",
                "离开寄件城市时间": "离开寄件城市时间_anjian",
                "到达收件城市时间": "到达收件城市时间_anjian",
                "派送时间": "派送时间_anjian",
                "签收时间": "签收时间_anjian",
            },
            inplace=True,
        )
        # 统一安监数据中的企业名称
        df_anjian_yunda["企业"] = config["internal_name"]

    # --- 猪猪云数据处理 (保持不变) ---
    company_file = zhuzhuyun_merge_path / config["zhuzhu_filename"]
    if not company_file.exists():
        print(f"[WARNING] 未找到文件: {config['zhuzhu_filename']}，已跳过。")
        return
    df_company_zhu = pd.read_excel(company_file, dtype={"快递单号": str})
    df_company_zhu.rename(columns={"快递单号": "单号"}, inplace=True)
    df_company_zhu["企业"] = config["internal_name"]
    df_company_zhu["单号"] = df_company_zhu["单号"].str.strip()
    base_info_df = all_anjian_df[all_anjian_df["企业"] == config["anjian_map_key"]][
        ["单号", "企业", "寄出城市", "寄达城市"]
    ].drop_duplicates("单号")
    base_info_df["企业"] = config["internal_name"]
    df_merged = pd.merge(df_company_zhu, base_info_df, on=["单号", "企业"], how="left")
    base_profile = COMPANY_PROFILES_YUNDA["default"].copy()
    company_specific_profile = COMPANY_PROFILES_YUNDA.get(config["internal_name"], {})
    profile = base_profile.copy()
    for key, value in company_specific_profile.items():
        if (
            key in profile
            and value
            and key not in ["center_exclude", "dest_arrival_exclude"]
        ):
            profile[key] = f"{value}|{profile[key]}"
        elif value:
            profile[key] = value
    extracted_df = extractor_apply_based(
        df_merged, profile, location_maps, config["parser"]
    )
    df_zhuzhuyun_final = pd.concat([df_merged, extracted_df], axis=1)

    # 【新增】整合两大数据源
    zhuzhuyun_cols_to_keep = [
        "单号",
        "企业",
        "寄出城市",
        "寄达城市",
        "完整物流信息",
    ] + [col for col in df_zhuzhuyun_final if "_zhuzhuyun" in col]
    df_zhuzhuyun_subset = df_zhuzhuyun_final[zhuzhuyun_cols_to_keep]
    anjian_cols_to_keep = ["单号", "企业"] + [
        col for col in df_anjian_yunda if "_anjian" in col
    ]
    df_anjian_subset = df_anjian_yunda[anjian_cols_to_keep]
    df_final = pd.merge(
        df_zhuzhuyun_subset, df_anjian_subset, on=["单号", "企业"], how="outer"
    )
    print(f"  -> 已整合安监与猪猪云数据，共 {len(df_final)} 条唯一记录。")

    print("  -> 根据全局配置选择时间戳来源...")
    source_config = DATA_SOURCE_CONFIG.get(company_name, {})
    if not source_config:
        raise ValueError(
            f"错误: 在 DATA_SOURCE_CONFIG 中未找到 '{company_name}' 的配置!"
        )
    timestamp_map = {
        "揽收时间": "collection_time_source",
        "离开寄件城市时间": "departure_time_source",
        "到达收件城市时间": "arrival_time_source",
        "派送时间": "dispatch_time_source",
        "签收时间": "signed_time_source",
    }
    for final_col, config_key in timestamp_map.items():
        source_type = source_config.get(config_key, "anjian")
        primary_col = f"{final_col}_{source_type}"
        fallback_col = (
            f"{final_col}_{'zhuzhuyun' if source_type == 'anjian' else 'anjian'}"
        )
        df_final[final_col] = df_final.get(primary_col)
        if fallback_col in df_final:
            df_final[final_col] = df_final[final_col].fillna(df_final[fallback_col])
        print(f"    - '{final_col}' 已配置使用 '{source_type}' 数据源。")
    df_final["到达分拣中心时间"] = df_final.get("到达分拣中心时间_zhuzhuyun")
    df_final["离开收件城市分拣中心时间"] = df_final.get(
        "离开收件城市分拣中心时间_zhuzhuyun"
    )
    # --------------------------------------------------------------------------------------

    # --- 后续处理与保存 (保持不变) ---
    if df_base_subset is not None:
        if "寄出城市_x" in df_final.columns and "寄出城市_y" in df_final.columns:
            df_final["寄出城市"] = df_final["寄出城市_x"].fillna(df_final["寄出城市_y"])
            df_final["寄达城市"] = df_final["寄达城市_x"].fillna(df_final["寄达城市_y"])
        df_final["路线_std"] = (
            df_final["寄出城市"].astype(str).str.replace("市", "", regex=False)
            + "-"
            + df_final["寄达城市"].astype(str).str.replace("市", "", regex=False)
        )
        df_final = pd.merge(df_final, df_base_subset, on="路线_std", how="left")
    df_final["企业"] = company_name
    output_columns = [
        "企业",
        "单号",
        "寄出城市",
        "寄达城市",
        "揽收时间",
        "离开寄件城市时间",
        "到达收件城市时间",
        "派送时间",
        "签收时间",
        "公里",
        "寄出省份",
        "寄达省份",
        "到达分拣中心时间",
        "离开收件城市分拣中心时间",
        "完整物流信息",
    ]
    final_df_to_save = df_final.reindex(columns=output_columns)
    output_file = output_dir / f"{company_name}_logistics_data_{period}.xlsx"
    final_df_to_save.to_excel(output_file, index=False)
    company_end_time = perf_counter()
    print(
        f"✅ {company_name} 处理完成，耗时 {company_end_time - company_start_time:.2f} 秒。文件已保存至: {output_file.name}"
    )


# --- 运行: 韵达 (保持不变) ---
if "period" in locals() and "DATA_SOURCE_CONFIG" in locals():
    process_yunda_data(
        zhuzhuyun_merge_path,
        anjian_data_path,
        pycharm_input_path,
        base_data_path,
        period,
    )
else:
    print(
        "[警告] 未定义 'period' 或 'DATA_SOURCE_CONFIG' 变量，跳过 process_yunda_data 的执行。请在前面的单元格中定义这些变量。"
    )

--- [开始] 处理 韵达 ---
  -> 正在读取安监数据: 2025年6月韵达抽样.xlsx
  -> 已从安监数据中筛选出 81797 条 韵达 (YUNDA) 记录。
  -> 使用 'parse_logistics_events_yunda' 解析器通过 .apply() 运行...
  -> 已整合安监与猪猪云数据，共 81797 条唯一记录。
  -> 根据全局配置选择时间戳来源...
    - '揽收时间' 已配置使用 'anjian' 数据源。
    - '离开寄件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '到达收件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '派送时间' 已配置使用 'zhuzhuyun' 数据源。
    - '签收时间' 已配置使用 'zhuzhuyun' 数据源。
✅ 韵达 处理完成，耗时 51.89 秒。文件已保存至: 韵达_logistics_data_202506.xlsx


In [35]:
# --------------------------------------------------
# Cell 4.6: 德邦logistics数据提取
# --------------------------------------------------
import re
import warnings
from pathlib import Path
from time import perf_counter
from typing import Any, Dict, List, Optional

import pandas as pd

# --- 初始化 ---
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("--- [开始] 处理 德邦 ---")

# --- 项目路径设置 (请确保路径正确) ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
temp_path = report_path / "temp"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"

# --- 全局预编译正则表达式 (保持不变) ---
DATETIME_CAPTURE_PATTERN = re.compile(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})")

# --- 公司档案: 德邦 (保持不变) ---
COMPANY_PROFILES_DEPPON = {
    "default": {
        "collect": r"揽收|已收取|揽件|已取件",
        "delivery": r"派送中|为您派送|正在为您派件|已安排派送",
        "leave": r"离开|已发出|发往|航班起飞",
        "arrive": r"到达|抵达|航班到达",
        "center": r"处理中心|分拨中心|转运场|转运中心|运营区|枢纽中心|集散中心|分拨站|空运总调|机场运作部|运作部|集配站",
        "exclude": r"揽投部|营业部|经营分部|网点|集货点|营销中心",
        "sign_p1": r"已派送至|已投至|已到站|自提柜|智能柜|菜鸟驿站|邮政营业网点|包裹已存放至",
        "sign_p2": r"本人签收|他人代签收|已代收|放至家门口|指定位置签收",
        "sign_p3": r"DONT_MATCH_ANYTHING",
        "sign_fallback": r"已签收|已妥投|已妥收",
    },
    "德邦": {
        "sign_p1": r"快件已暂存至|包裹已存放至|集运仓签收|菜鸟驿站签收|水电表箱签收|丰巢柜|妈妈驿站",
        "sign_p2": r"家门口签收|本人签收|代收|已由同事签收|收发室签收|前台签收|亲属签收|其他签收|门卫签收|物业签收|正常签收",
        "sign_p3": r"经收货人同意，此件放置在",
        "exclude": r"经营分部",
    },
}


# --- 解析器辅助函数 (保持不变) ---
def get_location_pattern(city_key: Optional[str], location_maps: Dict) -> Optional[str]:
    if not city_key or not isinstance(city_key, str):
        return None
    aliases = set(
        location_maps["city_alias_map"].get(city_key, [city_key.replace("市", "")])
    )
    if city_key in location_maps["capital_cities_set"]:
        province = location_maps["city_to_province_map"].get(city_key)
        if province:
            aliases.add(province.replace("省", "").replace("市", ""))
    valid_aliases = {alias for alias in aliases if alias}
    return "|".join(map(re.escape, valid_aliases))


# --- 主解析器: 德邦 (最终修正版) (保持不变, 仅为统一命名规范修改后缀) ---
def parse_logistics_events_deppon_final_logic(
    row: pd.Series, profile: Dict, location_maps: Dict
) -> pd.Series:
    log_text, sender_city, dest_city = (
        row["完整物流信息"],
        row["寄出城市"],
        row["寄达城市"],
    )
    # 【改动】为统一命名规范，将列后缀从 _zzy 改为 _zhuzhuyun
    time_cols = [
        "揽收时间_zhuzhuyun",
        "离开寄件城市时间_zhuzhuyun",
        "到达收件城市时间_zhuzhuyun",
        "派送时间_zhuzhuyun",
        "签收时间_zhuzhuyun",
        "到达分拣中心时间_zhuzhuyun",
        "离开收件城市分拣中心时间_zhuzhuyun",
    ]
    extracted_times = {col: pd.NaT for col in time_cols}
    if not isinstance(log_text, str) or not log_text.strip():
        return pd.Series(extracted_times)
    # --- 您其他的解析器逻辑保持完全不变 ---
    all_events = []
    for line in log_text.strip().split("\n"):
        match = DATETIME_CAPTURE_PATTERN.search(line)
        if match:
            dt = pd.to_datetime(match.group(1), errors="coerce")
            if pd.notna(dt):
                all_events.append({"dt": dt, "line": line})
    if not all_events:
        return pd.Series(extracted_times)
    all_events.sort(key=lambda x: x["dt"])
    collect_event = next(
        (e for e in all_events if re.search(profile.get("collect"), e["line"])), None
    )
    if collect_event:
        extracted_times["揽收时间_zhuzhuyun"] = collect_event["dt"]
    delivery_event = next(
        (e for e in all_events if re.search(profile.get("delivery"), e["line"])), None
    )
    if delivery_event:
        extracted_times["派送时间_zhuzhuyun"] = delivery_event["dt"]
    sign_event = None
    patterns_to_check = ["sign_p1", "sign_p2", "sign_p3", "sign_fallback"]
    for pattern_key in patterns_to_check:
        pattern = profile.get(pattern_key)
        if pattern:
            sign_event = next(
                (e for e in reversed(all_events) if re.search(pattern, e["line"])), None
            )
        if sign_event:
            break
    if sign_event:
        extracted_times["签收时间_zhuzhuyun"] = sign_event["dt"]
    sender_pattern = get_location_pattern(sender_city, location_maps)
    dest_pattern = get_location_pattern(dest_city, location_maps)
    kw_center, kw_leave, kw_arrive, kw_exclude = (
        profile.get("center"),
        profile.get("leave"),
        profile.get("arrive"),
        profile.get("exclude", "DONT_MATCH_ANYTHING"),
    )
    if pd.notna(extracted_times["揽收时间_zhuzhuyun"]) and sender_pattern:
        true_leave_event = None
        for event in reversed(all_events):
            if event["dt"] > extracted_times["揽收时间_zhuzhuyun"]:
                if re.search(kw_leave, event["line"]) and re.search(
                    sender_pattern, event["line"]
                ):
                    if not re.search(kw_exclude, event["line"]):
                        true_leave_event = event
                        break
        if true_leave_event:
            extracted_times["离开寄件城市时间_zhuzhuyun"] = true_leave_event["dt"]
        window_end = true_leave_event["dt"] if true_leave_event else pd.Timestamp.max
        last_arrival_in_origin = None
        for event in all_events:
            if not (extracted_times["揽收时间_zhuzhuyun"] < event["dt"] < window_end):
                continue
            if (
                re.search(kw_arrive, event["line"])
                and not re.search(kw_leave, event["line"])
                and re.search(kw_center, event["line"])
                and re.search(sender_pattern, event["line"])
                and not re.search(kw_exclude, event["line"])
            ):
                last_arrival_in_origin = event
        if last_arrival_in_origin:
            extracted_times["到达分拣中心时间_zhuzhuyun"] = last_arrival_in_origin["dt"]
    if dest_pattern:
        arrive_dest_event = next(
            (
                e
                for e in all_events
                if (
                    re.search(kw_arrive, e["line"])
                    and re.search(dest_pattern, e["line"])
                    and not re.search(kw_leave, e["line"])
                )
            ),
            None,
        )
        if arrive_dest_event:
            extracted_times["到达收件城市时间_zhuzhuyun"] = arrive_dest_event["dt"]
    last_leave_dest_center = None
    if pd.notna(extracted_times["派送时间_zhuzhuyun"]) and dest_pattern and kw_center:
        leave_dest_center_pattern = re.compile(
            f"({kw_leave}).*?【(?P<location_name>.*?)】"
        )
        start_window = extracted_times.get(
            "到达收件城市时间_zhuzhuyun", pd.Timestamp.min
        )
        end_window = extracted_times["派送时间_zhuzhuyun"]
        for event in all_events:
            if not (start_window < event["dt"] < end_window):
                continue
            match = leave_dest_center_pattern.search(event["line"])
            if match:
                location_name = match.group("location_name")
                if (
                    re.search(dest_pattern, location_name)
                    and re.search(kw_center, location_name)
                    and not re.search(kw_exclude, location_name)
                ):
                    last_leave_dest_center = event
    if last_leave_dest_center:
        extracted_times["离开收件城市分拣中心时间_zhuzhuyun"] = last_leave_dest_center[
            "dt"
        ]
    return pd.Series(extracted_times)


# --- 通用辅助函数 (保持不变) ---
def extractor_apply_based(
    df: pd.DataFrame, profile: dict, location_maps: dict, parser_func
) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame()
    print(f"  -> 使用 '{parser_func.__name__}' 解析器通过 .apply() 运行...")
    return df.apply(lambda row: parser_func(row, profile, location_maps), axis=1)


# --- 主流程: 德邦 (已修改数据读取逻辑) ---
def process_deppon_data(
    zhuzhuyun_dir: Path,
    anjian_dir: Path,
    output_dir: Path,
    base_data_file: Path,
    period: str,
):
    company_name = "德邦"
    config = {
        "zhuzhu_filename": f"德邦_{period}.xlsx",
        "internal_name": "德邦",
        "anjian_map_key": "DEPPON",
        "parser": parse_logistics_events_deppon_final_logic,
    }
    company_start_time = perf_counter()

    location_maps = {
        "city_alias_map": {},
        "capital_cities_set": set(),
        "city_to_province_map": {},
    }
    try:
        if base_data_file.exists():
            df_hierarchy = pd.read_excel(base_data_file, sheet_name="city_hierarchy")
            df_hierarchy.dropna(subset=["Province", "City", "District"], inplace=True)
            df_hierarchy["City_clean"] = df_hierarchy["City"].str.replace(
                "市", "", regex=False
            )
            df_hierarchy["District_clean"] = df_hierarchy["District"].str.replace(
                r"市|区|县", "", regex=True
            )
            for city, group in df_hierarchy.groupby("City"):
                aliases = [group.iloc[0]["City_clean"]] + group[
                    "District_clean"
                ].unique().tolist()
                location_maps["city_alias_map"][city] = [a for a in aliases if a]
            if "IsCapital" in df_hierarchy.columns:
                df_capitals = df_hierarchy[df_hierarchy["IsCapital"] == 1]
                location_maps["capital_cities_set"] = set(df_capitals["City"].unique())
            city_to_province = df_hierarchy.drop_duplicates("City").set_index("City")[
                "Province"
            ]
            location_maps["city_to_province_map"] = city_to_province.to_dict()
    except Exception as e:
        print(f"[ERROR] 构建地理位置地图失败: {e}。")

    df_base_subset = None
    try:
        if base_data_file.exists():
            df_base = pd.read_excel(base_data_file, sheet_name="inter-city_routes")
            df_base["路线_std"] = (
                df_base["寄出城市"].str.replace("市", "", regex=False)
                + "-"
                + df_base["寄达城市"].str.replace("市", "", regex=False)
            )
            df_base_subset = df_base[
                ["路线_std", "公里", "寄出省份", "寄达省份"]
            ].drop_duplicates("路线_std")
    except Exception as e:
        print(f"[WARNING] 处理 'basic_data.xlsx' 出错: {e}。")
    year = period[:4]
    month = int(period[4:])  # '202506' -> 6, 以匹配 '6月' 而非 '06月'
    anjian_filename = f"{year}年{month}月{company_name}抽样.xlsx"
    anjian_file_path = anjian_dir / anjian_filename

    if not anjian_file_path.exists():
        print(f"[ERROR] 安监数据文件未找到，流程中止: {anjian_file_path.name}")
        return

    print(f"  -> 正在读取安监数据: {anjian_file_path.name}")
    try:
        all_anjian_df = pd.read_excel(
            anjian_file_path, dtype={"单号": str, "寄出城市": str, "寄达城市": str}
        )
    except Exception as e:
        print(f"[ERROR] 读取安监文件 '{anjian_filename}' 时出错: {e}")
        return
    # -----------------------------------------------------------------

    if "快递公司" in all_anjian_df.columns:
        all_anjian_df.rename(columns={"快递公司": "企业"}, inplace=True)

    # 【新增】准备安监数据源
    df_anjian_deppon = all_anjian_df[
        all_anjian_df["企业"] == config["anjian_map_key"]
    ].copy()
    if not df_anjian_deppon.empty:
        print(
            f"  -> 已从安监数据中筛选出 {len(df_anjian_deppon)} 条 {company_name} (DEPPON) 记录。"
        )
        df_anjian_deppon.rename(
            columns={
                "揽收时间": "揽收时间_anjian",
                "离开寄件城市时间": "离开寄件城市时间_anjian",
                "到达收件城市时间": "到达收件城市时间_anjian",
                "派送时间": "派送时间_anjian",
                "签收时间": "签收时间_anjian",
            },
            inplace=True,
        )
        df_anjian_deppon["企业"] = config["internal_name"]  # 统一企业名

    # --- 猪猪云数据处理 ---
    company_file = zhuzhuyun_dir / config["zhuzhu_filename"]
    if not company_file.exists():
        print(f"[WARNING] 未找到文件: {config['zhuzhu_filename']}，已跳过。")
        return
    df_company_zhu = pd.read_excel(company_file, dtype={"快递单号": str})
    df_company_zhu.rename(
        columns={"快递单号": "单号", "快递公司": "企业"}, inplace=True
    )
    base_info_df = all_anjian_df[all_anjian_df["企业"] == config["anjian_map_key"]][
        ["单号", "企业", "寄出城市", "寄达城市"]
    ].drop_duplicates("单号")
    base_info_df["企业"] = config["internal_name"]  # 统一企业名，以匹配猪猪云
    df_merged = pd.merge(df_company_zhu, base_info_df, on=["单号", "企业"], how="left")
    base_profile = COMPANY_PROFILES_DEPPON["default"].copy()
    company_specific_profile = COMPANY_PROFILES_DEPPON.get(config["internal_name"], {})
    profile = {**base_profile, **company_specific_profile}
    extracted_df = extractor_apply_based(
        df_merged, profile, location_maps, config["parser"]
    )
    df_zhuzhuyun_final = pd.concat([df_merged, extracted_df], axis=1)

    # 【新增】整合两大数据源
    zhuzhuyun_cols_to_keep = [
        "单号",
        "企业",
        "寄出城市",
        "寄达城市",
        "完整物流信息",
    ] + [col for col in df_zhuzhuyun_final if "_zhuzhuyun" in col]
    df_zhuzhuyun_subset = df_zhuzhuyun_final[zhuzhuyun_cols_to_keep]
    anjian_cols_to_keep = ["单号", "企业"] + [
        col for col in df_anjian_deppon if "_anjian" in col
    ]
    df_anjian_subset = df_anjian_deppon[anjian_cols_to_keep]
    df_final = pd.merge(
        df_zhuzhuyun_subset, df_anjian_subset, on=["单号", "企业"], how="outer"
    )
    print(f"  -> 已整合安监与猪猪云数据，共 {len(df_final)} 条唯一记录。")
    print("  -> 根据全局配置选择时间戳来源...")
    source_config = DATA_SOURCE_CONFIG.get(company_name, {})
    if not source_config:
        raise ValueError(
            f"错误: 在 DATA_SOURCE_CONFIG 中未找到 '{company_name}' 的配置!"
        )
    timestamp_map = {
        "揽收时间": "collection_time_source",
        "离开寄件城市时间": "departure_time_source",
        "到达收件城市时间": "arrival_time_source",
        "派送时间": "dispatch_time_source",
        "签收时间": "signed_time_source",
    }
    for final_col, config_key in timestamp_map.items():
        source_type = source_config.get(config_key, "anjian")
        primary_col = f"{final_col}_{source_type}"
        fallback_col = (
            f"{final_col}_{'zhuzhuyun' if source_type == 'anjian' else 'anjian'}"
        )
        df_final[final_col] = df_final.get(primary_col)
        if fallback_col in df_final:
            df_final[final_col] = df_final[final_col].fillna(df_final[fallback_col])
        print(f"    - '{final_col}' 已配置使用 '{source_type}' 数据源。")
    df_final["到达分拣中心时间"] = df_final.get("到达分拣中心时间_zhuzhuyun")
    df_final["离开收件城市分拣中心时间"] = df_final.get(
        "离开收件城市分拣中心时间_zhuzhuyun"
    )
    # --------------------------------------------------------------------------------------

    # --- 后续处理与保存 (保持不变) ---
    if df_base_subset is not None:
        if "寄出城市_x" in df_final.columns and "寄出城市_y" in df_final.columns:
            df_final["寄出城市"] = df_final["寄出城市_x"].fillna(df_final["寄出城市_y"])
            df_final["寄达城市"] = df_final["寄达城市_x"].fillna(df_final["寄达城市_y"])
        df_final["路线_std"] = (
            df_final["寄出城市"].astype(str).str.replace("市", "", regex=False)
            + "-"
            + df_final["寄达城市"].astype(str).str.replace("市", "", regex=False)
        )
        df_final = pd.merge(df_final, df_base_subset, on="路线_std", how="left")
    df_final["企业"] = company_name
    output_columns = [
        "企业",
        "单号",
        "寄出城市",
        "寄达城市",
        "揽收时间",
        "离开寄件城市时间",
        "到达收件城市时间",
        "派送时间",
        "签收时间",
        "公里",
        "寄出省份",
        "寄达省份",
        "到达分拣中心时间",
        "离开收件城市分拣中心时间",
        "完整物流信息",
    ]
    final_df_to_save = df_final.reindex(columns=output_columns)
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / f"{company_name}_logistics_data_{period}.xlsx"
    final_df_to_save.to_excel(output_file, index=False)
    company_end_time = perf_counter()
    print(
        f"✅ {company_name} 处理完成，耗时 {company_end_time - company_start_time:.2f} 秒。文件已保存至: {output_file.name}"
    )


# --- 运行主流程 (保持不变) ---
if "period" in locals() and "DATA_SOURCE_CONFIG" in locals():
    process_deppon_data(
        zhuzhuyun_merge_path,
        anjian_data_path,
        pycharm_input_path,
        base_data_path,
        period,
    )
else:
    print(
        "[警告] 未定义 'period' 或 'DATA_SOURCE_CONFIG' 变量，跳过 process_deppon_data 的执行。请在前面的单元格中定义这些变量。"
    )

--- [开始] 处理 德邦 ---
  -> 正在读取安监数据: 2025年6月德邦抽样.xlsx
  -> 已从安监数据中筛选出 78551 条 德邦 (DEPPON) 记录。
  -> 使用 'parse_logistics_events_deppon_final_logic' 解析器通过 .apply() 运行...
  -> 已整合安监与猪猪云数据，共 78551 条唯一记录。
  -> 根据全局配置选择时间戳来源...
    - '揽收时间' 已配置使用 'anjian' 数据源。
    - '离开寄件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '到达收件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '派送时间' 已配置使用 'zhuzhuyun' 数据源。
    - '签收时间' 已配置使用 'zhuzhuyun' 数据源。
✅ 德邦 处理完成，耗时 55.42 秒。文件已保存至: 德邦_logistics_data_202506.xlsx


In [37]:
# --------------------------------------------------
# Cell 4.7: 申通logistics数据提取 (已修改)
# -------------------------------------------------
import re
import warnings
from pathlib import Path
from time import perf_counter

import pandas as pd

# --- 初始化 (仅在独立运行此单元格时需要) ---
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("--- [开始] 处理 申通 ---")

# --- 项目路径设置 (请确保在Cell 0中已正确设置) ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
temp_path = report_path / "temp"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"


# --- 通用辅助函数 (保持不变) ---
def extractor_apply_based(
    df: pd.DataFrame, profile: dict, location_maps: dict, parser_func
) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame()
    print(f"  -> 使用 '{parser_func.__name__}' 解析器通过 .apply() 运行...")
    return df.apply(lambda row: parser_func(row, profile, location_maps), axis=1)


# --- 公司档案: 申通 (保持不变) ---
COMPANY_PROFILES_STO = {
    "default": {
        "collect": r"揽收|已收取|揽件|已取件",
        "delivery": r"派送中|为您派送|正在为您派件|已安排派送|正在为您派送|正在派送途中",
        "leave": r"离开|已发出|发往",
        "arrive": r"到达|抵达",
        "center": r"转运中心|处理中心|分拨中心|转运场|运营区|枢纽中心|集散中心",
        "exclude": r"揽投部|营业部|经营分部|网点|集货点|营销中心",
        "sign_p1": r"已被.*?接收|已被.*?代收|已经妥投|已送达|作废处理|快件已到达\[.*?驿站\]|已抵达.*?公司|包裹已完成签收|已放入|已存放至|到达退货服务站点|被退回|超市|正在验收中|已送货上門|按地址投递|已放入.*?驿站|已投递|已抵达.*?服务点|已派送至|已投至|已到站|自提柜|智能柜|菜鸟驿站|邮政营业网点",
        "sign_p2": r"本人签收|他人代签收|已代收|放至家门口|指定位置签收",
        "sign_fallback": r"已签收|已妥投|已妥收",
    },
    "申通": {"sign_p2": r"已由【.*?】签收|已签收|代收"},
}


# --- 解析器: 申通 (最终定稿版) (保持不变, 仅为统一命名规范修改后缀) ---
def parse_logistics_events_sto(
    row: pd.Series, profile: dict, location_maps: dict
) -> pd.Series:
    log_text = row["完整物流信息"]
    sender_city_key, dest_city_key = row["寄出城市"], row["寄达城市"]
    # 【改动】为统一命名规范，将列后缀从 _zzy 改为 _zhuzhuyun
    time_cols = [
        "揽收时间_zhuzhuyun",
        "离开寄件城市时间_zhuzhuyun",
        "到达收件城市时间_zhuzhuyun",
        "派送时间_zhuzhuyun",
        "签收时间_zhuzhuyun",
        "到达分拣中心时间_zhuzhuyun",
        "离开收件城市分拣中心时间_zhuzhuyun",
    ]
    extracted_times = {col: pd.NaT for col in time_cols}
    if not isinstance(log_text, str) or not log_text.strip():
        return pd.Series(extracted_times)
    # --- 您其他的解析器逻辑保持完全不变 ---
    kw_collect, kw_delivery = profile.get("collect"), profile.get("delivery")
    kw_leave, kw_arrive = profile.get("leave"), profile.get("arrive")
    kw_sign_p1, kw_sign_p2, kw_sign_fallback = (
        profile.get("sign_p1"),
        profile.get("sign_p2"),
        profile.get("sign_fallback"),
    )
    datetime_capture_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    all_events = []
    for line in log_text.strip().split("\n"):
        match = re.search(datetime_capture_pattern, line)
        if match:
            dt = pd.to_datetime(match.group(1), errors="coerce")
            if pd.notna(dt):
                location_match = re.search(r"【(.*?)】", line)
                location = location_match.group(1) if location_match else None
                all_events.append({"dt": dt, "line": line, "location": location})
    if not all_events:
        return pd.Series(extracted_times)
    all_events.sort(key=lambda x: x["dt"])

    def get_location_pattern(city_key, location_maps, include_province=True):
        if not city_key or not isinstance(city_key, str) or not city_key.strip():
            return None
        aliases = {city_key.replace("市", "")}
        aliases.update(location_maps["city_alias_map"].get(city_key, []))
        if include_province:
            province = location_maps["city_to_province_map"].get(city_key)
            if province:
                aliases.add(province.replace("省", "").replace("市", ""))
        return "|".join(map(re.escape, {a for a in aliases if a}))

    def event_matches_location(event, pattern):
        if not pattern:
            return False
        if event["location"]:
            return bool(re.search(pattern, event["location"]))
        else:
            return bool(re.search(pattern, event["line"]))

    sender_pattern = get_location_pattern(sender_city_key, location_maps)
    sender_pattern_strict = get_location_pattern(
        sender_city_key, location_maps, include_province=False
    )
    dest_pattern_strict = get_location_pattern(
        dest_city_key, location_maps, include_province=False
    )
    dest_pattern_broad = get_location_pattern(
        dest_city_key, location_maps, include_province=True
    )
    for event in all_events:
        if kw_collect and re.search(kw_collect, event["line"]):
            extracted_times["揽收时间_zhuzhuyun"] = event["dt"]
            break
    for event in all_events:
        if kw_delivery and re.search(kw_delivery, event["line"]):
            extracted_times["派送时间_zhuzhuyun"] = event["dt"]
            break
    for event in all_events:
        if (
            pd.notna(extracted_times["派送时间_zhuzhuyun"])
            and event["dt"] < extracted_times["派送时间_zhuzhuyun"]
        ):
            continue
        if kw_delivery and re.search(kw_delivery, event["line"]):
            continue
        if kw_sign_p1 and re.search(kw_sign_p1, event["line"]):
            extracted_times["签收时间_zhuzhuyun"] = event["dt"]
            break
        if kw_sign_p2 and re.search(kw_sign_p2, event["line"]):
            extracted_times["签收时间_zhuzhuyun"] = event["dt"]
            break
        if kw_sign_fallback and re.search(kw_sign_fallback, event["line"]):
            extracted_times["签收时间_zhuzhuyun"] = event["dt"]
            break
    if sender_city_key in ["西安市", "贵阳市"]:
        hub_pattern_map = {"西安市": r"陕西西安转运中心", "贵阳市": r"贵州贵阳转运中心"}
        hub_pattern = hub_pattern_map.get(sender_city_key)
        last_hub_arrival_event = None
        for event in reversed(all_events):
            if (
                kw_arrive
                and re.search(kw_arrive, event["line"])
                and hub_pattern
                and re.search(hub_pattern, event["line"])
            ):
                last_hub_arrival_event = event
                break
        if last_hub_arrival_event:
            extracted_times["到达分拣中心时间_zhuzhuyun"] = last_hub_arrival_event["dt"]
            for event in all_events:
                if event["dt"] > last_hub_arrival_event["dt"]:
                    if kw_leave and re.search(kw_leave, event["line"]):
                        extracted_times["离开寄件城市时间_zhuzhuyun"] = event["dt"]
                        break
    sender_events = [e for e in all_events if event_matches_location(e, sender_pattern)]
    dest_events = []
    if dest_city_key and isinstance(dest_city_key, str) and dest_city_key.strip():
        dest_events = (
            sender_events
            if sender_city_key == dest_city_key
            else [
                e for e in all_events if event_matches_location(e, dest_pattern_broad)
            ]
        )
    if pd.isna(extracted_times["离开寄件城市时间_zhuzhuyun"]):
        if sender_events:
            true_intercity_departure = pd.NaT
            for event in reversed(sender_events):
                if kw_leave and re.search(kw_leave, event["line"]):
                    destination_match = re.search(rf"{kw_leave}\s*(.*)", event["line"])
                    if destination_match:
                        destination_text_raw = destination_match.group(1)
                        if destination_text_raw:
                            destination_text = destination_text_raw.strip()
                            if sender_pattern_strict and not re.search(
                                sender_pattern_strict, destination_text
                            ):
                                true_intercity_departure = event["dt"]
                                break
            if pd.isna(true_intercity_departure):
                for event in reversed(sender_events):
                    if kw_leave and re.search(kw_leave, event["line"]):
                        if (
                            pd.notna(extracted_times["揽收时间_zhuzhuyun"])
                            and event["dt"] > extracted_times["揽收时间_zhuzhuyun"]
                        ):
                            true_intercity_departure = event["dt"]
                            break
            extracted_times["离开寄件城市时间_zhuzhuyun"] = true_intercity_departure
    if pd.isna(extracted_times["到达收件城市时间_zhuzhuyun"]):
        if dest_pattern_strict:
            for event in all_events:
                if "预计" in event["line"]:
                    continue
                if (
                    kw_arrive
                    and re.search(kw_arrive, event["line"])
                    and event_matches_location(event, dest_pattern_strict)
                ):
                    extracted_times["到达收件城市时间_zhuzhuyun"] = event["dt"]
                    break
        if pd.isna(extracted_times["到达收件城市时间_zhuzhuyun"]) and dest_events:
            for event in dest_events:
                if "预计" in event["line"]:
                    continue
                if kw_arrive and re.search(kw_arrive, event["line"]):
                    extracted_times["到达收件城市时间_zhuzhuyun"] = event["dt"]
                    break
    if pd.isna(extracted_times["到达分拣中心时间_zhuzhuyun"]) and pd.notna(
        extracted_times["离开寄件城市时间_zhuzhuyun"]
    ):
        for i, event in enumerate(all_events):
            if event["dt"] == extracted_times["离开寄件城市时间_zhuzhuyun"]:
                if i > 0:
                    previous_event = all_events[i - 1]
                    if (
                        kw_arrive
                        and re.search(kw_arrive, previous_event["line"])
                        and event_matches_location(previous_event, sender_pattern)
                    ):
                        extracted_times["到达分拣中心时间_zhuzhuyun"] = previous_event[
                            "dt"
                        ]
                break
    if pd.isna(extracted_times["离开收件城市分拣中心时间_zhuzhuyun"]):
        if pd.notna(extracted_times["到达收件城市时间_zhuzhuyun"]) and pd.notna(
            extracted_times["派送时间_zhuzhuyun"]
        ):
            start_time = extracted_times["到达收件城市时间_zhuzhuyun"]
            end_time = extracted_times["派送时间_zhuzhuyun"]
            for event in all_events:
                if start_time < event["dt"] < end_time:
                    if (
                        kw_leave
                        and re.search(kw_leave, event["line"])
                        and event_matches_location(event, dest_pattern_broad)
                    ):
                        extracted_times["离开收件城市分拣中心时间_zhuzhuyun"] = event[
                            "dt"
                        ]
                        break
    return pd.Series(extracted_times)


# --- 主流程: 申通 (已修改数据读取逻辑) ---
def process_sto_data(zhuzhuyun_dir, anjian_dir, output_dir, base_data_file, period):
    company_name = "申通"
    config = {
        "zhuzhu_filename": f"申通_{period}.xlsx",
        "internal_name": "申通",
        "anjian_map_key": "STO",
        "parser": parse_logistics_events_sto,
    }
    company_start_time = perf_counter()

    location_maps = {
        "city_alias_map": {},
        "capital_cities_set": set(),
        "city_to_province_map": {},
    }
    if base_data_file.exists():
        try:
            df_hierarchy = pd.read_excel(base_data_file, sheet_name="city_hierarchy")
            df_hierarchy.dropna(subset=["Province", "City", "District"], inplace=True)
            df_hierarchy["City_clean"] = df_hierarchy["City"].str.replace(
                "市", "", regex=False
            )
            df_hierarchy["District_clean"] = df_hierarchy["District"].str.replace(
                "市", "", regex=False
            )
            location_maps["city_to_province_map"] = (
                df_hierarchy.drop_duplicates("City")
                .set_index("City")["Province"]
                .to_dict()
            )
            if "IsCapital" in df_hierarchy.columns:
                location_maps["capital_cities_set"] = set(
                    df_hierarchy[df_hierarchy["IsCapital"] == 1]["City"].unique()
                )
            for city, group in df_hierarchy.groupby("City"):
                aliases = [group.iloc[0]["City_clean"]] + group[
                    "District_clean"
                ].unique().tolist()
                location_maps["city_alias_map"][city] = [a for a in aliases if a]
        except Exception as e:
            print(f"[ERROR] 构建地理位置地图失败: {e}。")

    df_base_subset = None
    if base_data_file.exists():
        try:
            df_base = pd.read_excel(base_data_file, sheet_name="inter-city_routes")
            df_base["路线_std"] = (
                df_base["寄出城市"].str.replace("市", "", regex=False)
                + "-"
                + df_base["寄达城市"].str.replace("市", "", regex=False)
            )
            df_base_subset = df_base[
                ["路线_std", "公里", "寄出省份", "寄达省份"]
            ].drop_duplicates("路线_std")
        except Exception as e:
            print(f"[WARNING] 处理 'basic_data.xlsx' 出错: {e}。")

    # -----------------------------------------------------------------
    # 【核心改动】根据period动态读取本期的安监数据文件
    # -----------------------------------------------------------------
    year = period[:4]
    month = int(period[4:])  # '202506' -> 6, 以匹配 '6月' 而非 '06月'
    anjian_filename = f"{year}年{month}月{company_name}抽样.xlsx"
    anjian_file_path = anjian_dir / anjian_filename

    if not anjian_file_path.exists():
        print(f"[ERROR] 安监数据文件未找到，流程中止: {anjian_file_path.name}")
        return

    print(f"  -> 正在读取安监数据: {anjian_file_path.name}")
    try:
        all_anjian_df = pd.read_excel(
            anjian_file_path, dtype={"单号": str, "寄出城市": str, "寄达城市": str}
        )
    except Exception as e:
        print(f"[ERROR] 读取安监文件 '{anjian_filename}' 时出错: {e}")
        return
    # -----------------------------------------------------------------

    if "快递公司" in all_anjian_df.columns:
        all_anjian_df.rename(columns={"快递公司": "企业"}, inplace=True)

    # 【新增】准备安监数据源
    df_anjian_sto = all_anjian_df[
        all_anjian_df["企业"] == config["anjian_map_key"]
    ].copy()
    if not df_anjian_sto.empty:
        print(
            f"  -> 已从安监数据中筛选出 {len(df_anjian_sto)} 条 {company_name} (STO) 记录。"
        )
        df_anjian_sto.rename(
            columns={
                "揽收时间": "揽收时间_anjian",
                "离开寄件城市时间": "离开寄件城市时间_anjian",
                "到达收件城市时间": "到达收件城市时间_anjian",
                "派送时间": "派送时间_anjian",
                "签收时间": "签收时间_anjian",
            },
            inplace=True,
        )
        df_anjian_sto["企业"] = config["internal_name"]

    # --- 猪猪云数据处理 ---
    company_file = zhuzhuyun_dir / config["zhuzhu_filename"]
    if not company_file.exists():
        print(f"[WARNING] 未找到文件: {config['zhuzhu_filename']}，已跳过。")
        return
    df_company_zhu = pd.read_excel(company_file, dtype={"快递单号": str})
    df_company_zhu.rename(
        columns={"快递单号": "单号", "快递公司": "企业"}, inplace=True
    )
    base_info_df = all_anjian_df[all_anjian_df["企业"] == config["anjian_map_key"]][
        ["单号", "企业", "寄出城市", "寄达城市"]
    ].drop_duplicates("单号")
    base_info_df["企业"] = config["internal_name"]
    df_merged = pd.merge(df_company_zhu, base_info_df, on=["单号", "企业"], how="left")
    base_profile = COMPANY_PROFILES_STO["default"].copy()
    company_specific_profile = COMPANY_PROFILES_STO.get(config["internal_name"], {})
    profile = {**base_profile, **company_specific_profile}
    extracted_df = extractor_apply_based(
        df_merged, profile, location_maps, config["parser"]
    )
    df_zhuzhuyun_final = pd.concat([df_merged, extracted_df], axis=1)

    # 【新增】整合两大数据源
    zhuzhuyun_cols_to_keep = [
        "单号",
        "企业",
        "寄出城市",
        "寄达城市",
        "完整物流信息",
    ] + [col for col in df_zhuzhuyun_final if "_zhuzhuyun" in col]
    df_zhuzhuyun_subset = df_zhuzhuyun_final[zhuzhuyun_cols_to_keep]
    anjian_cols_to_keep = ["单号", "企业"] + [
        col for col in df_anjian_sto if "_anjian" in col
    ]
    df_anjian_subset = df_anjian_sto[anjian_cols_to_keep]
    df_final = pd.merge(
        df_zhuzhuyun_subset, df_anjian_subset, on=["单号", "企业"], how="outer"
    )
    print(f"  -> 已整合安监与猪猪云数据，共 {len(df_final)} 条唯一记录。")

    # --------------------------------------------------------------------------------------
    # 【核心改动】替换原有的时间戳赋值逻辑，改为根据全局配置选择
    # --------------------------------------------------------------------------------------
    print("  -> 根据全局配置选择时间戳来源...")
    # 假设 DATA_SOURCE_CONFIG 已在别处定义
    # e.g. DATA_SOURCE_CONFIG = {"申通": {"collection_time_source": "anjian", ...}}
    source_config = DATA_SOURCE_CONFIG.get(company_name, {})
    if not source_config:
        raise ValueError(
            f"错误: 在 DATA_SOURCE_CONFIG 中未找到 '{company_name}' 的配置!"
        )
    timestamp_map = {
        "揽收时间": "collection_time_source",
        "离开寄件城市时间": "departure_time_source",
        "到达收件城市时间": "arrival_time_source",
        "派送时间": "dispatch_time_source",
        "签收时间": "signed_time_source",
    }
    for final_col, config_key in timestamp_map.items():
        source_type = source_config.get(config_key, "anjian")
        primary_col = f"{final_col}_{source_type}"
        fallback_col = (
            f"{final_col}_{'zhuzhuyun' if source_type == 'anjian' else 'anjian'}"
        )
        df_final[final_col] = df_final.get(primary_col)
        if fallback_col in df_final:
            df_final[final_col] = df_final[final_col].fillna(df_final[fallback_col])
        print(f"    - '{final_col}' 已配置使用 '{source_type}' 数据源。")
    df_final["到达分拣中心时间"] = df_final.get("到达分拣中心时间_zhuzhuyun")
    df_final["离开收件城市分拣中心时间"] = df_final.get(
        "离开收件城市分拣中心时间_zhuzhuyun"
    )
    # --------------------------------------------------------------------------------------

    # --- 后续处理与保存 (保持不变) ---
    if df_base_subset is not None:
        if "寄出城市_x" in df_final.columns and "寄出城市_y" in df_final.columns:
            df_final["寄出城市"] = df_final["寄出城市_x"].fillna(df_final["寄出城市_y"])
            df_final["寄达城市"] = df_final["寄达城市_x"].fillna(df_final["寄达城市_y"])
        df_final["路线_std"] = (
            df_final["寄出城市"].astype(str).str.replace("市", "", regex=False)
            + "-"
            + df_final["寄达城市"].astype(str).str.replace("市", "", regex=False)
        )
        df_final = pd.merge(df_final, df_base_subset, on="路线_std", how="left")
    df_final["企业"] = company_name
    output_columns = [
        "企业",
        "单号",
        "寄出城市",
        "寄达城市",
        "揽收时间",
        "离开寄件城市时间",
        "到达收件城市时间",
        "派送时间",
        "签收时间",
        "公里",
        "寄出省份",
        "寄达省份",
        "到达分拣中心时间",
        "离开收件城市分拣中心时间",
        "完整物流信息",
    ]
    final_df_to_save = df_final.reindex(columns=output_columns)
    output_file = output_dir / f"{company_name}_logistics_data_{period}.xlsx"
    final_df_to_save.to_excel(output_file, index=False)
    company_end_time = perf_counter()
    print(
        f"✅ {company_name} 处理完成，耗时 {company_end_time - company_start_time:.2f} 秒。文件已保存至: {output_file.name}"
    )


# --- 运行: 申通 (保持不变) ---
if "period" in locals() and "DATA_SOURCE_CONFIG" in locals():
    process_sto_data(
        zhuzhuyun_merge_path,
        anjian_data_path,
        pycharm_input_path,
        base_data_path,
        period,
    )
else:
    print(
        "[警告] 未定义 'period' 或 'DATA_SOURCE_CONFIG' 变量，跳过 process_sto_data 的执行。请在前面的单元格中定义这些变量。"
    )

--- [开始] 处理 申通 ---
  -> 正在读取安监数据: 2025年6月申通抽样.xlsx
  -> 已从安监数据中筛选出 83029 条 申通 (STO) 记录。
  -> 使用 'parse_logistics_events_sto' 解析器通过 .apply() 运行...
  -> 已整合安监与猪猪云数据，共 83029 条唯一记录。
  -> 根据全局配置选择时间戳来源...
    - '揽收时间' 已配置使用 'anjian' 数据源。
    - '离开寄件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '到达收件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '派送时间' 已配置使用 'zhuzhuyun' 数据源。
    - '签收时间' 已配置使用 'zhuzhuyun' 数据源。
✅ 申通 处理完成，耗时 51.72 秒。文件已保存至: 申通_logistics_data_202506.xlsx


In [38]:
# --------------------------------------------------
# Cell 4.8: 圆通logistics数据提取
# --------------------------------------------------
import re
import warnings
from pathlib import Path
from time import perf_counter
from typing import Any, Dict, Optional

import pandas as pd

# --- 初始化 ---
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
print("--- [开始] 处理 圆通 ---")

# --- 项目路径设置 (请确保在Cell 0中已正确设置) ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
temp_path = report_path / "temp"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"


# --- 公司档案: 圆通 (YTO) (保持不变) ---
COMPANY_PROFILES_YTO = {
    "圆通": {
        "keywords": {
            "collect": r"已揽收",
            "delivery": r"正在派件",
            "leave": r"离开|准备飞往",
            "center": r"转运中心|接驳点",
            "airport": r"机场",
            "sign_p1": r"已到达.*?驿站|已到达.*?代收点|已到达.*?智能柜|已到达.*?自提柜|已到达.*?云柜|已到达.*?智柜|已到达.*?快递柜|已到达【.*?(仓|仓库|组|公司)】|已到达.*?包裹自提|已到达.*?丰巢|已到达.*?兔喜生活|已到达.*?腾云宝|已到达.*?韵达超市|已到达.*?驿收发|已到达.*?溪鸟|已到达.*?格格货栈|已到达.*?近邻宝|已到达.*?富友|已到达.*?熊猫快收|已到达.*?巧目|已到达.*?和驿|已到达.*?喵站|已到达.*?蜜罐|已到达.*?峰栈|已到达.*?腾云小站|已验收成功|本人签收|收件人:家门口",
            "sign_p2": r"已签收",
            "sign_fallback": r"快件已投递",
        },
        "parser": "parse_logistics_events_yto_test",
    }
}


# --- 辅助函数 (保持不变) ---
def get_location_pattern(
    city_key: Optional[str], location_maps: Dict[str, Any]
) -> Optional[str]:
    if not city_key or not isinstance(city_key, str):
        return None
    city_clean = city_key.replace("市", "")
    aliases = {city_clean}
    aliases.update(location_maps["city_alias_map"].get(city_key, []))
    if city_key in location_maps["capital_cities_set"]:
        province = location_maps["city_to_province_map"].get(city_key)
        if province:
            province_clean = re.sub(r"省|市|自治区|维吾尔|壮族", "", province)
            aliases.add(province_clean)
    unique_aliases = set(filter(None, aliases))
    return "|".join(map(re.escape, unique_aliases)) if unique_aliases else None


# --- 核心解析器: 圆通 (TEST版) (保持不变, 仅为统一命名规范修改后缀) ---
def parse_logistics_events_yto_test(
    row: pd.Series, profile: Dict[str, Any], location_maps: Dict[str, Any]
) -> pd.Series:
    log_text = row.get("完整物流信息")
    kw = profile["keywords"]
    # 【改动】为统一命名规范，将列后缀从 _zzy 改为 _zhuzhuyun
    time_cols = [
        "揽收时间_zhuzhuyun",
        "离开寄件城市时间_zhuzhuyun",
        "到达收件城市时间_zhuzhuyun",
        "派送时间_zhuzhuyun",
        "签收时间_zhuzhuyun",
        "到达分拣中心时间_zhuzhuyun",
        "离开收件城市分拣中心时间_zhuzhuyun",
    ]
    extracted_times = {col: pd.NaT for col in time_cols}
    if not isinstance(log_text, str) or not log_text.strip():
        return pd.Series(extracted_times)
    # --- 您其他的解析器逻辑保持完全不变 ---
    datetime_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    events_data = []
    for line in log_text.strip().split("\n"):
        match = re.search(datetime_pattern, line)
        if match:
            dt = pd.to_datetime(match.group(1), errors="coerce")
            if pd.notna(dt):
                location_match = re.search(r"【(.*?)】", line)
                location = location_match.group(1) if location_match else None
                events_data.append({"dt": dt, "line": line, "location": location})
    if not events_data:
        return pd.Series(extracted_times)
    events_df = (
        pd.DataFrame(events_data)
        .sort_values(by="dt", ascending=True)
        .reset_index(drop=True)
    )
    collect_events = events_df[events_df["line"].str.contains(kw["collect"], na=False)]
    if not collect_events.empty:
        extracted_times["揽收时间_zhuzhuyun"] = collect_events.iloc[0]["dt"]
    delivery_events = events_df[
        events_df["line"].str.contains(kw["delivery"], na=False)
    ]
    if not delivery_events.empty:
        extracted_times["派送时间_zhuzhuyun"] = delivery_events.iloc[0]["dt"]
    for sign_kw_key in ["sign_p1", "sign_p2", "sign_fallback"]:
        if pd.isna(extracted_times["签收时间_zhuzhuyun"]) and kw.get(sign_kw_key):
            sign_events = events_df[
                events_df["line"].str.contains(kw[sign_kw_key], na=False, regex=True)
            ]
            if not sign_events.empty:
                extracted_times["签收时间_zhuzhuyun"] = sign_events.iloc[-1]["dt"]
    sender_city_key = row.get("寄出城市")
    sender_pattern = get_location_pattern(sender_city_key, location_maps)
    if pd.notna(extracted_times["揽收时间_zhuzhuyun"]) and sender_pattern:
        post_collect_events = events_df[
            events_df["dt"] > extracted_times["揽收时间_zhuzhuyun"]
        ]
        all_hub_keywords = kw["center"] + "|" + kw.get("airport", "")
        sender_hub_events = post_collect_events[
            post_collect_events["location"].str.contains(
                all_hub_keywords, na=False, regex=True
            )
            & post_collect_events["location"].str.contains(
                sender_pattern, na=False, regex=True
            )
        ]
        if not sender_hub_events.empty:
            sender_arrive = sender_hub_events[
                sender_hub_events["line"].str.contains("已经到达", na=False)
            ]
            if not sender_arrive.empty:
                extracted_times["到达分拣中心时间_zhuzhuyun"] = sender_arrive.iloc[-1][
                    "dt"
                ]
            sender_leave = sender_hub_events[
                sender_hub_events["line"].str.contains(
                    kw["leave"], na=False, regex=True
                )
            ]
            if not sender_leave.empty:
                extracted_times["离开寄件城市时间_zhuzhuyun"] = sender_leave.iloc[-1][
                    "dt"
                ]
    dest_city_key = row.get("寄达城市")
    dest_pattern = get_location_pattern(dest_city_key, location_maps)
    if dest_pattern:
        dest_hub_arrive_events = events_df[
            events_df["location"].str.contains(
                kw["center"] + "|" + kw.get("airport", ""), na=False, regex=True
            )
            & events_df["location"].str.contains(dest_pattern, na=False, regex=True)
            & events_df["line"].str.contains("已经到达", na=False)
        ]
        if not dest_hub_arrive_events.empty:
            extracted_times["到达收件城市时间_zhuzhuyun"] = dest_hub_arrive_events.iloc[
                0
            ]["dt"]
        else:
            dest_arrive_events = events_df[
                events_df["line"].str.contains("已经到达", na=False)
                & events_df["location"].str.contains(dest_pattern, na=False, regex=True)
            ]
            if not dest_arrive_events.empty:
                extracted_times["到达收件城市时间_zhuzhuyun"] = dest_arrive_events.iloc[
                    0
                ]["dt"]
        if pd.notna(extracted_times["派送时间_zhuzhuyun"]):
            pre_delivery_events = events_df[
                events_df["dt"] < extracted_times["派送时间_zhuzhuyun"]
            ]
            dest_location_events = pre_delivery_events[
                pre_delivery_events["location"].str.contains(
                    dest_pattern, na=False, case=False, regex=True
                )
            ]
            if not dest_location_events.empty:
                dest_leave_events = dest_location_events[
                    dest_location_events["line"].str.contains("离开", na=False)
                    & dest_location_events["location"].str.contains(
                        kw["center"], na=False, regex=True
                    )
                ]
                if not dest_leave_events.empty:
                    extracted_times["离开收件城市分拣中心时间_zhuzhuyun"] = (
                        dest_leave_events.iloc[-1]["dt"]
                    )
    return pd.Series(extracted_times)


# --- 主流程: 圆通 (已修改) ---
def process_yto_data(
    zhuzhuyun_dir: Path,
    anjian_dir: Path,
    output_dir: Path,
    base_data_file: Path,
    period: str,
):
    company_name = "圆通"
    config = {
        "zhuzhu_filename": f"圆通_{period}.xlsx",
        "parser": "parse_logistics_events_yto_test",
        "anjian_map_key": "YTO",
    }
    company_start_time = perf_counter()

    # 1. 加载通用数据
    location_maps = {
        "city_alias_map": {},
        "capital_cities_set": set(),
        "city_to_province_map": {},
    }
    if base_data_file.exists():
        try:
            df_hierarchy = pd.read_excel(base_data_file, sheet_name="city_hierarchy")
            df_hierarchy.dropna(subset=["Province", "City", "District"], inplace=True)
            df_hierarchy["City_clean"] = df_hierarchy["City"].str.replace(
                "市", "", regex=False
            )
            df_hierarchy["District_clean"] = df_hierarchy["District"].str.replace(
                r"市|区|县", "", regex=True
            )
            for city, group in df_hierarchy.groupby("City"):
                aliases = [group.iloc[0]["City_clean"]] + group[
                    "District_clean"
                ].unique().tolist()
                location_maps["city_alias_map"][city] = [a for a in aliases if a]
            if "IsCapital" in df_hierarchy.columns:
                df_capitals = df_hierarchy[df_hierarchy["IsCapital"] == 1]
                location_maps["capital_cities_set"] = set(df_capitals["City"].unique())
            city_to_province = df_hierarchy.drop_duplicates("City").set_index("City")
            location_maps["city_to_province_map"] = city_to_province[
                "Province"
            ].to_dict()
        except Exception as e:
            print(f"[错误] 构建地理位置地图失败: {e}。")

    base_routes_df = pd.DataFrame()
    if base_data_file.exists():
        try:
            df_base = pd.read_excel(base_data_file, sheet_name="inter-city_routes")
            df_base["路线_std"] = (
                df_base["寄出城市"].str.replace("市", "", regex=False)
                + "-"
                + df_base["寄达城市"].str.replace("市", "", regex=False)
            )
            base_routes_df = df_base[
                ["路线_std", "公里", "寄出省份", "寄达省份"]
            ].drop_duplicates("路线_std")
        except Exception as e:
            print(
                f"[警告] 处理 'basic_data.xlsx' 的 'inter-city_routes' sheet 出错: {e}。"
            )

    # 2. 加载安监数据 (核心修改点)
    year = period[:4]
    month = int(period[4:])
    anjian_filename = f"{year}年{month}月{company_name}抽样.xlsx"
    anjian_file_path = anjian_dir / anjian_filename

    if not anjian_file_path.exists():
        print(f"[警告] 安监数据文件 {anjian_filename} 未找到，将仅使用猪猪云数据。")
        all_anjian_df = pd.DataFrame()
    else:
        print(f"  -> 正在读取安监数据: {anjian_filename}")
        try:
            all_anjian_df = pd.read_excel(
                anjian_file_path, dtype={"单号": str, "寄出城市": str, "寄达城市": str}
            )
        except Exception as e:
            print(f"[错误] 读取安监文件 '{anjian_filename}' 时出错: {e}")
            all_anjian_df = pd.DataFrame()

    if "快递公司" in all_anjian_df.columns:
        all_anjian_df.rename(columns={"快递公司": "企业"}, inplace=True)

    # 准备安监数据
    df_anjian_company_std = all_anjian_df[
        all_anjian_df["企业"] == config["anjian_map_key"]
    ].copy()
    if not df_anjian_company_std.empty:
        print(
            f"  -> 已从安监数据中筛选出 {len(df_anjian_company_std)} 条 {company_name} (YTO) 记录。"
        )
        anjian_ts_cols = [
            "揽收时间",
            "离开寄件城市时间",
            "到达收件城市时间",
            "派送时间",
            "签收时间",
        ]
        for col in anjian_ts_cols:
            if col in df_anjian_company_std.columns:
                df_anjian_company_std.rename(
                    columns={col: f"{col}_anjian"}, inplace=True
                )
        df_anjian_company_std["企业"] = company_name

    # 3. 加载猪猪云数据
    company_file = zhuzhuyun_dir / config["zhuzhu_filename"]
    if not company_file.exists():
        print(f"[警告] 未找到文件: {config['zhuzhu_filename']}，已跳过。")
        return

    df_company_zhu = pd.read_excel(company_file, dtype={"快递单号": str})
    df_company_zhu.rename(
        columns={"快递单号": "单号", "快递公司": "企业"}, inplace=True
    )
    df_company_zhu["企业"] = company_name

    # 4. 合并数据
    df_merged_full = pd.merge(
        df_company_zhu, df_anjian_company_std, on=["单号", "企业"], how="outer"
    )
    print(f"  -> 已整合安监与猪猪云数据，共 {len(df_merged_full)} 条唯一记录。")

    # 5. 运行解析器并选择数据源
    parser_func_name = config["parser"]
    parser_func = globals()[parser_func_name]
    profile = COMPANY_PROFILES_YTO[company_name]
    print(f"  -> 使用 '{parser_func_name}' 解析器运行...")
    extracted_df = df_merged_full.apply(
        lambda row: parser_func(row, profile, location_maps), axis=1
    )
    df_final = pd.concat([df_merged_full, extracted_df], axis=1)

    print("  -> 根据全局配置选择时间戳来源...")
    source_config = DATA_SOURCE_CONFIG.get(company_name, {})
    if not source_config:
        raise ValueError(
            f"错误: 在 DATA_SOURCE_CONFIG 中未找到 '{company_name}' 的配置!"
        )

    timestamp_map = {
        "揽收时间": "collection_time_source",
        "离开寄件城市时间": "departure_time_source",
        "到达收件城市时间": "arrival_time_source",
        "派送时间": "dispatch_time_source",
        "签收时间": "signed_time_source",
    }
    for final_col, config_key in timestamp_map.items():
        source_type = source_config.get(config_key, "anjian")
        primary_col = f"{final_col}_{source_type}"
        fallback_col = (
            f"{final_col}_{'zhuzhuyun' if source_type == 'anjian' else 'anjian'}"
        )
        df_final[final_col] = df_final.get(primary_col)
        if fallback_col in df_final:
            df_final[final_col] = df_final[final_col].fillna(df_final[fallback_col])
        print(f"    - '{final_col}' 已配置使用 '{source_type}' 数据源。")

    df_final["到达分拣中心时间"] = df_final.get("到达分拣中心时间_zhuzhuyun")
    df_final["离开收件城市分拣中心时间"] = df_final.get(
        "离开收件城市分拣中心时间_zhuzhuyun"
    )

    # 6. 后续处理与保存
    if not base_routes_df.empty:
        if "寄出城市_x" in df_final.columns and "寄出城市_y" in df_final.columns:
            df_final["寄出城市"] = df_final["寄出城市_x"].fillna(df_final["寄出城市_y"])
            df_final["寄达城市"] = df_final["寄达城市_x"].fillna(df_final["寄达城市_y"])
        df_final["路线_std"] = (
            df_final["寄出城市"].astype(str).str.replace("市", "", regex=False)
            + "-"
            + df_final["寄达城市"].astype(str).str.replace("市", "", regex=False)
        )
        df_final = pd.merge(df_final, base_routes_df, on="路线_std", how="left")

    df_final["企业"] = company_name
    output_columns = [
        "企业",
        "单号",
        "寄出城市",
        "寄达城市",
        "揽收时间",
        "离开寄件城市时间",
        "到达收件城市时间",
        "派送时间",
        "签收时间",
        "公里",
        "寄出省份",
        "寄达省份",
        "到达分拣中心时间",
        "离开收件城市分拣中心时间",
        "完整物流信息",
    ]
    final_df_to_save = df_final.reindex(columns=output_columns)
    output_file = output_dir / f"{company_name}_logistics_data_{period}.xlsx"
    final_df_to_save.to_excel(output_file, index=False)
    company_end_time = perf_counter()
    print(
        f"✅ {company_name} 处理完成，耗时 {company_end_time - company_start_time:.2f} 秒。文件已保存至: {output_file.name}"
    )


# --- 运行主流程 ---
if "period" in locals() and "DATA_SOURCE_CONFIG" in locals():
    process_yto_data(
        zhuzhuyun_merge_path,
        anjian_data_path,
        pycharm_input_path,
        base_data_path,
        period,
    )
else:
    print(
        "[警告] 未定义 'period' 或 'DATA_SOURCE_CONFIG' 变量，跳过 process_yto_data 的执行。请在前面的单元格中定义这些变量。"
    )

--- [开始] 处理 圆通 ---
  -> 正在读取安监数据: 2025年6月圆通抽样.xlsx
  -> 已从安监数据中筛选出 82733 条 圆通 (YTO) 记录。
  -> 已整合安监与猪猪云数据，共 82733 条唯一记录。
  -> 使用 'parse_logistics_events_yto_test' 解析器运行...


/var/folders/j4/7fywcbxd7t52ftqjt6sy8cs40000gn/T/ipykernel_49637/244163722.py:111: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  events_df["line"].str.contains(kw[sign_kw_key], na=False, regex=True)


  -> 根据全局配置选择时间戳来源...
    - '揽收时间' 已配置使用 'anjian' 数据源。
    - '离开寄件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '到达收件城市时间' 已配置使用 'zhuzhuyun' 数据源。
    - '派送时间' 已配置使用 'zhuzhuyun' 数据源。
    - '签收时间' 已配置使用 'zhuzhuyun' 数据源。
✅ 圆通 处理完成，耗时 206.70 秒。文件已保存至: 圆通_logistics_data_202506.xlsx


In [ ]:
# -------------------------------------------------------------------
# Cell 4.9: 顺丰、中通logistics数据提取（由于无猪猪云输出，仅基于安监数据生成）
# -------------------------------------------------------------------
import time
from pathlib import Path

import numpy as np
import pandas as pd

# --- 假设的路径和变量 (实际使用时请确保这些已在之前的单元格中定义) ---
period = "202506"
input_path = Path("报告数据/输入")
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
pycharm_input_path = Path("报告数据/temp/4_logistics数据")
# --------------------------------------------------------------------

# 定义要处理的公司列表
companies_to_process = {"SF": "顺丰", "ZTO": "中通"}

# 定义目标logistics数据文件的完整列名顺序 (已修正)
target_columns = [
    "企业",
    "单号",
    "寄出城市",
    "寄达城市",
    "揽收时间",
    "离开寄件城市时间",
    "到达收件城市时间",
    "派送时间",
    "签收时间",
    "公里",
    "寄出省份",
    "寄达省份",
    "到达分拣中心时间",
    "离开收件城市分拣中心时间",
    "完整物流信息",
]

# 1. 提前读取城市线路基础数据
try:
    print(f"正在读取基础路线数据: {base_data_path}")
    routes_df = pd.read_excel(
        base_data_path,
        sheet_name="inter-city_routes",
        usecols=["寄出城市", "寄达城市", "寄出省份", "寄达省份", "公里"],
    )
    print("✅ 基础路线数据读取成功。")
except FileNotFoundError:
    print(
        f"❌ 严重错误: 基础数据文件 '{base_data_path.name}' 未在 '{input_path}' 目录下找到。脚本无法继续。"
    )
    routes_df = None
except Exception as e:
    print(f"❌ 读取基础数据文件时发生错误: {e}")
    routes_df = None

# 动态生成处理通知
if routes_df is not None:
    companies_str = "、".join(companies_to_process.values())
    print(f"开始处理 {companies_str} 的安监数据...")

    # 循环处理每个公司
    for raw_name, company_name in companies_to_process.items():
        company_start_time = time.perf_counter()
        try:
            # 2. 根据命名规则构建输入文件名
            input_filename = f"{period[:4]}年{int(period[4:])}月{company_name}抽样.xlsx"
            input_filepath = anjian_data_path / input_filename

            output_file = (
                pycharm_input_path / f"{company_name}_logistics_data_{period}.xlsx"
            )

            # 3. 读取原始安监数据Excel文件
            df = pd.read_excel(input_filepath)

            # 4. 标准化企业名称
            df["企业"] = company_name

            # 5. 合并数据以填充省份和公里数
            df_merged = pd.merge(df, routes_df, on=["寄出城市", "寄达城市"], how="left")

            # 6. 添加剩余的空列
            df_merged["到达分拣中心时间"] = pd.NaT
            df_merged["离开收件城市分拣中心时间"] = pd.NaT  # 这一列的名称在之前是错误的
            df_merged["完整物流信息"] = ""

            # 7. 重新排列DataFrame的列
            df_reordered = df_merged[target_columns]

            # 8. 将处理好的DataFrame保存为新的Excel文件
            df_reordered.to_excel(output_file, index=False)

            company_end_time = time.perf_counter()
            print(
                f"✅ {company_name} 处理完成，耗时 {company_end_time - company_start_time:.2f} 秒。文件已保存至: {output_file.name}"
            )

        except FileNotFoundError:
            print(
                f"❌ 错误: 文件 '{input_filename}' 未在 '{anjian_data_path}' 目录下找到。"
            )
        except Exception as e:
            print(f"❌ 处理 {company_name} 时发生未知错误: {e}")

正在读取基础路线数据: 报告数据/输入/basic_data.xlsx
✅ 基础路线数据读取成功。
开始处理 顺丰、中通 的安监数据...
✅ 顺丰 处理完成，耗时 10.99 秒。文件已保存至: 顺丰_logistics_data_202506.xlsx
✅ 中通 处理完成，耗时 10.61 秒。文件已保存至: 中通_logistics_data_202506.xlsx


In [27]:
# --------------------------------------------------
# Cell 5: 中转数据生成
# --------------------------------------------------
# --- 1. 设置文件路径 ---
# 输入路径
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_dir = report_path / "输入"
anjian_dir = input_dir / "安监数据"
# 中间过程文件路径
temp_path = report_path / "temp"
# 物流明细数据来自上一步生成的猪猪云合并数据
logistics_dir = temp_path / "3_猪猪云合并数据"
output_dir = temp_path / "5_中转数据"
basic_data_path = input_dir / "basic_data.xlsx"

anjian_dir.mkdir(parents=True, exist_ok=True)
logistics_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)
print(f"基础数据文件应位于: {basic_data_path}")
print(f"安监数据文件夹: {anjian_dir}")
print(f"物流明细文件夹: {logistics_dir}")
print(f"计算结果将输出至: {output_dir}")
print("-" * 40)


# --- 2. 城市加载与匹配逻辑 ---
def load_city_pattern(path_to_basic_data):
    try:
        if not path_to_basic_data.exists():
            return None
        df_city = pd.read_excel(
            path_to_basic_data, sheet_name="city_names_complete_2025", engine="openpyxl"
        )
        if "城市" not in df_city.columns or "行政级别" not in df_city.columns:
            return None
        df_city["行政级别"] = df_city["行政级别"].str.strip()
        target_levels = ["地级市", "直辖市", "副省级城市", "省直辖县级市"]
        target_df = df_city[df_city["行政级别"].isin(target_levels)].copy()
        target_df["城市_clean"] = target_df["城市"].str.replace(
            r"(市|省|自治区|特别行政区)$", "", regex=True
        )
        city_list = target_df["城市_clean"].dropna().unique().tolist()
        city_list.sort(key=len, reverse=True)
        print(f"成功加载 {len(city_list)} 个目标城市。")
        return re.compile("|".join(city_list))
    except Exception as e:
        print(f"加载和处理城市数据时出错: {e}")
        return None


BASE_CITY_PATTERN = load_city_pattern(basic_data_path)
AMBIGUOUS_MAP = {
    "朝阳": "北京",
    "湘潭": "天津",
}


def find_all_valid_cities_in_text(text, pattern):
    if not isinstance(text, str) or not pattern:
        return []
    cities_found = pattern.findall(text)
    valid_cities = []
    for city in cities_found:
        is_noise = False
        if city in AMBIGUOUS_MAP:
            context_word = AMBIGUOUS_MAP[city]
            if context_word in text:
                is_noise = True
        if not is_noise:
            valid_cities.append(city + "市")
    return list(dict.fromkeys(valid_cities))


# --- “锚点切割”算法---
def extract_transit_log_slice(log_entries, origin_city, dest_city):
    last_origin_anchor = -1
    first_dest_anchor = -1
    origin_city_clean = origin_city.replace("市", "")
    dest_city_clean = dest_city.replace("市", "")
    for i, entry in enumerate(log_entries):
        if origin_city_clean in entry:
            last_origin_anchor = i
    for i, entry in enumerate(log_entries):
        if dest_city_clean in entry:
            first_dest_anchor = i
            break
    if (
        last_origin_anchor != -1
        and first_dest_anchor != -1
        and last_origin_anchor < first_dest_anchor
    ):
        return log_entries[last_origin_anchor + 1 : first_dest_anchor]
    return []


# --- 3. 核心处理逻辑 ---
def process_company_data(logistics_path, anjian_path, period):
    company_name_raw = logistics_path.stem
    company_name = company_name_raw.split("_")[0]
    rename_map = {"邮政国内小包": "邮政"}
    company_name = rename_map.get(company_name, company_name)
    print(f"\n--- 正在处理: {company_name} (数据期数: {period}) ---")

    try:
        df_anjian = pd.read_excel(anjian_path)
        df_logistics = pd.read_excel(logistics_path)

        df_anjian = df_anjian[["单号", "寄出城市", "寄达城市"]].dropna()
        df_anjian["寄出城市"] = (
            df_anjian["寄出城市"].str.replace("市", "", regex=False) + "市"
        )
        df_anjian["寄达城市"] = (
            df_anjian["寄达城市"].str.replace("市", "", regex=False) + "市"
        )
        df_anjian["单号"] = df_anjian["单号"].astype(str)

        df_logistics = df_logistics[["快递单号", "完整物流信息"]].dropna()
        df_logistics["快递单号"] = df_logistics["快递单号"].astype(str)

        merged_df = pd.merge(
            df_logistics, df_anjian, left_on="快递单号", right_on="单号", how="inner"
        )
        print(
            f"在 {logistics_path.name} 和 {anjian_path.name} 中共找到 {len(merged_df)} 条匹配的运单。"
        )

    except Exception as e:
        print(f"处理公司 {company_name} 时发生错误: {e}")
        return

    all_routes = []
    for _, row in merged_df.iterrows():
        origin_city_auth = row["寄出城市"]
        dest_city_auth = row["寄达城市"]
        full_log = row["完整物流信息"]

        log_entries = [entry.strip() for entry in full_log.split("\n") if entry.strip()]
        log_entries.reverse()

        transit_slice = extract_transit_log_slice(
            log_entries, origin_city_auth, dest_city_auth
        )

        transit_cities = set()
        for entry in transit_slice:
            cities_in_entry = find_all_valid_cities_in_text(entry, BASE_CITY_PATTERN)
            for city in cities_in_entry:
                if city not in [origin_city_auth, dest_city_auth]:
                    transit_cities.add(city)

        all_routes.append(
            {
                "出发城市": origin_city_auth,
                "到达城市": dest_city_auth,
                "中转城市列表": sorted(list(transit_cities)),
                "中转次数": len(transit_cities),
            }
        )

    if not all_routes:
        print(f"未能在 {company_name} 的数据中提取出任何有效的中转路径。")
        return

    routes_df = pd.DataFrame(all_routes)
    agg_result = (
        routes_df.groupby(["出发城市", "到达城市"])
        .agg(
            平均中转次数=("中转次数", "mean"),
            中转城市=(
                "中转城市列表",
                lambda s: sorted(list(set(c for sub in s for c in sub))),
            ),
        )
        .reset_index()
    )
    agg_result["中转城市"] = agg_result["中转城市"].apply(lambda x: ",".join(x))
    agg_result["平均中转次数"] = agg_result["平均中转次数"].round(2)
    agg_result = agg_result.sort_values(by="平均中转次数", ascending=False).reset_index(
        drop=True
    )
    final_columns = ["出发城市", "到达城市", "中转城市", "平均中转次数"]
    agg_result = agg_result[final_columns]

    # 新增修改: 输出文件名中加入 period
    output_filename = f"{company_name}_transit_data_{period}.xlsx"
    output_path = output_dir / output_filename
    agg_result.to_excel(output_path, index=False, engine="openpyxl")
    print(f"✔ {company_name} 的中转数据计算完成，已保存至: {output_path.name}")


# --- 7. 主程序入口 ---
def main(period: str):
    if not BASE_CITY_PATTERN:
        print("由于城市列表未能加载，无法继续处理文件。")
        return

    logistics_files = list(logistics_dir.glob(f"*_{period}.xlsx")) + list(
        logistics_dir.glob(f"*_{period}.xls")
    )
    if not logistics_files:
        print(
            f"警告：在物流明细文件夹中未找到任何包含期数 '{period}' 的Excel文件！请检查路径: {logistics_dir}"
        )
        return

    # 新增修改: 从 period (例如 '202507') 中解析出年份和月份
    year = period[:4]
    month = int(period[4:])  # 使用 int() 去掉可能存在的前导零，例如 '07' -> 7

    for log_path in logistics_files:
        # 从物流文件名 (如 "京东_202507.xlsx") 中提取公司名 "京东"
        company_name_raw = log_path.stem
        company_name_for_search = company_name_raw.split("_")[0]

        print(f"\n正在为物流文件 [{log_path.name}] 寻找对应的安监数据...")

        # 新增修改: 根据解析出的年月和公司名，构建精确的安监文件名
        target_anjian_filename = f"{year}年{month}月{company_name_for_search}抽样.xlsx"
        anjian_path = anjian_dir / target_anjian_filename

        # 检查文件是否存在，如果不存在，则给出明确提示
        if not anjian_path.exists():
            # 为了兼容性，也检查一下 .xls 后缀
            target_anjian_filename_xls = target_anjian_filename.replace(".xlsx", ".xls")
            anjian_path_xls = anjian_dir / target_anjian_filename_xls
            if not anjian_path_xls.exists():
                print(
                    f"--> 警告：在安监文件夹中未找到精确匹配的文件 '{target_anjian_filename}' 或 '{target_anjian_filename_xls}'，跳过该公司。"
                )
                continue
            else:
                anjian_path = anjian_path_xls

        print(f"--> 成功匹配: [{anjian_path.name}]")
        process_company_data(log_path, anjian_path, period)

    print("\n--- 所有文件处理完毕！ ---")


# 在Jupyter环境中，period变量应已在Cell 0中定义
if "period" in locals() or "period" in globals():
    main(period)
else:
    print("错误：未找到 'period' 变量。请确保您已运行定义本期的第一个cell。")

基础数据文件应位于: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输入/basic_data.xlsx
安监数据文件夹: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输入/安监数据
物流明细文件夹: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/temp/3_猪猪云合并数据
计算结果将输出至: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/temp/5_中转数据
----------------------------------------
成功加载 297 个目标城市。

正在为物流文件 [申通_202506.xlsx] 寻找对应的安监数据...
--> 成功匹配: [2025年6月申通抽样.xlsx]

--- 正在处理: 申通 (数据期数: 202506) ---
在 申通_202506.xlsx 和 2025年6月申通抽样.xlsx 中共找到 83029 条匹配的运单。
✔ 申通 的中转数据计算完成，已保存至: 申通_transit_data_202506.xlsx

正在为物流文件 [圆通_202506.xlsx] 寻找对应的安监数据...
--> 成功匹配: [2025年6月圆通抽样.xlsx]

--- 正在处理: 圆通 (数据期数: 202506) ---
在 圆通_202506.xlsx 和 2025年6月圆通抽样.xlsx 中共找到 82733 条匹配的运单。
✔ 圆通 的中转数据计算完成，已保存至: 圆通_transit_data_202506.xlsx

正在为物流文件 [EMS_202506.xlsx] 寻找对应的安监数据...
--> 成功匹配: [2025年6月EMS抽样.xlsx]

--- 正在处理: EMS (数据期数: 202506) ---
在 EMS_202506.xlsx 和 2025年6月EMS抽样.xlsx 中共找到 43390 条匹配的运单。
✔ EMS 的中转数据计算完成，已保存至: EMS_trans

In [28]:
# ==============================================================================
# Cell 6: 核心数据计算层 (最终版)
# ==============================================================================
import sys
import warnings
from pathlib import Path
from time import perf_counter

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")
print("库导入完成。")


# --- 1. 运行模式开关 ---
RUN_MODE = "ALL"
TARGET_COMPANY = "邮政"

# --- 3. 项目路径设置 ---
base_path = Path.cwd()
report_path = base_path / "报告数据"
input_data_path = report_path / "temp" / "4_logistics数据"
zhuzhuyun_data_path = report_path / "temp" / "3_猪猪云合并数据"
transit_data_path = report_path / "temp" / "5_中转数据"
output_path = report_path / "输出" / "data_analysis_result"
basic_data_file = report_path / "输入" / "basic_data.xlsx"

# (新增) 公司名到文件名的映射，以及反向映射
COMPANY_TO_FILENAME_MAP = {
    "EMS": "EMS",
    "中通": "中通",
    "京东": "京东",
    "圆通": "圆通",
    "德邦": "德邦",
    "极兔": "极兔",
    "申通": "申通",
    "韵达": "韵达",
    "顺丰": "顺丰",
    "快包": "邮政",
}
FILENAME_TO_COMPANY_MAP = {v: k for k, v in COMPANY_TO_FILENAME_MAP.items()}


def load_top_cities(file_path: Path) -> set:
    sheet_name = "30_top_volume_city_2024"
    column_name = "城市"
    print(f"\n正在从 '{file_path.name}' 加载 Top 30 城市列表...")
    try:
        if not file_path.exists():
            raise FileNotFoundError(f"基础数据文件不存在: {file_path}")
        df = pd.read_excel(file_path, sheet_name=sheet_name, engine="openpyxl")
        if column_name not in df.columns:
            raise ValueError(
                f"在Sheet '{sheet_name}' 中未找到名为 '{column_name}' 的列。"
            )
        cities_set = set(df[column_name].dropna().astype(str).str.strip().tolist())
        if not cities_set:
            print(f"警告: 从 '{file_path.name}' 中加载的城市列表为空。")
        else:
            print(f"成功加载 {len(cities_set)} 个 Top 30 城市。")
        return cities_set
    except Exception as e:
        print(f"错误: 加载Top 30城市列表失败！错误信息: {e}")
        sys.exit(1)


def generate_basic_metrics_df(
    filtered_data: pd.DataFrame,
    original_data_count: int,
    top_30_cities: set,
    total_dropped_count: int,
) -> pd.DataFrame:
    metrics_data, distance_metrics = [], []
    total_count = filtered_data.shape[0]
    top_30_data_all = filtered_data[
        (filtered_data["寄出城市"].isin(top_30_cities))
        & (filtered_data["寄达城市"].isin(top_30_cities))
    ]
    total_top_30_count = top_30_data_all.shape[0]
    if not filtered_data.empty:
        time_cols_map = {
            "全程时限": "全程时限",
            "寄出地处理时限": "寄出地处理时限",
            "运输时限": "运输时限",
            "寄达地处理时限": "寄达地处理时限",
            "投递时限": "投递时限",
        }
        for name, col in time_cols_map.items():
            metrics_data.append(
                [
                    name,
                    filtered_data[col].mean(),
                    filtered_data[col].max(),
                    filtered_data[col].min(),
                ]
            )
        metrics_data.append(
            [
                "72小时准时率",
                (filtered_data["全程时限"] <= 72).mean() if total_count > 0 else 0,
                "",
                "",
            ]
        )
        metrics_data.append(
            [
                "48小时准时率",
                (top_30_data_all["全程时限"] <= 48).mean()
                if total_top_30_count > 0
                else 0,
                "",
                "",
            ]
        )
        metrics_data.append(
            [
                f"总数据量 {original_data_count}\n总筛选掉数据量 {total_dropped_count}\n最终有效数据量 {total_count}\n业务量前30的城市间数据量 {total_top_30_count}",
                "",
                "",
                "",
            ]
        )
        buckets = {
            "0-600": filtered_data[filtered_data["公里"] < 600],
            "600-1500": filtered_data[
                (filtered_data["公里"] >= 600) & (filtered_data["公里"] < 1500)
            ],
            "1500-2500": filtered_data[
                (filtered_data["公里"] >= 1500) & (filtered_data["公里"] < 2500)
            ],
            "2500以上": filtered_data[filtered_data["公里"] >= 2500],
        }
        for _, col_key in time_cols_map.items():
            distance_metrics.append([buckets[b][col_key].mean() for b in buckets])
        distance_metrics.extend([[""] * 4] * 3)
    else:
        metrics_data, distance_metrics = (
            ([["无有效数据", "", "", ""]] * 8),
            ([[""] * 4] * 8),
        )
    metrics_df = pd.DataFrame(metrics_data, columns=["项目", "mean", "max", "min"])
    distance_df = pd.DataFrame(
        distance_metrics, columns=["0-600", "600-1500", "1500-2500", "2500以上"]
    )
    return pd.concat([metrics_df, distance_df], axis=1).round(4).fillna("")


# 新增修改: 增加 period 参数
def main(period: str):
    for p in [
        report_path,
        input_data_path,
        zhuzhuyun_data_path,
        transit_data_path,
        output_path,
    ]:
        p.mkdir(exist_ok=True, parents=True)
    print("项目文件夹结构设置/检查完毕。")
    print(f"输入数据(Logistics数据)应位于: {input_data_path}")
    print(f"分析报告将输出至: {output_path}")

    top_30_cities = load_top_cities(basic_data_file)
    start_time = perf_counter()

    # 新增修改: 精准匹配本期的 logistics 数据文件
    files_to_process = list(input_data_path.glob(f"*_logistics_data_{period}.xlsx"))

    if RUN_MODE == "SINGLE":
        files_to_process = [
            f for f in files_to_process if f.name.startswith(TARGET_COMPANY)
        ]

    if not files_to_process:
        print(f"\n警告: 未找到任何包含期数 '{period}' 的待处理数据文件，程序即将退出。")
        sys.exit(0)

    summary_stats = []
    files_to_process = sorted(list(set(files_to_process)))
    print(f"\n找到 {len(files_to_process)} 个待处理文件 (期数: {period})，开始分析...")
    for file_path in files_to_process:
        # 新增修改: 将 period 传递给处理函数
        stats = process_single_file(file_path, top_30_cities, period)
        if stats:
            summary_stats.append(stats)

    end_time = perf_counter()
    print(f"\n\n所有文件处理完毕！总耗时: {end_time - start_time:.2f} 秒。")

    if summary_stats:
        print("\n" + "=" * 20 + " 数据筛选流程汇总统计 " + "=" * 20)
        summary_df = pd.DataFrame(summary_stats)
        summary_df["**总计删除**"] = (
            summary_df["因时间戳缺失删除"]
            + summary_df["因时限空值删除"]
            + summary_df["因范围不符删除"]
        )
        summary_df["最终保留"] = summary_df["原始数据量"] - summary_df["**总计删除**"]
        summary_df["**保留率**"] = (
            summary_df["最终保留"] / summary_df["原始数据量"]
        ).apply(lambda x: f"{x:.1%}")
        summary_df = summary_df[
            [
                "公司 (Company)",
                "原始数据量",
                "因时间戳缺失删除",
                "因时限空值删除",
                "因范围不符删除",
                "**总计删除**",
                "最终保留",
                "**保留率**",
            ]
        ]
        total_row = summary_df.select_dtypes(include=np.number).sum()
        total_row["公司 (Company)"] = "**总计**"
        total_row["**保留率**"] = (
            f"{(total_row['最终保留'] / total_row['原始数据量']):.1%}"
        )
        summary_df = pd.concat(
            [summary_df, pd.DataFrame(total_row).T], ignore_index=True
        )
        print(summary_df.to_markdown(index=False))


# 新增修改: 增加 period 参数
def process_single_file(file_path: Path, top_30_cities: set, period: str):
    file_name = file_path.name
    print(f"\n--- 正在处理文件: {file_name} ---")

    try:
        data = pd.read_excel(
            file_path, header=0, dtype={"单号": str}, engine="openpyxl"
        )
        original_data_count = data.shape[0]
        print(f"原始数据量: {original_data_count} 条")
    except Exception as e:
        print(f"读取文件 {file_name} 失败: {e}。跳过此文件。")
        return None

    core_required_cols = [
        "单号",
        "寄出省份",
        "寄出城市",
        "寄达省份",
        "寄达城市",
        "公里",
        "揽收时间",
        "离开寄件城市时间",
        "到达收件城市时间",
        "派送时间",
        "签收时间",
    ]
    optional_cols = ["到达分拣中心时间", "离开收件城市分拣中心时间"]
    if not all(c in data.columns for c in core_required_cols):
        print(f"错误: 文件 {file_name} 缺少必要的核心列。跳过此文件。")
        return None
    for col in optional_cols:
        if col not in data.columns:
            data[col] = pd.NaT

    # 新增修改: 修正猪猪云和中转数据的匹配逻辑
    # 从 "京东_logistics_data_202507.xlsx" 中提取出 "京东"
    base_company_name = file_name.split("_")[0]

    zhuzhuyun_file = zhuzhuyun_data_path / f"{base_company_name}_{period}.xlsx"
    if zhuzhuyun_file.exists():
        try:
            df_zhuzhu = pd.read_excel(
                zhuzhuyun_file,
                usecols=["快递单号", "完整物流信息"],
                dtype={"快递单号": str},
                engine="openpyxl",
            ).rename(columns={"快递单号": "单号"})
            data = pd.merge(
                data, df_zhuzhu.drop_duplicates(subset=["单号"]), on="单号", how="left"
            )
            if "完整物流信息_y" in data.columns:
                data["完整物流信息"] = data["完整物流信息_y"].fillna(
                    data["完整物流信息_x"]
                )
                data.drop(columns=["完整物流信息_x", "完整物流信息_y"], inplace=True)
        except Exception as e:
            print(f"  - 警告: 合并猪猪云数据失败: {e}")
    if "完整物流信息" not in data.columns:
        data["完整物流信息"] = ""

    all_time_cols = [c for c in core_required_cols + optional_cols if "时间" in c]
    for col in all_time_cols:
        data[col] = pd.to_datetime(data[col], errors="coerce")

    count_before_time_dropna = len(data)
    data.dropna(subset=[c for c in core_required_cols if "时间" in c], inplace=True)
    time_dropped_count = count_before_time_dropna - len(data)
    print(f"  - 因核心时间戳缺失，删除了 {time_dropped_count} 行。")

    data["全程时限"] = (data["签收时间"] - data["揽收时间"]) / np.timedelta64(1, "h")
    data["寄出地处理时限"] = (
        data["离开寄件城市时间"] - data["揽收时间"]
    ) / np.timedelta64(1, "h")
    data["寄达地处理时限"] = (
        data["派送时间"] - data["到达收件城市时间"]
    ) / np.timedelta64(1, "h")
    data["投递时限"] = (data["签收时间"] - data["派送时间"]) / np.timedelta64(1, "h")
    data["运输时限"] = (
        data["到达收件城市时间"] - data["离开寄件城市时间"]
    ) / np.timedelta64(1, "h")
    data["揽收-到达寄出地分拣中心时长"] = (
        data["到达分拣中心时间"] - data["揽收时间"]
    ) / np.timedelta64(1, "h")
    data["到达寄出地分拣中心-离开寄出地城市时长"] = (
        data["离开寄件城市时间"] - data["到达分拣中心时间"]
    ) / np.timedelta64(1, "h")
    data["到达寄达地城市-离开寄达地分拣中心时长"] = (
        data["离开收件城市分拣中心时间"] - data["到达收件城市时间"]
    ) / np.timedelta64(1, "h")
    data["离开寄达地分拣中心-派件"] = (
        data["派送时间"] - data["离开收件城市分拣中心时间"]
    ) / np.timedelta64(1, "h")

    company_key = FILENAME_TO_COMPANY_MAP.get(base_company_name, base_company_name)
    EXCLUDED_COMPANIES = {"中通", "顺丰"}
    count_before_filter = len(data)
    core_duration_cols = [
        "全程时限",
        "寄出地处理时限",
        "寄达地处理时限",
        "投递时限",
        "运输时限",
    ]
    all_duration_cols = core_duration_cols + [
        "揽收-到达寄出地分拣中心时长",
        "到达寄出地分拣中心-离开寄出地城市时长",
        "到达寄达地城市-离开寄达地分拣中心时长",
        "离开寄达地分拣中心-派件",
    ]
    columns_to_check_for_nan = (
        core_duration_cols
        if company_key in EXCLUDED_COMPANIES
        else [col for col in all_duration_cols if col in data.columns]
    )
    data.dropna(subset=columns_to_check_for_nan, inplace=True)
    nan_dropped_count = count_before_filter - len(data)
    print(f"    -> 因时限存在空值，删除了 {nan_dropped_count} 行。")

    base_mask = (
        data["寄出地处理时限"].between(0.1, 48)
        & data["运输时限"].between(0.5, 200)
        & data["寄达地处理时限"].between(0.1, 60)
        & data["投递时限"].between(0, 36)
    )
    if company_key not in EXCLUDED_COMPANIES:
        for col in [
            "揽收-到达寄出地分拣中心时长",
            "到达寄出地分拣中心-离开寄出地城市时长",
            "到达寄达地城市-离开寄达地分拣中心时长",
            "离开寄达地分拣中心-派件",
        ]:
            if col in data.columns:
                base_mask &= data[col] > 0
    filtered_data = data.loc[base_mask].copy()
    value_dropped_count = len(data) - len(filtered_data)
    print(f"    -> 因时限值不符合业务范围，又删除了 {value_dropped_count} 行。")

    total_dropped_count = time_dropped_count + nan_dropped_count + value_dropped_count
    if filtered_data.empty:
        print("筛选后无有效数据，无法生成报告。跳过此文件。")
        return None

    filtered_data["T+1_achieved"] = filtered_data["签收时间"].dt.normalize() <= (
        filtered_data["揽收时间"].dt.normalize() + pd.Timedelta(days=1)
    )
    filtered_data["T+2_achieved"] = filtered_data["签收时间"].dt.normalize() <= (
        filtered_data["揽收时间"].dt.normalize() + pd.Timedelta(days=2)
    )
    air_keyword = {"EMS": "飞往|发往航站准备安检", "快包": "飞往|发往航站准备安检"}.get(
        company_key, ""
    )
    filtered_data["is_air"] = (
        filtered_data["完整物流信息"].fillna("").str.contains(air_keyword, regex=True)
        if air_keyword
        else False
    )

    basic_metrics_df = generate_basic_metrics_df(
        filtered_data, original_data_count, top_30_cities, total_dropped_count
    )
    detailed_routes_df = filtered_data[
        [
            col
            for col in [
                "单号",
                "寄出省份",
                "寄出城市",
                "寄达省份",
                "寄达城市",
                "公里",
                "揽收时间",
                "离开寄件城市时间",
                "到达收件城市时间",
                "派送时间",
                "签收时间",
                "到达分拣中心时间",
                "离开收件城市分拣中心时间",
                "完整物流信息",
                "全程时限",
                "寄出地处理时限",
                "寄达地处理时限",
                "投递时限",
                "运输时限",
                "揽收-到达寄出地分拣中心时长",
                "到达寄出地分拣中心-离开寄出地城市时长",
                "到达寄达地城市-离开寄达地分拣中心时长",
                "离开寄达地分拣中心-派件",
                "T+1_achieved",
                "T+2_achieved",
                "is_air",
            ]
            if col in filtered_data.columns
        ]
    ].round(3)

    grouped = filtered_data.groupby(["寄出城市", "寄达城市"])
    agg_dict = {
        "快递数量": ("单号", "count"),
        "全程时限": ("全程时限", "mean"),
        "寄出地处理时限": ("寄出地处理时限", "mean"),
        "运输时限": ("运输时限", "mean"),
        "寄达地处理时限": ("寄达地处理时限", "mean"),
        "投递时限": ("投递时限", "mean"),
        "揽收-到达寄出地分拣中心时长": ("揽收-到达寄出地分拣中心时长", "mean"),
        "到达寄出地分拣中心-离开寄出地城市时长": (
            "到达寄出地分拣中心-离开寄出地城市时长",
            "mean",
        ),
        "到达寄达地城市-离开寄达地分拣中心时长": (
            "到达寄达地城市-离开寄达地分拣中心时长",
            "mean",
        ),
        "离开寄达地分拣中心-派件": ("离开寄达地分拣中心-派件", "mean"),
    }
    summary_df = grouped.agg(**agg_dict)
    total_counts = grouped.size()
    summary_df["72小时准时率"] = (
        filtered_data[filtered_data["全程时限"] <= 72]
        .groupby(["寄出城市", "寄达城市"])
        .size()
        / total_counts
    ).fillna(0)
    summary_df["48小时准时率"] = (
        filtered_data[filtered_data["全程时限"] <= 48]
        .groupby(["寄出城市", "寄达城市"])
        .size()
        / total_counts
    ).fillna(0)
    filtered_data["送达天数"] = (
        filtered_data["签收时间"].dt.normalize()
        - filtered_data["揽收时间"].dt.normalize()
    ).dt.days
    summary_df["送达天数_80分位"] = filtered_data.groupby(["寄出城市", "寄达城市"])[
        "送达天数"
    ].quantile(0.8, interpolation="higher")
    summary_df.reset_index(inplace=True)
    summary_df["路线"] = summary_df["寄出城市"] + "-" + summary_df["寄达城市"]

    transit_filename_key = "邮政" if company_key == "快包" else base_company_name
    # 新增修改: 匹配的中转文件名也需要包含 period
    transfer_file = (
        transit_data_path / f"{transit_filename_key}_transit_data_{period}.xlsx"
    )
    if transfer_file.exists():
        try:
            df_transfer = pd.read_excel(transfer_file, engine="openpyxl")
            df_transfer["路线"] = (
                df_transfer["出发城市"] + "-" + df_transfer["到达城市"]
            )
            df_agg = (
                df_transfer.groupby("路线")[["平均中转次数"]]
                .mean()
                .rename(columns={"平均中转次数": "中转次数"})
            )
            summary_df = pd.merge(summary_df, df_agg, on="路线", how="left")
        except Exception as e:
            print(f"   -> 警告: 处理中转文件 {transfer_file.name} 失败: {e}")
    else:
        print(f"   -> 提示: 未找到对应的中转文件: {transfer_file.name}")
    if "中转次数" not in summary_df.columns:
        summary_df["中转次数"] = np.nan

    summary_df = summary_df[
        [
            col
            for col in [
                "寄出城市",
                "寄达城市",
                "路线",
                "快递数量",
                "全程时限",
                "寄出地处理时限",
                "运输时限",
                "寄达地处理时限",
                "投递时限",
                "揽收-到达寄出地分拣中心时长",
                "到达寄出地分拣中心-离开寄出地城市时长",
                "到达寄达地城市-离开寄达地分拣中心时长",
                "离开寄达地分拣中心-派件",
                "72小时准时率",
                "48小时准时率",
                "送达天数_80分位",
                "中转次数",
            ]
            if col in summary_df.columns
        ]
    ].round(4)

    # 新增修改: 输出文件名中加入 period
    output_file_path = (
        output_path / f"{base_company_name}_data_analysis_result_{period}.xlsx"
    )
    with pd.ExcelWriter(output_file_path) as writer:
        basic_metrics_df.to_excel(writer, sheet_name="基础指标", index=False)
        detailed_routes_df.to_excel(writer, sheet_name="线路详细数据", index=False)
        summary_df.to_excel(writer, sheet_name="线路汇总数据", index=False)
    print(f"分析完成, 结果已保存至: {output_file_path.name}")

    stats_dict = {
        "公司 (Company)": company_key,
        "原始数据量": original_data_count,
        "因时间戳缺失删除": time_dropped_count,
        "因时限空值删除": nan_dropped_count,
        "因范围不符删除": value_dropped_count,
    }
    return stats_dict


# 在Jupyter环境中，period变量应已在Cell 0中定义
if "period" in locals() or "period" in globals():
    main(period)
else:
    print("错误：未找到 'period' 变量。请确保您已运行定义本期的第一个cell。")

库导入完成。
项目文件夹结构设置/检查完毕。
输入数据(Logistics数据)应位于: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/temp/4_logistics数据
分析报告将输出至: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输出/data_analysis_result

正在从 'basic_data.xlsx' 加载 Top 30 城市列表...
成功加载 30 个 Top 30 城市。

找到 10 个待处理文件 (期数: 202506)，开始分析...

--- 正在处理文件: EMS_logistics_data_202506.xlsx ---
原始数据量: 43390 条
  - 因核心时间戳缺失，删除了 58 行。
    -> 因时限存在空值，删除了 901 行。
    -> 因时限值不符合业务范围，又删除了 1625 行。
分析完成, 结果已保存至: EMS_data_analysis_result_202506.xlsx

--- 正在处理文件: 中通_logistics_data_202506.xlsx ---
原始数据量: 83160 条
  - 因核心时间戳缺失，删除了 0 行。
    -> 因时限存在空值，删除了 0 行。
    -> 因时限值不符合业务范围，又删除了 20372 行。
   -> 提示: 未找到对应的中转文件: 中通_transit_data_202506.xlsx
分析完成, 结果已保存至: 中通_data_analysis_result_202506.xlsx

--- 正在处理文件: 京东_logistics_data_202506.xlsx ---
原始数据量: 79688 条
  - 因核心时间戳缺失，删除了 14952 行。
    -> 因时限存在空值，删除了 3042 行。
    -> 因时限值不符合业务范围，又删除了 1359 行。
分析完成, 结果已保存至: 京东_data_analysis_result_202506.xlsx

--- 正在处理文件: 圆通_logistics_data_202506.xlsx ---
原